In [22]:

from ultralytics import YOLO, solutions
from mot_utilities import det_utilities
from mot_utilities import window_analysis

import matplotlib.pyplot as plt
import supervision as sv
import pandas as pd
import numpy as np
import math
import cv2

det_utils = det_utilities()
win_analysis = window_analysis()


### Previous Approach

Adjust the tracker, the default is botsort, if needed bytetrack - arg: tracker='bytetrack.yaml'

#### Utilities

In [2]:
def process_tracker_tensors(tensor):
    processed_list = []
    for i in range(29):  
        frame_list = []
        frame = i + 1

        # Check if 'xywh' is present and has items
        if tensor[i][0].boxes.xywh is not None and len(tensor[i][0].boxes.xywh) > 0:
            tensor_len = len(tensor[i][0].boxes.xywh)
            for j in range(tensor_len):
                # Check if 'id' is present and has sufficient items
                if tensor[i][0].boxes.id is not None:
                    bbox_id = int(tensor[i][0].boxes.id[j])  # ID of each bbox in frame 'i'
                    x_center, y_center, w, h = map(float, tensor[i][0].boxes.xywh[j][:])  # x, y, w, h
                    conf = float(tensor[i][0].boxes.conf[j]) 
                    x_min, y_min = x_center - (w / 2), y_center - (h / 2)
                    if y_center >= 660:
                        frame_list.append((frame, bbox_id, np.round(x_min, 4), np.round(y_min, 4),
                                           np.round(w, 4), np.round(h, 4), np.round(conf, 3), -1, -1, -1))
            # Append the frame_list only if it contains data
            if frame_list:
                processed_list.append(frame_list)

    return processed_list

def annotations_to_text(tensor_list, output_path):
    with open(output_path, 'w') as file:
        for frame_list in tensor_list:
            for annotation in frame_list:
                file.write(','.join(map(str, annotation)) + '\n')

In [3]:

def track_and_count_crops(ymin, ymax, xmin, xmax, model, video_source):

    base_path = 'C:/Users/ashis/Desktop/THESIS/DT_flow'
    weight = f'{base_path}/resources/weights/setA/{model}.pt'   # -- model = small, v9c etc
    vid = f'{base_path}/resources/data/{video_source}'    # -- video - source video with extension - 'wc_vid1.AVI', 'wc_vid2.mp4'
    outp_vid = f"{base_path}/det_models/files/sample_folder/roi/samp_seg_xy.avi"

    model = YOLO(weight)
    cap = cv2.VideoCapture(vid)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Video writer
    video_writer = cv2.VideoWriter(outp_vid, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # Init Object Counter
    counter = solutions.ObjectCounter(
        view_img=True,
        reg_pts=[(xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)],
        classes_names=model.names,
        draw_tracks=True,
        line_thickness=2,
    )

    tensorT = []

    try:
        while True:
            success, im0 = cap.read()
            if not success:
                print("Video frame is empty or video processing has been successfully completed.")
                break

            # Create a mask where only the ROI is visible
            mask = np.zeros_like(im0)
            mask[ymin:ymax, xmin:xmax] = im0[ymin:ymax, xmin:xmax]

            # Track using the masked image
            tracks = model.track(mask, persist=True, show=False, conf=.2, iou=.4, verbose=False)
            tensorT.append(tracks)

            im0 = counter.start_counting(im0, tracks)
            video_writer.write(im0)

        # Process tensors to retrieve structured data
        tensor_list = process_tracker_tensors(tensorT)

        if len(tensor_list) > 0:
            
            flat_list = [item for sublist in tensor_list for item in sublist]
            df = pd.DataFrame(flat_list)
            df = df[df.iloc[:,3] >= 660] 
            crop_count = df.iloc[:,1].nunique()
        
        else:
            crop_count=0
    finally:
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()

    return crop_count

def track_and_count_crops_r7(ymin, ymax, model, video_source, output_path, x1, x2, x3, x4):

    base_path = 'C:/Users/ashis/Desktop/THESIS/DT_flow'
    weight = f'{base_path}/resources/weights/setA/{model}.pt'   # -- model = small, v9c etc
    vid = f'{base_path}/resources/data/{video_source}'    # -- video - source video with extension - wc_vid1.AVI
    outp_vid = f"{base_path}/det_models/files/sample_folder/roi/samp_seg_x.avi"
    op_path=f'{base_path}/det_models/files/sample_folder/roi/{output_path}.txt'
    
    model = YOLO(weight)
    cap = cv2.VideoCapture(vid)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Video writer
    video_writer = cv2.VideoWriter(outp_vid, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # Init Object Counter
    counter = solutions.ObjectCounter(
        view_img=True,
        reg_pts=[(x1, ymin), (x2, ymin), (x3, 1400), (x4, ymax), (x1, ymax)],
        classes_names=model.names,
        draw_tracks=True,
        line_thickness=2,
    )
    
    tensorT = []

    try:
        while True:
            success, im0 = cap.read()
            if not success:
                print("Video frame is empty or video processing has been successfully completed.")
                break

            # Create masks where only the ROI is visible
            mask1 = np.zeros_like(im0)
            mask2 = np.zeros_like(im0)
            
            
            mask1[ymin:ymax, x1:x3] = im0[ymin:ymax, x1:x3]
            mask2[ymin:ymax, x3:x4] = im0[ymin:ymax, x3:x4]
            
            mask = mask1+mask2
            
            # Track using the masked image
            tracks = model.track(mask, persist=True, show=False, conf=.2, iou=.4, verbose=False)
            tensorT.append(tracks.cpu().numpy())

            im0 = counter.start_counting(im0, tracks)
            video_writer.write(im0)

        # Process tensors to retrieve structured data
        tensor_list = process_tracker_tensors(tensorT)
        flat_list = [item for sublist in tensor_list for item in sublist]
        df = pd.DataFrame(flat_list)
        df = df[((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3300) & (df.iloc[:,3] <= 2160) & (df.iloc[:,3] > 1500)) | ((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3500) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]
        
        if [df.iloc[:,3] >= 660]:
            
            crop_count = df.iloc[:,1].nunique()
        
        # annotations_to_text(tensor_list, op_path)
        
    finally:
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()

    return crop_count

# for the row8 with polygon window 

def track_and_count_crops_r8(ymin, ymax, model, video_source, output_path, x1, x2, x3, x4):

    base_path = 'C:/Users/ashis/Desktop/THESIS/DT_flow'
    weight = f'{base_path}/resources/weights/setA/{model}.pt'   # -- model = small, v9c etc
    vid = f'{base_path}/resources/data/{video_source}'    # -- video - source video with extension - wc_vid1.AVI
    outp_vid = f"{base_path}/det_models/files/sample_folder/roi/zsamp_seg_xx.avi"
    op_path=f'{base_path}/det_models/files/sample_folder/roi/{output_path}.txt'
    
    model = YOLO(weight)
    cap = cv2.VideoCapture(vid)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Video writer
    video_writer = cv2.VideoWriter(outp_vid, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # Init Object Counter
    counter = solutions.ObjectCounter(
        view_img=True,
        # reg_pts=[(x1, ymin), (x2, ymin), (x3, 1400), (x4, ymax), (x1, ymax)],
        reg_pts=[(x1, ymin), (x2, ymin), (x2, 1400), (x3, ymax), (x4, ymax), (x1, ymin)],
        
        classes_names=model.names,
        draw_tracks=True,
        line_thickness=2,
    )
    
    tensorT = []

    try:
        while True:
            success, im0 = cap.read()
            if not success:
                print("Video frame is empty or video processing has been successfully completed.")
                break

            mask1 = np.zeros_like(im0)  # Create masks where only the ROI is visible
            mask2 = np.zeros_like(im0)
            
            mask1[ymin:ymax, x2:x1] = im0[ymin:ymax, x2:x1]     # masking 
            mask2[ymin:ymax, x3:x2] = im0[ymin:ymax, x3:x2]
            
            mask = mask1 + mask2
            
            # Track using the masked image
            tracks = model.track(mask, persist=True, show=False, conf=.2, iou=.4, verbose=False)
            tensorT.append(tracks)

            im0 = counter.start_counting(im0, tracks)
            video_writer.write(im0)

        # Process tensors to retrieve structured data
        tensor_list = process_tracker_tensors(tensorT)
        flat_list = [item for sublist in tensor_list for item in sublist]
        df = pd.DataFrame(flat_list)
        
        df = df[((df.iloc[:,2] >= 3325) & (df.iloc[:,3] > 1500)) | ((df.iloc[:,2] >= 3493) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]
        
        if [df.iloc[:,3] >= 660]:            
            crop_count = df.iloc[:,1].nunique()
        
        # annotations_to_text(tensor_list, op_path)
        
    finally:
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()

    return crop_count

In [6]:
import torch
import gc

gc.collect()

torch.cuda.empty_cache()

In [1]:
import torch

torch.cuda.is_available()

True

In [ ]:
        # df = df[((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3300) & (df.iloc[:,3] <= 2160) & (df.iloc[:,3] > 1500)) | ((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3500) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]
        # df = df[((df.iloc[:,2] >= 3325) & (df.iloc[:,3] > 1500)) | ((df.iloc[:,2] >= 3493) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]




video2 - 
row7
[3065, ymin], [3440, ymin], [3440, 1400], [3326, ymax], [3065, ymax]

row8
[3768, ymin], [3500, ymin], [3500, 1400], [3325, ymax], [3768, ymax]

In [ ]:
# # -- row8

# count_in_row8 = []

# for ymin, ymax in zip(y1_new[:10], y2_new[:10]):
    
#     crops_count = track_and_count_crops(ymin, ymax, xmin = 3493, xmax = 3747, model='v9c', video_source='wc_vid1.AVI')
#     count_in_row8.append(crops_count)

# # -- row8 (remianing windows)

# count_in_row8_end = []

# for ymin, ymax in zip(y1_new[10:], y2_new[10:]):
    
#     crops_count = track_and_count_crops_r8(ymin, ymax, 'v9c', 'wc_vid1.AVI', 'empty', x1=3747, x2=3493, x3=3325, x4=3747)
#     count_in_row8_end.append(crops_count)

**TODO** 


2. Then create dataframe that save the accuracy for ach window and vertical field of view (vfov) and horizontal viewing angle (hva)

3. Then generate the plots

Note: Do not forget to run 2nd video as well

In [4]:

def count_crops_rows_1to6(y1_bounds, y2_bounds, xmin_list, xmax_list, model_name, video):

    results = {}
    
    for i in range(len(xmin_list)):
        
        count_in_row = []   
        
        for ymin, ymax in zip(y1_bounds, y2_bounds):
            crops_count = track_and_count_crops(ymin, ymax, xmin=xmin_list[i], xmax=xmax_list[i],
                                                model=model_name, video_source=video)
            count_in_row.append(crops_count)
        print(f'row{i+1}: {count_in_row}')
        
        results[f'row{i+1}'] = count_in_row
        
    return results


In [5]:

def count_crops_rows_78(y1_bounds, y2_bounds, xmin, xmax, limit, model_name, video):
    
    # -- here we will write a, if - else cond, where 20 is the threshold for the limit 
    # -- for row 7 - limit > 20 as the limit is 25
    # -- for row 8 - limit < 20 as the limit is 15

    if limit > 20:
        
        # -- logic for row 7
        
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            crops_count1 = track_and_count_crops(ymin, ymax, xmin=xmin, xmax=xmax, model=model_name, video_source=video)
            count_in_row.append(crops_count1)
            
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            crops_count2 = track_and_count_crops_r7(ymin, ymax, model_name, video, 'empty', x1=3090, x2=3453, x3=3453, x4=3300)
            count_in_row.append(crops_count2)
            
        return count_in_row
        
    else:
    
    # -- logic for row 8
    
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            crops_count1 = track_and_count_crops(ymin, ymax, xmin=xmin, xmax=xmax, model=model_name, video_source=video)
            count_in_row.append(crops_count1)
            
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            crops_count2 = track_and_count_crops_r8(ymin, ymax, model_name, video, 'empty', x1=3747, x2=3493, x3=3325, x4=3747)
            count_in_row.append(crops_count2)
            
        return count_in_row


def count_crops_rows_78B(y1_bounds, y2_bounds, xmin, xmax, limit, model_name, video):
    
    # -- here we will write a, if - else cond, where 20 is the threshold for the limit 
    # -- for row 7 - limit > 20 as the limit is 25
    # -- for row 8 - limit < 20 as the limit is 15

    if limit > 20:
        
        # -- logic for row 7
        
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            crops_count1 = track_and_count_crops(ymin, ymax, xmin=xmin, xmax=xmax, model=model_name, video_source=video)
            count_in_row.append(crops_count1)
            
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            crops_count2 = track_and_count_crops_r7(ymin, ymax, model_name, video, 'empty', x1=3065, x2=3440, x3=3440, x4=3326)
            count_in_row.append(crops_count2)
            
        return count_in_row
        
    else:
    
    # -- logic for row 8
    
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            crops_count1 = track_and_count_crops(ymin, ymax, xmin=xmin, xmax=xmax, model=model_name, video_source=video)
            count_in_row.append(crops_count1)
            
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            crops_count2 = track_and_count_crops_r8(ymin, ymax, model_name, video, 'empty', x1=3768, x2=3500, x3=3325, x4=3768)
            count_in_row.append(crops_count2)
            
        return count_in_row


In [6]:
# -- Counting accuracy related functions

def ret_gt_cropcount(df, y1, y2):
    
    trim_df = df[(df['CenterY'] > y1) & (df['CenterY'] < y2)]
    crop_count = trim_df['id'].nunique()
    
    return crop_count

def get_accy(gt_count, pred_count):
    
    error_rate = abs(pred_count - gt_count) / gt_count
    if error_rate > 1:
        return 0  # Returns 0% accuracy if error rate exceeds 100%
    else:
        accuracy = (1 - error_rate) * 100
        return round(accuracy, 2)

def process_gt_counts(row_data, y1_data, y2_data):      # row_data - a list of 'row' dataframes
    results = {}  # Dictionary to store the results for each row

    for row_idx in range(1, len(row_data) + 1):
        row_key = f'gt_row{row_idx}'
        gt_counts = []
        
        for i in range(len(y1)):  # Assuming each row has 25 entries
            gt_count = ret_gt_cropcount(row_data[row_idx - 1], y1_data[i], y2_data[i])
            gt_counts.append(gt_count)
        
        results[row_key] = gt_counts
    
    return results

def calculate_accuracies(gt_data, count_data):
    results = {}  # Dictionary to store the accuracy lists for each row

    for row_idx in range(1, len(gt_data) + 1):
        accuracies = []
        gt_row = gt_data[f'gt_row{row_idx}']
        count_row = count_data[f'row{row_idx}']

        for i in range(len(y1)):  # Assuming 25 measurements per row
            accuracy = get_accy(gt_row[i], count_row[i])
            accuracies.append(accuracy)
        
        results[f'accuracy{row_idx}'] = accuracies
    
    return results

define 'x' limits for video2 and name them as xmin_limits2 and xmax_limits2

In [7]:

xmin_limits1 = [350, 735, 1147, 1689, 2270, 2716, 3090, 3493]
xmax_limits1 = [736, 1089, 1540, 2089, 2533, 3048, 3453, 3747]

xmin_limits2 = [350, 725, 1134, 1674, 2236, 2678, 3140, 3476]
xmax_limits2 = [720, 1100, 1517, 1974, 2540, 3034, 3440, 3768]

y1 = [1045, 1010, 975, 940, 905, 870, 835, 800, 765, 730, 695, 660, 625, 590, 555, 520, 485, 450, 415, 380, 345, 
      310, 275, 240, 205, 170, 135, 100, 65, 30, 0]

y2 = [1115, 1150, 1185, 1220, 1255, 1290, 1325, 1360, 1395, 1430, 1465, 1500, 1535, 1570, 1605, 1640, 1675, 1710, 
      1745, 1780, 1815, 1850, 1885, 1920, 1955, 1990, 2025, 2060, 2095, 2130, 2160]

# xcenters1 = [543.0, 886.5, 1343.5, 1889.0, 2401.5, 2882.0, 3253.5, 3561.5] 
# xcenters2 = [487.0, 760.0, 1150.0, 1673.5, 2226.0, 2746.0, 3201.0, 3510.5] 

xcenters1 = [(a + b) / 2 for a, b in zip(xmin_limits1, xmax_limits1)]     # -- X centers of setA
xcenters2 = [(c + d) / 2 for c, d in zip(xmin_limits2, xmax_limits2)]   # -- X centers of setA

ycenters = [1080] * 8

horizontal_va1 = win_analysis.calculate_angles(xcenters1, ycenters)
horizontal_va2 = win_analysis.calculate_angles(xcenters2, ycenters)
vertical_fov = win_analysis.calculate_vertical_fov(y1, y2)

In [8]:
# -- Counting accuracy related functions

def ret_gt_cropcount(df, y1, y2):
    
    trim_df = df[(df['CenterY'] > y1) & (df['CenterY'] < y2)]
    crop_count = trim_df['id'].nunique()
    
    return crop_count

def get_accy(gt_count, pred_count):
    
    error_rate = abs(pred_count - gt_count) / gt_count
    if error_rate > 1:
        return 0  # Returns 0% accuracy if error rate exceeds 100%
    else:
        accuracy = (1 - error_rate) * 100
        return round(accuracy, 2)

def process_gt_counts(row_data, y1_data, y2_data):      # row_data - a list of 'row' dataframes
    results = {}  # Dictionary to store the results for each row

    for row_idx in range(1, len(row_data) + 1):
        row_key = f'gt_row{row_idx}'
        gt_counts = []
        
        for i in range(len(y1)):  # Assuming each row has 25 entries
            gt_count = ret_gt_cropcount(row_data[row_idx - 1], y1_data[i], y2_data[i])
            gt_counts.append(gt_count)
        
        results[row_key] = gt_counts
    
    return results

def calculate_accuracies(gt_data, count_data):
    results = {}  # Dictionary to store the accuracy lists for each row

    for row_idx in range(1, len(gt_data) + 1):
        accuracies = []
        gt_row = gt_data[f'gt_row{row_idx}']
        count_row = count_data[f'row{row_idx}']

        for i in range(len(y1)):  # Assuming 25 measurements per row
            accuracy = get_accy(gt_row[i], count_row[i])
            accuracies.append(accuracy)
        
        results[f'accuracy{row_idx}'] = accuracies
    
    return results

def gen_gt_results(model, type):

    gt_counts_results={}
    df = pd.read_csv(f'C:/Users/ashis/Desktop/THESIS/DT_flow/resources/dataframes{type}/{model}_gt.csv')  # -- type=A/B, model=small
    
    if type=='A': 
        row1, row2, row3, row4, row5, row6, row7, row8 = det_utils.rows_setA(df)
    else:
        row1, row2, row3, row4, row5, row6, row7, row8 = det_utils.rows_setB(df)
        
    gt_df = [row1, row2, row3, row4, row5, row6, row7, row8]
    gt_counts_results = process_gt_counts(gt_df, y1, y2)
    
    return gt_counts_results

def get_avg_accuracy(accuracy_results1, accuracy_results2):

    average_results = {}

    for key in accuracy_results1:
        list_a = accuracy_results1[key]
        list_b = accuracy_results2[key]
        average_list = [(a + b) / 2 for a, b in zip(list_a, list_b)]
        average_results[key] = average_list
    
    return average_results


In [202]:
def plot_accuracy(accuracy_results):
    
    threshold = 95  # Set the threshold for accuracy
    fig, axs = plt.subplots(1, 8, figsize=(20, 5))  # Create 1 row and 7 columns of subplots

    for i in range(8):
        accuracy_key = f'accuracy{i+1}'
        data_x = vertical_fov
        data_y = accuracy_results[accuracy_key]

        # Plot line segments individually
        for j in range(len(data_x) - 1):
            # Select color based on the value of the current point
            color = 'red' if data_y[j] >= threshold else 'green'
            axs[i].plot([data_x[j], data_x[j+1]], [data_y[j], data_y[j+1]], marker='o', markersize=3, color=color, linewidth=2)

        # Draw a horizontal line at the threshold value
        axs[i].axhline(y=threshold, color='red', linestyle='--', linewidth=1)

        axs[i].set_title(f'Row {i+1}')
        axs[i].set_ylim(0, 105)
        axs[i].grid()
        
    axs[3].set_xlabel('Vertical FOV')      
    axs[0].set_ylabel('Accuracy')  

    plt.tight_layout()
    plt.show()


In [9]:

def plot_accuracy9(accuracy_results_list, model_names, vertical_fov):
    
    threshold = 95  # Set the threshold for accuracy
    fig, axs = plt.subplots(1, 8, figsize=(20, 5))  # Create 1 row and 8 columns of subplots

    # Define specific colors for each model
    colors = ['orange', 'blue', 'cyan', 'magenta', 'green']

    if len(model_names) > len(colors):
        raise ValueError("Not enough unique colors defined for the number of models.")

    for i in range(8):
        for accuracy_results, model_name, color in zip(accuracy_results_list, model_names, colors):
            accuracy_key = f'accuracy{i+1}'
            data_x = vertical_fov
            data_y = accuracy_results[accuracy_key]

            # Plot line segments individually
            for j in range(len(data_x) - 1):
                segment_color = 'red' if data_y[j] >= threshold else color
                axs[i].plot([data_x[j], data_x[j+1]], [data_y[j], data_y[j+1]], marker='o', markersize=.7, color=segment_color, linewidth=0.3, label=model_name if j == 0 else "")

            # Draw a horizontal line at the threshold value
            axs[i].axhline(y=threshold, color='red', linestyle='--', linewidth=.5)

        axs[i].set_title(f'Row{i+1} ({horz_va1[i]}°, {horz_va2[i]}°)', fontsize=10)  # Set the font size here
        axs[i].set_ylim(0, 105)
        axs[i].grid()

    axs[3].set_xlabel('Vertical FOV (degrees°)')      
    axs[0].set_ylabel('Accuracy')  

    # Create a legend for the models and place it above the subplots
    handles = [plt.Line2D([0], [0], color=color, lw=1) for color in colors[:len(model_names)]]
    labels = model_names

    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=len(model_names))
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  # Adjust the top to make room for the legend
    plt.show()


#### botsort

In [ ]:
# -- vid1

nano1 = {'row1': [0, 1, 2, 8, 8, 9, 10, 10, 10, 11, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 16, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20], 'row2': [0, 1, 3, 10, 11, 12, 12, 12, 14, 13, 13, 13, 14, 13, 15, 15, 18, 16, 16, 16, 17, 18, 18, 17, 17, 18, 18, 19, 19, 19, 19], 'row3': [0, 2, 8, 17, 18, 18, 20, 20, 21, 21, 22, 24, 23, 24, 25, 24, 25, 25, 25, 24, 24, 25, 26, 27, 27, 28, 27, 27, 27, 28, 28], 'row4': [0, 4, 3, 17, 17, 19, 22, 23, 24, 23, 23, 24, 26, 26, 26, 25, 24, 25, 25, 26, 25, 26, 26, 26, 26, 26, 27, 28, 27, 27, 27], 'row5': [0, 1, 2, 10, 13, 13, 16, 16, 17, 16, 15, 17, 17, 18, 18, 19, 20, 19, 21, 21, 21, 21, 23, 23, 22, 22, 22, 22, 22, 22, 23], 'row6': [0, 2, 4, 14, 15, 16, 17, 17, 17, 18, 17, 20, 20, 21, 22, 21, 24, 23, 22, 22, 23, 24, 23, 23, 23, 23, 25, 25, 26, 26, 26], 'row7': [0, 1, 3, 11, 12, 12, 13, 15, 15, 15, 16, 17, 18, 18, 19, 20, 20, 19, 21, 18, 18, 20, 21, 21, 22, 20, 20, 22, 23, 23, 24], 'row8': [0, 1, 3, 5, 7, 7, 8, 9, 7, 9, 9, 10, 10, 10, 10, 11, 12, 11, 11, 11, 13, 12, 13, 12, 11, 11, 13, 13, 12, 12, 14]}
small1 = {'row1': [0, 1, 1, 7, 9, 10, 11, 11, 12, 13, 15, 13, 15, 16, 16, 17, 17, 16, 17, 16, 16, 16, 18, 18, 18, 19, 20, 20, 20, 22, 22], 'row2': [0, 1, 1, 7, 13, 13, 13, 15, 16, 16, 17, 18, 18, 18, 18, 19, 19, 19, 20, 19, 20, 21, 21, 20, 20, 21, 21, 22, 22, 23, 23], 'row3': [0, 3, 12, 21, 23, 24, 25, 25, 27, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 32, 32, 33, 33, 33, 34, 34, 34], 'row4': [0, 4, 6, 21, 23, 24, 25, 26, 28, 29, 29, 30, 32, 33, 33, 34, 34, 35, 34, 34, 35, 36, 36, 37, 38, 38, 38, 39, 40, 39, 39], 'row5': [0, 2, 4, 18, 20, 21, 22, 21, 21, 21, 21, 21, 23, 23, 23, 25, 25, 24, 26, 26, 26, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28], 'row6': [0, 1, 11, 20, 20, 22, 22, 22, 22, 24, 24, 24, 27, 27, 27, 27, 28, 27, 28, 28, 28, 29, 30, 30, 31, 31, 32, 32, 32, 32, 32], 'row7': [0, 1, 5, 16, 17, 19, 19, 22, 23, 24, 25, 25, 25, 27, 27, 28, 28, 30, 30, 30, 30, 29, 30, 31, 31, 31, 31, 31, 31, 32, 33], 'row8': [0, 1, 4, 8, 11, 11, 12, 15, 15, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 21, 21, 21, 21]}
medium1 = {'row1': [0, 1, 1, 5, 7, 9, 11, 12, 13, 13, 14, 15, 15, 15, 15, 17, 17, 17, 19, 20, 20, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23], 'row2': [0, 2, 2, 7, 10, 12, 12, 14, 16, 16, 17, 17, 18, 19, 21, 20, 21, 22, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 25, 26, 26], 'row3': [0, 5, 12, 21, 23, 24, 25, 25, 27, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 32, 32, 32, 32, 34, 34, 34, 34, 34, 34], 'row4': [0, 3, 9, 21, 25, 27, 28, 28, 29, 29, 30, 33, 34, 34, 34, 35, 36, 36, 36, 36, 37, 37, 38, 38, 38, 40, 40, 41, 42, 41, 42], 'row5': [0, 2, 5, 19, 21, 22, 22, 22, 22, 24, 24, 25, 26, 26, 26, 28, 28, 28, 28, 29, 29, 30, 31, 31, 31, 31, 31, 31, 31, 32, 31], 'row6': [0, 2, 11, 21, 20, 21, 22, 22, 24, 24, 24, 24, 26, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 33, 33, 33, 34, 33], 'row7': [0, 1, 3, 15, 17, 21, 21, 23, 25, 27, 28, 28, 28, 29, 30, 29, 31, 30, 31, 32, 32, 32, 33, 34, 35, 35, 34, 34, 33, 34, 34], 'row8': [0, 1, 3, 5, 7, 6, 6, 9, 8, 10, 12, 13, 14, 13, 14, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 22, 22, 22, 22]}
large1 = {'row1': [0, 1, 1, 9, 10, 14, 13, 12, 13, 14, 16, 16, 16, 15, 16, 18, 18, 18, 19, 19, 19, 20, 19, 19, 19, 20, 20, 20, 21, 21, 21], 'row2': [0, 0, 3, 8, 14, 15, 16, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 21, 22, 22, 22, 22, 22, 23, 23, 23, 25, 24, 24], 'row3': [0, 2, 12, 22, 23, 23, 24, 24, 26, 27, 27, 27, 27, 28, 28, 29, 29, 29, 29, 30, 30, 30, 31, 33, 33, 34, 34, 34, 34, 34, 35], 'row4': [0, 4, 9, 23, 23, 26, 27, 27, 29, 30, 29, 32, 34, 34, 34, 35, 36, 36, 36, 37, 37, 38, 38, 39, 39, 39, 40, 41, 41, 41, 43], 'row5': [0, 2, 3, 18, 20, 21, 22, 22, 22, 22, 24, 25, 26, 26, 26, 28, 28, 28, 28, 28, 28, 29, 31, 31, 31, 31, 31, 31, 31, 31, 32], 'row6': [0, 2, 12, 21, 20, 21, 22, 22, 23, 25, 26, 27, 27, 30, 30, 30, 30, 31, 30, 30, 31, 31, 31, 31, 32, 33, 34, 34, 34, 34, 34], 'row7': [0, 2, 5, 18, 17, 21, 21, 23, 22, 24, 25, 25, 24, 26, 27, 27, 27, 28, 28, 28, 29, 29, 30, 30, 33, 33, 33, 34, 34, 34, 34], 'row8': [0, 1, 3, 9, 11, 12, 13, 14, 14, 14, 17, 17, 18, 18, 17, 19, 18, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 22, 22, 22, 22]}
v9c1 = {'row1': [0, 0, 3, 11, 11, 13, 14, 14, 15, 15, 17, 17, 17, 17, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 23, 23, 23, 24, 25], 'row2': [0, 3, 4, 13, 16, 15, 19, 19, 21, 21, 21, 21, 22, 24, 24, 24, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 29, 29, 30, 30], 'row3': [0, 3, 13, 23, 24, 25, 26, 27, 28, 30, 31, 30, 31, 31, 33, 33, 33, 33, 33, 33, 33, 35, 36, 36, 37, 37, 37, 37, 38, 37, 38], 'row4': [0, 3, 9, 23, 26, 27, 27, 28, 30, 30, 31, 33, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 42, 42, 43, 43], 'row5': [0, 3, 5, 20, 21, 22, 22, 22, 23, 23, 23, 24, 25, 26, 27, 28, 28, 28, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 33, 33, 33], 'row6': [0, 2, 13, 21, 20, 22, 22, 23, 24, 25, 25, 26, 28, 29, 28, 30, 30, 30, 30, 30, 31, 31, 31, 32, 32, 33, 33, 34, 34, 34, 34], 'row7': [0, 2, 10, 20, 20, 21, 21, 23, 23, 25, 25, 26, 26, 28, 28, 28, 29, 29, 29, 29, 29, 31, 32, 32, 32, 33, 35, 35, 36, 36, 35], 'row8': [0, 2, 5, 9, 12, 12, 14, 15, 15, 15, 15, 15, 16, 17, 17, 19, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 23, 23, 23, 23, 22]}

# -- vid2

nano2 = {'row1': [0, 1, 1, 2, 2, 2, 4, 5, 6, 5, 4, 6, 6, 7, 9, 9, 11, 11, 12, 13, 13, 14, 14, 15, 15, 17, 17, 18, 19, 20, 20], 'row2': [0, 0, 1, 0, 1, 1, 2, 3, 6, 5, 8, 8, 7, 7, 6, 7, 7, 9, 9, 9, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 10], 'row3': [0, 2, 3, 5, 7, 11, 12, 14, 18, 19, 20, 20, 21, 22, 25, 26, 24, 23, 24, 26, 26, 24, 26, 27, 29, 29, 27, 26, 27, 27, 27], 'row4': [0, 2, 3, 5, 7, 12, 8, 10, 10, 13, 11, 13, 13, 16, 16, 13, 15, 18, 19, 19, 20, 19, 17, 18, 19, 18, 17, 18, 19, 20, 20], 'row5': [0, 1, 3, 4, 4, 7, 7, 10, 6, 11, 11, 10, 12, 13, 11, 12, 13, 13, 14, 15, 13, 16, 16, 17, 18, 19, 18, 18, 18, 18, 18], 'row6': [0, 1, 1, 3, 4, 6, 6, 7, 7, 9, 10, 10, 11, 13, 12, 12, 13, 13, 13, 16, 15, 15, 17, 17, 20, 21, 21, 21, 22, 22, 23], 'row7': [0, 0, 1, 3, 3, 4, 6, 6, 5, 6, 8, 7, 6, 6, 6, 6, 6, 6, 6, 6, 7, 10, 11, 11, 10, 10, 10, 10, 10, 10, 10], 'row8': [0, 1, 1, 4, 6, 6, 6, 9, 7, 10, 11, 10, 11, 10, 11, 10, 10, 14, 14, 14, 14, 14, 13, 14, 15, 15, 14, 13, 13, 12, 12]}
small2 = {'row1': [0, 1, 2, 10, 12, 11, 13, 14, 15, 14, 16, 16, 17, 17, 17, 18, 19, 18, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22], 'row2': [0, 0, 0, 1, 6, 8, 9, 10, 10, 12, 13, 13, 14, 14, 15, 16, 15, 15, 15, 14, 14, 15, 15, 15, 16, 18, 18, 17, 17, 18, 18], 'row3': [0, 3, 3, 22, 26, 25, 25, 28, 30, 30, 30, 31, 33, 33, 32, 33, 33, 33, 33, 35, 34, 34, 35, 35, 36, 36, 36, 36, 38, 38, 37], 'row4': [0, 2, 2, 17, 20, 21, 21, 22, 22, 22, 22, 24, 24, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 27, 28, 28, 29, 30], 'row5': [0, 1, 7, 20, 22, 23, 23, 24, 25, 25, 26, 26, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 31, 32, 33, 32, 32, 32, 32, 33], 'row6': [0, 1, 3, 18, 22, 21, 23, 24, 25, 26, 26, 28, 29, 30, 29, 31, 31, 31, 32, 32, 31, 31, 32, 32, 32, 33, 33, 34, 34, 34, 34], 'row7': [0, 0, 3, 9, 13, 13, 16, 15, 16, 16, 17, 17, 17, 17, 17, 17, 17, 19, 19, 20, 19, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23], 'row8': [0, 0, 2, 9, 11, 14, 15, 15, 14, 16, 16, 18, 17, 19, 19, 20, 20, 21, 21, 21, 21, 22, 21, 22, 22, 22, 23, 23, 23, 23, 23]}
# medium2 = {'row1': [0, 1, 1, 9, 10, 13, 13, 13, 13, 13, 16, 18, 18, 20, 20, 21, 21, 21, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 26, 26, 26], 'row2': [0, 1, 0, 2, 5, 8, 11, 11, 14, 14, 15, 18, 19, 19, 19, 19, 19, 19, 19, 19, 21, 21, 21, 21, 22, 21, 22, 23, 23, 22, 23], 'row3': [0, 2, 4, 20, 23, 24, 25, 28, 30, 29, 31, 32, 33, 32, 32, 33, 33, 34, 37, 37, 37, 37, 38, 38, 38, 38, 38, 39, 40, 40, 41],  }
large2 = {'row1': [0, 1, 2, 13, 15, 15, 16, 16, 16, 16, 17, 18, 17, 18, 19, 20, 20, 21, 21, 22, 22, 22, 23, 23, 25, 24, 24, 24, 26, 25, 25], 'row2': [0, 0, 2, 7, 9, 9, 12, 14, 14, 15, 16, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 22, 22], 'row3': [1, 2, 3, 18, 24, 23, 25, 26, 29, 29, 30, 31, 31, 32, 32, 33, 33, 35, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 37, 37, 38], 'row4': [1, 1, 5, 17, 20, 20, 20, 20, 22, 22, 22, 23, 24, 24, 24, 24, 25, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 28, 28, 29, 29], 'row5': [1, 1, 8, 20, 22, 23, 24, 25, 25, 25, 26, 26, 28, 26, 27, 27, 27, 28, 28, 28, 28, 29, 30, 30, 30, 32, 32, 33, 33, 33, 34], 'row6': [0, 2, 7, 16, 23, 22, 23, 25, 26, 26, 27, 27, 29, 29, 30, 31, 31, 30, 30, 30, 30, 31, 32, 32, 33, 33, 33, 34, 35, 35, 35], 'row7': [0, 0, 5, 13, 17, 15, 17, 16, 17, 17, 19, 19, 21, 21, 21, 21, 21, 22, 23, 23, 23, 24, 24, 24, 24, 24, 25, 25, 25, 26, 26], 'row8': [0, 1, 5, 10, 12, 14, 15, 16, 16, 16, 18, 18, 18, 20, 20, 20, 20, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 24, 26]}
v9c2 = {}



In [24]:
print()

{'row1': [0, 1, 1, 9, 10, 14, 13, 12, 13, 14, 16, 16, 16, 15, 16, 18, 18, 18, 19, 19, 19, 20, 19, 19, 19, 20, 20, 20, 21, 21, 21], 'row2': [0, 0, 3, 8, 14, 15, 16, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 21, 22, 22, 22, 22, 22, 23, 23, 23, 25, 24, 24], 'row3': [0, 2, 12, 22, 23, 23, 24, 24, 26, 27, 27, 27, 27, 28, 28, 29, 29, 29, 29, 30, 30, 30, 31, 33, 33, 34, 34, 34, 34, 34, 35], 'row4': [0, 4, 9, 23, 23, 26, 27, 27, 29, 30, 29, 32, 34, 34, 34, 35, 36, 36, 36, 37, 37, 38, 38, 39, 39, 39, 40, 41, 41, 41, 43], 'row5': [0, 2, 3, 18, 20, 21, 22, 22, 22, 22, 24, 25, 26, 26, 26, 28, 28, 28, 28, 28, 28, 29, 31, 31, 31, 31, 31, 31, 31, 31, 32], 'row6': [0, 2, 12, 21, 20, 21, 22, 22, 23, 25, 26, 27, 27, 30, 30, 30, 30, 31, 30, 30, 31, 31, 31, 31, 32, 33, 34, 34, 34, 34, 34], 'row7': [0, 2, 5, 18, 17, 21, 21, 23, 22, 24, 25, 25, 24, 26, 27, 27, 27, 28, 28, 28, 29, 29, 30, 30, 33, 33, 33, 34, 34, 34, 34], 'row8': [0, 1, 3, 9, 11, 12, 13, 14, 14, 14, 17, 17, 18, 18, 17, 19, 18, 19, 20, 2

In [10]:
# -- small 

small1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], 'small', 'wc_vid1.AVI')
row7_small1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, 'small', 'wc_vid1.AVI') # -- row7
row8_small1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, 'small', 'wc_vid1.AVI') # -- row8
small1['row7'] = row7_small1 #  -- row7 and ro8 count data to the result dictionary
small1['row8'] = row8_small1
print(small1['row7'])
print(small1['row8'])
print()

# -- medium 

medium1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], 'medium', 'wc_vid1.AVI')
row7_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, 'medium', 'wc_vid1.AVI') # -- row7
row8_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, 'medium', 'wc_vid1.AVI') # -- row8
medium1['row7'] = row7_medium1 #  -- row7 and ro8 count data to the result dictionary
medium1['row8'] = row8_medium1
print(medium1['row7'])
print(medium1['row8'])
print()

Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [23]:
print(small1)
print(medium1)

{'row1': [0, 1, 1, 7, 9, 10, 11, 11, 12, 13, 15, 13, 15, 16, 16, 17, 17, 16, 17, 16, 16, 16, 18, 18, 18, 19, 20, 20, 20, 22, 22], 'row2': [0, 1, 1, 7, 13, 13, 13, 15, 16, 16, 17, 18, 18, 18, 18, 19, 19, 19, 20, 19, 20, 21, 21, 20, 20, 21, 21, 22, 22, 23, 23], 'row3': [0, 3, 12, 21, 23, 24, 25, 25, 27, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 32, 32, 33, 33, 33, 34, 34, 34], 'row4': [0, 4, 6, 21, 23, 24, 25, 26, 28, 29, 29, 30, 32, 33, 33, 34, 34, 35, 34, 34, 35, 36, 36, 37, 38, 38, 38, 39, 40, 39, 39], 'row5': [0, 2, 4, 18, 20, 21, 22, 21, 21, 21, 21, 21, 23, 23, 23, 25, 25, 24, 26, 26, 26, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28], 'row6': [0, 1, 11, 20, 20, 22, 22, 22, 22, 24, 24, 24, 27, 27, 27, 27, 28, 27, 28, 28, 28, 29, 30, 30, 31, 31, 32, 32, 32, 32, 32], 'row7': [0, 1, 5, 16, 17, 19, 19, 22, 23, 24, 25, 25, 25, 27, 27, 28, 28, 30, 30, 30, 30, 29, 30, 31, 31, 31, 31, 31, 31, 32, 33], 'row8': [0, 1, 4, 8, 11, 11, 12, 15, 15, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18

In [12]:
# -- video1

large1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], 'large', 'wc_vid1.AVI')
row7_large1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, 'large', 'wc_vid1.AVI') # -- row7
row8_large1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, 'large', 'wc_vid1.AVI') # -- row8
large1['row7'] = row7_large1 #  -- row7 and row8 count data to the result dictionary
large1['row8'] = row8_large1
print(large1['row7'])
print(large1['row8'])
print()
print(large1)

Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [13]:
# -- video2

# -- small
small2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], 'small', 'wc_vid2.mp4')
row7_small2 = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'small', 'wc_vid2.mp4')
row8_small2 = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'small', 'wc_vid2.mp4')
small2['row7'] = row7_small2
small2['row8'] = row8_small2
print(small2['row7'])
print(small2['row8'])
print()

# -- large
large2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], 'large', 'wc_vid2.mp4')
row7_large2 = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'large', 'wc_vid2.mp4')
row8_large2 = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'large', 'wc_vid2.mp4')
large2['row7'] = row7_large2
large2['row8'] = row8_large2
print(large2['row7'])
print(large2['row8'])
print()


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [14]:
print(small2)
print(large2)


{'row1': [0, 1, 2, 10, 12, 11, 13, 14, 15, 14, 16, 16, 17, 17, 17, 18, 19, 18, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22], 'row2': [0, 0, 0, 1, 6, 8, 9, 10, 10, 12, 13, 13, 14, 14, 15, 16, 15, 15, 15, 14, 14, 15, 15, 15, 16, 18, 18, 17, 17, 18, 18], 'row3': [0, 3, 3, 22, 26, 25, 25, 28, 30, 30, 30, 31, 33, 33, 32, 33, 33, 33, 33, 35, 34, 34, 35, 35, 36, 36, 36, 36, 38, 38, 37], 'row4': [0, 2, 2, 17, 20, 21, 21, 22, 22, 22, 22, 24, 24, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 27, 28, 28, 29, 30], 'row5': [0, 1, 7, 20, 22, 23, 23, 24, 25, 25, 26, 26, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 31, 32, 33, 32, 32, 32, 32, 33], 'row6': [0, 1, 3, 18, 22, 21, 23, 24, 25, 26, 26, 28, 29, 30, 29, 31, 31, 31, 32, 32, 31, 31, 32, 32, 32, 33, 33, 34, 34, 34, 34], 'row7': [0, 0, 3, 9, 13, 13, 16, 15, 16, 16, 17, 17, 17, 17, 17, 17, 17, 19, 19, 20, 19, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23], 'row8': [0, 0, 2, 9, 11, 14, 15, 15, 14, 16, 16, 18, 17, 19, 19, 20, 20, 21, 21, 21, 21

In [ ]:
# -- medium rows 1,2,3 - yet to get the results fot (4,5,6,7,8)

medium2 = {'row1': [0, 1, 1, 9, 10, 13, 13, 13, 13, 13, 16, 18, 18, 20, 20, 21, 21, 21, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 26, 26, 26], 'row2': [0, 1, 0, 2, 5, 8, 11, 11, 14, 14, 15, 18, 19, 19, 19, 19, 19, 19, 19, 19, 21, 21, 21, 21, 22, 21, 22, 23, 23, 22, 23], 'row3': [0, 2, 4, 20, 23, 24, 25, 28, 30, 29, 31, 32, 33, 32, 32, 33, 33, 34, 37, 37, 37, 37, 38, 38, 38, 38, 38, 39, 40, 40, 41],  }


In [15]:
# -- medium
medium2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], 'medium', 'wc_vid2.mp4')
row7_medium2 = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'medium', 'wc_vid2.mp4')
row8_medium2 = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'medium', 'wc_vid2.mp4')
medium2['row7'] = row7_medium2
medium2['row8'] = row8_medium2
print(medium2['row7'])
print(medium2['row8'])
print()


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

KeyboardInterrupt: 

In [ ]:
medium2 = count_crops_rows_1to6(y1, y2, xmin_limits2[3:6], xmax_limits2[3:6], 'medium', 'wc_vid2.mp4')
row7_medium2 = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'medium', 'wc_vid2.mp4')
row8_medium2 = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'medium', 'wc_vid2.mp4')
medium2['row7'] = row7_medium2
medium2['row8'] = row8_medium2
print(medium2['row7'])
print(medium2['row8'])
print()
print(medium2)

In [ ]:
# row 1,2,3, done 
# start from row 4 - 8

In [ ]:
# -- v9c
v9c2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], 'v9c', 'wc_vid2.mp4')
row7_v9c2 = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'v9c', 'wc_vid2.mp4')
row8_v9c2 = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'v9c', 'wc_vid2.mp4')
v9c2['row7'] = row7_v9c2
v9c2['row8'] = row8_v9c2
print(v9c2['row7'])
print(v9c2['row8'])
print()

In [ ]:
print(small2)
print(medium2)
print(large2)
print(v9c2)

calculate average accuracies for both videos by using the following approach.

In [ ]:
# -- Accuracies

# -- nano
gt_nano1 = gen_gt_results('nano', 'A')
accuracy_results_nano1 = calculate_accuracies(gt_nano1, nano1) 

gt_nano2 = gen_gt_results('nano', 'B')
accuracy_results_nano2 = calculate_accuracies(gt_nano2, nano2) 

nano_accuracies = get_avg_accuracy(accuracy_results_nano1, accuracy_results_nano2)

# -- small
gt_small1 = gen_gt_results('nano', 'A')
accuracy_results_small1 = calculate_accuracies(gt_small1, small1) 

gt_small2 = gen_gt_results('nano', 'B')
accuracy_results_small2 = calculate_accuracies(gt_small2, small2) 

small_accuracies = get_avg_accuracy(accuracy_results_small1, accuracy_results_small2)

#-- medium
gt_medium1 = gen_gt_results('nano', 'A')
accuracy_results_medium1 = calculate_accuracies(gt_medium1, medium1) 

gt_medium2 = gen_gt_results('nano', 'B')
accuracy_results_medium2 = calculate_accuracies(gt_medium2, medium2) 

medium_accuracies = get_avg_accuracy(accuracy_results_medium1, accuracy_results_medium2)

# -- large
gt_large1 = gen_gt_results('nano', 'A')
accuracy_results_large1 = calculate_accuracies(gt_large1, large1) 

gt_large2 = gen_gt_results('nano', 'B')
accuracy_results_large2 = calculate_accuracies(gt_large2, large2) 

large_accuracies = get_avg_accuracy(accuracy_results_large1, accuracy_results_large2)

# -- v9c
gt_v9c1 = gen_gt_results('nano', 'A')
accuracy_results_v9c1 = calculate_accuracies(gt_v9c1, v9c1) 

gt_v9c2 = gen_gt_results('nano', 'B')
accuracy_results_v9c2 = calculate_accuracies(gt_v9c2, v9c2) 

v9c_accuracies = get_avg_accuracy(accuracy_results_v9c1, accuracy_results_v9c2)


In [ ]:
# --

accuracy_results_list = [nano_accuracies, small_accuracies, medium_accuracies, v9c_accuracies, large_accuracies]
model_names = ['nano', 'small', 'medium', 'v9c', 'large']

plot_accuracy9(accuracy_results_list, model_names, vertical_fov)

#### YOLOv8-Nano & ByteTrack

In [96]:
# -- video1

# -- processing of rows from 1-6

result_dict = count_crops_rows_1to6(y1, y2, xmin_limits[:6], xmax_limits[:6], 'nano', 'wc_vid1.AVI')

# -- row7

results_row7 = count_crops_rows_78(y1, y2, xmin_limits[6], xmax_limits[6], 25, 'nano', 'wc_vid1.AVI')

# -- row8

results_row8 = count_crops_rows_78(y1, y2, xmin_limits[7], xmax_limits[7], 15, 'nano', 'wc_vid1.AVI')

#  -- row7 and ro8 count data to the 'result_dict'

result_dict['row7'] = results_row7
result_dict['row8'] = results_row8

Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [98]:
print(result_dict)

{'row1': [0, 1, 1, 4, 6, 7, 8, 8, 9, 9, 10, 13, 11, 13, 13, 14, 11, 12, 12, 10, 10, 12, 12, 14, 14, 15, 14, 14, 14, 15, 15], 'row2': [0, 1, 4, 4, 7, 9, 9, 9, 9, 10, 10, 10, 11, 10, 12, 12, 15, 14, 14, 14, 18, 17, 16, 16, 12, 16, 13, 17, 18, 19, 17], 'row3': [0, 2, 2, 7, 5, 9, 8, 11, 14, 13, 16, 13, 12, 12, 13, 13, 16, 14, 15, 15, 16, 19, 18, 19, 19, 19, 20, 19, 19, 21, 20], 'row4': [0, 3, 2, 4, 5, 7, 10, 10, 9, 10, 13, 12, 13, 12, 12, 12, 14, 14, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 16, 16, 17], 'row5': [0, 1, 1, 2, 2, 4, 4, 4, 4, 5, 5, 6, 7, 6, 8, 9, 8, 9, 10, 11, 11, 11, 13, 14, 13, 13, 15, 18, 17, 17, 18], 'row6': [0, 3, 2, 5, 4, 7, 7, 8, 7, 9, 11, 8, 9, 11, 13, 14, 18, 16, 17, 17, 18, 18, 18, 19, 20, 18, 18, 20, 20, 19, 19], 'row7': [0, 1, 1, 4, 4, 5, 4, 5, 7, 8, 8, 8, 10, 11, 13, 13, 13, 11, 13, 11, 10, 11, 11, 13, 14, 12, 12, 14, 14, 15, 16], 'row8': [0, 1, 2, 4, 5, 6, 5, 7, 7, 8, 8, 8, 9, 9, 8, 7, 9, 9, 8, 8, 9, 9, 9, 10, 11, 11, 11, 12, 11, 11, 11]}


Adjust the 'x' limits for video2

In [ ]:
# -- video2

# -- processing of rows from 1-6

result_dict = count_crops_rows_1to6(y1, y2, xmin_limits[:6], xmax_limits[:6], 'nano', 'wc_vid1.AVI')

# -- row7

results_row7 = count_crops_rows_78(y1, y2, xmin_limits[6], xmax_limits[6], 25, 'nano', 'wc_vid1.AVI')

# -- row8

results_row8 = count_crops_rows_78(y1, y2, xmin_limits[7], xmax_limits[7], 15, 'nano', 'wc_vid1.AVI')

#  -- row7 and ro8 count data to the 'result_dict'

result_dict['row7'] = results_row7
result_dict['row8'] = results_row8

0. Process the counts for the rows with the 'nano' weights and 'bytetrack' tracker
1. Done with the crop counting (processing) code, now onto the accuracy processing code
2. After v8 nano-bytetrack try v10 nano as well

In [102]:
# -- get the accuracies for all the rows

# gt_data - must be a dictionary with keys names as gt_row1,.. gt_row8
# count_data - must be a dictionary with keys names as row1,.. row8


accuracy_results = calculate_accuracies(gt_counts_results, result_dict) 


In [117]:
print(accuracy_results)

{'accuracy1': [0.0, 6.67, 5.88, 22.22, 31.58, 36.84, 40.0, 38.1, 40.91, 37.5, 41.67, 52.0, 44.0, 50.0, 50.0, 51.85, 40.74, 44.44, 44.44, 35.71, 33.33, 40.0, 40.0, 45.16, 43.75, 45.45, 42.42, 41.18, 41.18, 42.86, 42.86], 'accuracy2': [0.0, 5.56, 22.22, 21.05, 36.84, 45.0, 42.86, 40.91, 39.13, 41.67, 41.67, 41.67, 45.83, 40.0, 46.15, 46.15, 55.56, 51.85, 51.85, 50.0, 64.29, 60.71, 57.14, 55.17, 41.38, 55.17, 44.83, 56.67, 60.0, 63.33, 56.67], 'accuracy3': [0.0, 8.7, 8.33, 29.17, 19.23, 33.33, 29.63, 36.67, 46.67, 41.94, 50.0, 40.62, 37.5, 36.36, 39.39, 39.39, 48.48, 42.42, 45.45, 45.45, 45.71, 54.29, 50.0, 52.78, 51.35, 51.35, 52.63, 50.0, 48.72, 53.85, 51.28], 'accuracy4': [0.0, 11.54, 7.69, 14.81, 18.52, 25.93, 34.48, 34.48, 29.03, 31.25, 38.24, 35.29, 37.14, 33.33, 33.33, 32.43, 37.84, 37.84, 39.47, 38.46, 38.46, 41.03, 40.0, 39.02, 39.02, 38.1, 38.1, 40.48, 37.21, 37.21, 39.53], 'accuracy5': [0.0, 5.26, 5.26, 9.52, 9.52, 18.18, 16.67, 16.67, 16.67, 20.0, 19.23, 22.22, 25.93, 22.22, 2

In [116]:
import matplotlib.pyplot as plt

threshold = 95  # Set the threshold for accuracy
fig, axs = plt.subplots(1, 8, figsize=(20, 5))  # Create 1 row and 7 columns of subplots

for i in range(8):
    accuracy_key = f'accuracy{i+1}'
    data_x = vertical_fov
    data_y = accuracy_results[accuracy_key]

    # Plot line segments individually
    for j in range(len(data_x) - 1):
        # Select color based on the value of the current point
        color = 'red' if data_y[j] >= threshold else 'green'
        axs[i].plot([data_x[j], data_x[j+1]], [data_y[j], data_y[j+1]], marker='o', markersize=3, color=color, linewidth=2)

    # Draw a horizontal line at the threshold value
    axs[i].axhline(y=threshold, color='red', linestyle='--', linewidth=1)

    axs[i].set_title(f'Row {i+1}')
    axs[i].set_ylim(0, 105)
    axs[i].grid()
    
axs[3].set_xlabel('Vertical FOV')      
axs[0].set_ylabel('Accuracy')  

plt.tight_layout()
plt.show()


1. *Check the row7 results by performing the v9c and botsort combination in 'window_analysis' file, validate them *


#### V8-Small & bytetrack

In [118]:
# -- video1

# -- processing of rows from 1-6

result_dict_small = count_crops_rows_1to6(y1, y2, xmin_limits[:6], xmax_limits[:6], 'small', 'wc_vid1.AVI')

# -- row7

results_row7_small = count_crops_rows_78(y1, y2, xmin_limits[6], xmax_limits[6], 25, 'small', 'wc_vid1.AVI')

# -- row8

results_row8_small = count_crops_rows_78(y1, y2, xmin_limits[7], xmax_limits[7], 15, 'small', 'wc_vid1.AVI')

#  -- row7 and ro8 count data to the 'result_dict'

result_dict_small['row7'] = results_row7_small
result_dict_small['row8'] = results_row8_small

Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [119]:
print(result_dict_small)

{'row1': [0, 1, 1, 3, 4, 5, 7, 7, 8, 10, 10, 11, 11, 13, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 19, 19, 20, 19, 20, 19, 19], 'row2': [0, 3, 2, 4, 6, 12, 12, 10, 9, 17, 18, 17, 17, 18, 21, 19, 23, 25, 23, 24, 29, 29, 29, 30, 30, 36, 34, 35, 33, 33, 33], 'row3': [0, 2, 6, 9, 12, 15, 16, 24, 22, 25, 18, 27, 27, 29, 23, 29, 31, 32, 25, 24, 23, 32, 24, 25, 25, 27, 27, 29, 28, 28, 27], 'row4': [0, 2, 7, 8, 14, 18, 19, 19, 22, 22, 27, 24, 28, 31, 29, 31, 32, 32, 30, 30, 32, 31, 31, 31, 33, 31, 34, 36, 36, 35, 35], 'row5': [0, 2, 3, 6, 10, 11, 11, 15, 14, 17, 21, 20, 18, 17, 20, 25, 25, 24, 24, 24, 25, 26, 27, 26, 26, 27, 28, 29, 28, 30, 30], 'row6': [0, 2, 3, 6, 5, 9, 8, 10, 12, 15, 16, 16, 17, 17, 18, 18, 19, 19, 23, 23, 23, 23, 26, 25, 26, 27, 26, 26, 28, 28, 28], 'row7': [0, 2, 1, 4, 6, 7, 8, 9, 10, 13, 14, 14, 17, 18, 19, 21, 23, 26, 24, 23, 25, 28, 27, 28, 29, 30, 33, 35, 33, 34, 34], 'row8': [0, 2, 4, 4, 5, 10, 10, 13, 17, 17, 18, 17, 17, 20, 20, 19, 22, 23, 23, 25, 22, 27, 26, 26, 28,

In [3]:
result_dict_small = {'row1': [0, 1, 1, 3, 4, 5, 7, 7, 8, 10, 10, 11, 11, 13, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 19, 19, 20, 19, 20, 19, 19], 'row2': [0, 3, 2, 4, 6, 12, 12, 10, 9, 17, 18, 17, 17, 18, 21, 19, 23, 25, 23, 24, 29, 29, 29, 30, 30, 36, 34, 35, 33, 33, 33], 'row3': [0, 2, 6, 9, 12, 15, 16, 24, 22, 25, 18, 27, 27, 29, 23, 29, 31, 32, 25, 24, 23, 32, 24, 25, 25, 27, 27, 29, 28, 28, 27], 'row4': [0, 2, 7, 8, 14, 18, 19, 19, 22, 22, 27, 24, 28, 31, 29, 31, 32, 32, 30, 30, 32, 31, 31, 31, 33, 31, 34, 36, 36, 35, 35], 'row5': [0, 2, 3, 6, 10, 11, 11, 15, 14, 17, 21, 20, 18, 17, 20, 25, 25, 24, 24, 24, 25, 26, 27, 26, 26, 27, 28, 29, 28, 30, 30], 'row6': [0, 2, 3, 6, 5, 9, 8, 10, 12, 15, 16, 16, 17, 17, 18, 18, 19, 19, 23, 23, 23, 23, 26, 25, 26, 27, 26, 26, 28, 28, 28], 'row7': [0, 2, 1, 4, 6, 7, 8, 9, 10, 13, 14, 14, 17, 18, 19, 21, 23, 26, 24, 23, 25, 28, 27, 28, 29, 30, 33, 35, 33, 34, 34], 'row8': [0, 2, 4, 4, 5, 10, 10, 13, 17, 17, 18, 17, 17, 20, 20, 19, 22, 23, 23, 25, 22, 27, 26, 26, 28, 29, 31, 32, 31, 32, 30]}

print(result_dict_small)

{'row1': [0, 1, 1, 3, 4, 5, 7, 7, 8, 10, 10, 11, 11, 13, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 19, 19, 20, 19, 20, 19, 19], 'row2': [0, 3, 2, 4, 6, 12, 12, 10, 9, 17, 18, 17, 17, 18, 21, 19, 23, 25, 23, 24, 29, 29, 29, 30, 30, 36, 34, 35, 33, 33, 33], 'row3': [0, 2, 6, 9, 12, 15, 16, 24, 22, 25, 18, 27, 27, 29, 23, 29, 31, 32, 25, 24, 23, 32, 24, 25, 25, 27, 27, 29, 28, 28, 27], 'row4': [0, 2, 7, 8, 14, 18, 19, 19, 22, 22, 27, 24, 28, 31, 29, 31, 32, 32, 30, 30, 32, 31, 31, 31, 33, 31, 34, 36, 36, 35, 35], 'row5': [0, 2, 3, 6, 10, 11, 11, 15, 14, 17, 21, 20, 18, 17, 20, 25, 25, 24, 24, 24, 25, 26, 27, 26, 26, 27, 28, 29, 28, 30, 30], 'row6': [0, 2, 3, 6, 5, 9, 8, 10, 12, 15, 16, 16, 17, 17, 18, 18, 19, 19, 23, 23, 23, 23, 26, 25, 26, 27, 26, 26, 28, 28, 28], 'row7': [0, 2, 1, 4, 6, 7, 8, 9, 10, 13, 14, 14, 17, 18, 19, 21, 23, 26, 24, 23, 25, 28, 27, 28, 29, 30, 33, 35, 33, 34, 34], 'row8': [0, 2, 4, 4, 5, 10, 10, 13, 17, 17, 18, 17, 17, 20, 20, 19, 22, 23, 23, 25, 22, 27, 26, 26, 28,

In [15]:

accuracy_results_small = calculate_accuracies(gt_counts_results, result_dict_small) 


In [16]:
import matplotlib.pyplot as plt

threshold = 95  # Set the threshold for accuracy
fig, axs = plt.subplots(1, 8, figsize=(20, 5))  # Create 1 row and 7 columns of subplots

for i in range(8):
    accuracy_key = f'accuracy{i+1}'
    data_x = vertical_fov
    data_y = accuracy_results_small[accuracy_key]

    # Plot line segments individually
    for j in range(len(data_x) - 1):
        # Select color based on the value of the current point
        color = 'red' if data_y[j] >= threshold else 'green'
        axs[i].plot([data_x[j], data_x[j+1]], [data_y[j], data_y[j+1]], marker='o', markersize=3, color=color, linewidth=2)

    # Draw a horizontal line at the threshold value
    axs[i].axhline(y=threshold, color='red', linestyle='--', linewidth=1)

    axs[i].set_title(f'Row {i+1}')
    axs[i].set_ylim(0, 105)
    axs[i].grid()
    
axs[3].set_xlabel('Vertical FOV')      
axs[0].set_ylabel('Accuracy')  

plt.tight_layout()
plt.show()


#### v9c - BoT-SORT

In [30]:
# -- video1

# -- processing of rows from 1-6

result_dict_v9c = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], 'v9c', 'wc_vid1.AVI')

# -- row7

results_row7_v9c = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, 'v9c', 'wc_vid1.AVI')

# -- row8

results_row8_v9c = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, 'v9c', 'wc_vid1.AVI')

#  -- row7 and ro8 count data to the 'result_dict'

result_dict_v9c['row7'] = results_row7_v9c
result_dict_v9c['row8'] = results_row8_v9c

# getting the accuracies for from gvt and v9c&botaort combination

accuracy_results_v9c = calculate_accuracies(gt_counts_results1, result_dict_v9c) 


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [31]:

print(result_dict_v9c)

{'row1': [0, 0, 3, 11, 11, 13, 14, 14, 15, 15, 17, 17, 17, 17, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 23, 23, 23, 24, 25], 'row2': [0, 3, 4, 13, 16, 15, 19, 19, 21, 21, 21, 21, 22, 24, 24, 24, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 29, 29, 30, 30], 'row3': [0, 3, 13, 23, 24, 25, 26, 27, 28, 30, 31, 30, 31, 31, 33, 33, 33, 33, 33, 33, 33, 35, 36, 36, 37, 37, 37, 37, 38, 37, 38], 'row4': [0, 3, 9, 23, 26, 27, 27, 28, 30, 30, 31, 33, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 42, 42, 43, 43], 'row5': [0, 3, 5, 20, 21, 22, 22, 22, 23, 23, 23, 24, 25, 26, 27, 28, 28, 28, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 33, 33, 33], 'row6': [0, 2, 13, 21, 20, 22, 22, 23, 24, 25, 25, 26, 28, 29, 28, 30, 30, 30, 30, 30, 31, 31, 31, 32, 32, 33, 33, 34, 34, 34, 34], 'row7': [0, 2, 10, 20, 20, 21, 21, 23, 23, 25, 25, 26, 26, 28, 28, 28, 29, 29, 29, 29, 29, 31, 32, 32, 32, 33, 35, 35, 36, 36, 35], 'row8': [0, 2, 5, 9, 12, 12, 14, 15, 15, 15, 15, 15, 16, 17, 17, 19, 20, 20, 20

In [38]:
plot_accuracy(accuracy_results_v9c)

#### v8-Nano & BoT-SORT

In [33]:
# -- video1

# -- processing of rows from 1-6

result_dict_nbs = count_crops_rows_1to6(y1, y2, xmin_limits[:6], xmax_limits[:6], 'nano', 'wc_vid1.AVI')

# -- row7

results_row7_nbs = count_crops_rows_78(y1, y2, xmin_limits[6], xmax_limits[6], 25, 'nano', 'wc_vid1.AVI')

# -- row8

results_row8_nbs = count_crops_rows_78(y1, y2, xmin_limits[7], xmax_limits[7], 15, 'nano', 'wc_vid1.AVI')

#  -- row7 and ro8 count data to the 'result_dict'

result_dict_nbs['row7'] = results_row7_nbs
result_dict_nbs['row8'] = results_row8_nbs

# getting the accuracies for from gvt and v9c&botaort combination

accuracy_results_nbs = calculate_accuracies(gt_counts_results1, result_dict_nbs) 


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [34]:
print(result_dict_nbs)

{'row1': [0, 1, 2, 8, 8, 9, 10, 10, 10, 11, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 16, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20], 'row2': [0, 1, 3, 10, 11, 12, 12, 12, 14, 13, 13, 13, 14, 13, 15, 15, 18, 16, 16, 16, 17, 18, 18, 17, 17, 18, 18, 19, 19, 19, 19], 'row3': [0, 2, 8, 17, 18, 18, 20, 20, 21, 21, 22, 24, 23, 24, 25, 24, 25, 25, 25, 24, 24, 25, 26, 27, 27, 28, 27, 27, 27, 28, 28], 'row4': [0, 4, 3, 17, 17, 19, 22, 23, 24, 23, 23, 24, 26, 26, 26, 25, 24, 25, 25, 26, 25, 26, 26, 26, 26, 26, 27, 28, 27, 27, 27], 'row5': [0, 1, 2, 10, 13, 13, 16, 16, 17, 16, 15, 17, 17, 18, 18, 19, 20, 19, 21, 21, 21, 21, 23, 23, 22, 22, 22, 22, 22, 22, 23], 'row6': [0, 2, 4, 14, 15, 16, 17, 17, 17, 18, 17, 20, 20, 21, 22, 21, 24, 23, 22, 22, 23, 24, 23, 23, 23, 23, 25, 25, 26, 26, 26], 'row7': [0, 1, 3, 11, 12, 12, 13, 15, 15, 15, 16, 17, 18, 18, 19, 20, 20, 19, 21, 18, 18, 20, 21, 21, 22, 20, 20, 22, 23, 23, 24], 'row8': [0, 1, 3, 5, 7, 7, 8, 9, 7, 9, 9, 10, 10, 10, 10, 11, 12, 11, 11, 11, 13, 12,

In [35]:

plot_accuracy(accuracy_results_nbs)


Run the above combo and also the small-botsort combo
get the results (plot)
Move to the 2nd video, 
    define row x-limits and then follow the save process like the video1

#### Defining Video2 limits (x) 

In [11]:
xmin_limits2 = [350, 725, 1134, 1674, 2236, 2678, 3140, 3476]
xmax_limits2 = [720, 1100, 1517, 1974, 2540, 3034, 3440, 3768]

# row7 = [3065, ymin], [3440, ymin], [3440, 1400], [3326, ymax], [3065, ymax]
# row8 = [3768, ymin], [3500, ymin], [3500, 1400], [3325, ymax], [3768, ymax]


In [12]:

def count_crops_rows_78B(y1_bounds, y2_bounds, xmin, xmax, limit, model_name, video):
    
    # -- here we will write a, if - else cond, where 20 is the threshold for the limit 
    # -- for row 7 - limit > 20 as the limit is 25
    # -- for row 8 - limit < 20 as the limit is 15

    if limit > 20:
        
        # -- logic for row 7
        
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            crops_count1 = track_and_count_crops(ymin, ymax, xmin=xmin, xmax=xmax, model=model_name, video_source=video)
            count_in_row.append(crops_count1)
         
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            crops_count2 = track_and_count_crops_r7(ymin, ymax, model_name, video, 'empty', x1=3065, x2=3440, x3=3440, x4=3326)
            count_in_row.append(crops_count2)
            
        return count_in_row
        
    else:
    
    # -- logic for row 8
    
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            crops_count1 = track_and_count_crops(ymin, ymax, xmin=xmin, xmax=xmax, model=model_name, video_source=video)
            count_in_row.append(crops_count1)
            
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            crops_count2 = track_and_count_crops_r8(ymin, ymax, model_name, video, 'empty', x1=3768, x2=3500, x3=3325, x4=3768)
            count_in_row.append(crops_count2)
            
        return count_in_row


In [141]:

# df, count = track_and_count_crops_r7(y1[-1], y2[-1], 'v9c', 'wc_vid2.mp4', 'vidB_r721', x1=3065, x2=3440, x3=3440, x4=3326)  # row 7

dfop13, countop13 = track_and_count_crops_r8(y1[13], y2[13], 'v9c', 'wc_vid2.mp4', 'vidB_r821', x1=3768, x2=3500, x3=3325, x4=3768)   # row 8

Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.


In [143]:

dfop14, countop14 = track_and_count_crops_r8(y1[14], y2[14], 'v9c', 'wc_vid2.mp4', 'vidB_r821', x1=3768, x2=3500, x3=3325, x4=3768)   # row 8


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.


In [144]:

dfop15, countop15 = track_and_count_crops_r8(y1[15], y2[15], 'v9c', 'wc_vid2.mp4', 'vidB_r821', x1=3768, x2=3500, x3=3325, x4=3768)   # row 8


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.


In [153]:
y1[0]

1045

In [154]:
y1[0:15]

[1045, 1010, 975, 940, 905, 870, 835, 800, 765, 730, 695, 660, 625, 590, 555]

In [151]:
y1[15]

520

In [159]:
count_list = []

for i in range(len(y1)):
    if i < 15:
        countr = track_and_count_crops(y1[i], y2[i], xmin_limits2[7], xmax_limits2[7], 'v9c', 'wc_vid2.mp4')
    else:
        countr = track_and_count_crops_r8(y1[i], y2[i], 'v9c', 'wc_vid2.mp4', 'vidB_r821', x1=3768, x2=3500, x3=3325, x4=3768)
    
    count_list.append(countr)
    
print(count_list)

Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [160]:
print(count_list)

[0, 2, 4, 10, 12, 13, 16, 16, 17, 18, 19, 19, 21, 22, 22, 23, 24, 25, 25, 25, 26, 26, 26, 27, 28, 29, 30, 29, 30, 29, 29]


In [134]:
# -- scatter plot

plt.scatter(df.iloc[:,2], df.iloc[:,3], s=8)
plt.xlim(0, 3840)
plt.ylim(2160, 0)
plt.show()

In [ ]:
# -- video2

# -- processing of rows from 1-6
resultB_dict_v9c = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], 'v9c', 'wc_vid2.mp4')

# -- row7
resultsB_row7_v9c = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'v9c', 'wc_vid2.mp4')

# -- row8
resultsB_row8_v9c = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'v9c', 'wc_vid2.mp4')

#  -- row7 and ro8 count data to the 'result_dict'

resultB_dict_v9c['row7'] = resultsB_row7_v9c
resultB_dict_v9c['row8'] = resultsB_row8_v9c

# getting the accuracies for from gvt and v9c&botaort combination

accuracy_results_v9c = calculate_accuracies(gt_counts_results2, resultB_dict_v9c) 


In [11]:

resultsB_row8_v9c_8B = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'v9c', 'wc_vid2.mp4')


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

#### v8nano-bytetrack

*Add the tracker argument in the main function for 3 functs

In [18]:
resultsB_row7_nano = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'nano', 'wc_vid2.mp4')

# -- row8
resultsB_row8_nano = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'nano', 'wc_vid2.mp4')

Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [26]:
# -- processing of rows from 1-6
resultB_dict_nano_chunk = count_crops_rows_1to6(y1, y2, xmin_limits2[3:6], xmax_limits2[3:6], 'nano', 'wc_vid2.mp4')

Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [ ]:
# -- video2

# -- processing of rows from 1-6
resultB_dict_nano = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], 'nano', 'wc_vid2.mp4')

# -- row7
resultsB_row7_nano = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'nano', 'wc_vid2.mp4')

# -- row8
resultsB_row8_nano = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'nano', 'wc_vid2.mp4')

# -- row7 and ro8 count data to the 'result_dict'

resultB_dict_nano['row7'] = resultsB_row7_nano
resultB_dict_nano['row8'] = resultsB_row8_nano

# getting the accuracies for from gvt and v9c&botaort combination

accuracy_results_nano = calculate_accuracies(gt_counts_results2, resultB_dict_nano) 


In [35]:
resB_dict_nano={}

resB_dict_nano['row1'] = [0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 3, 3, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 7, 6, 8, 10, 9, 9, 11]
resB_dict_nano['row2'] = [0, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 4, 3, 2, 4, 4]
resB_dict_nano['row3'] = [0, 2, 2, 6, 7, 11, 11, 15, 19, 19, 21, 20, 26, 22, 24, 26, 28, 28, 26, 28, 30, 28, 29, 29, 30, 29, 30, 31, 31, 33, 34]
resB_dict_nano['row4'] = [0, 2, 5, 3, 9, 7, 8, 10, 11, 9, 9, 14, 14, 16, 16, 14, 13, 16, 15, 16, 16, 15, 15, 17, 16, 16, 16, 18, 18, 21, 21]
resB_dict_nano['row5'] = [0, 2, 6, 3, 5, 6, 9, 7, 6, 10, 7, 10, 14, 10, 10, 11, 13, 11, 12, 11, 13, 12, 13, 13, 14, 14, 14, 15, 15, 16, 16]
resB_dict_nano['row6'] = [0, 1, 3, 5, 7, 6, 8, 10, 9, 10, 11, 11, 12, 13, 15, 15, 16, 14, 15, 16, 17, 18, 20, 18, 16, 18, 18, 18, 17, 19, 19]
resB_dict_nano['row7'] = resultsB_row7_nano
resB_dict_nano['row8'] = resultsB_row8_nano

In [36]:
print(resB_dict_nano)

{'row1': [0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 3, 3, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 7, 6, 8, 10, 9, 9, 11], 'row2': [0, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 4, 3, 2, 4, 4], 'row3': [0, 2, 2, 6, 7, 11, 11, 15, 19, 19, 21, 20, 26, 22, 24, 26, 28, 28, 26, 28, 30, 28, 29, 29, 30, 29, 30, 31, 31, 33, 34], 'row4': [0, 2, 5, 3, 9, 7, 8, 10, 11, 9, 9, 14, 14, 16, 16, 14, 13, 16, 15, 16, 16, 15, 15, 17, 16, 16, 16, 18, 18, 21, 21], 'row5': [0, 2, 6, 3, 5, 6, 9, 7, 6, 10, 7, 10, 14, 10, 10, 11, 13, 11, 12, 11, 13, 12, 13, 13, 14, 14, 14, 15, 15, 16, 16], 'row6': [0, 1, 3, 5, 7, 6, 8, 10, 9, 10, 11, 11, 12, 13, 15, 15, 16, 14, 15, 16, 17, 18, 20, 18, 16, 18, 18, 18, 17, 19, 19], 'row7': [0, 1, 2, 3, 3, 3, 5, 5, 3, 4, 3, 5, 6, 5, 6, 6, 6, 6, 6, 6, 6, 8, 9, 8, 9, 9, 11, 11, 12, 12, 12], 'row8': [0, 1, 2, 2, 2, 3, 2, 3, 5, 6, 6, 7, 8, 8, 6, 8, 8, 7, 7, 7, 7, 7, 8, 8, 8, 9, 8, 10, 10, 9, 9]}


In [65]:
resultA_dict_nano = {'row1': [0, 1, 1, 4, 6, 7, 8, 8, 9, 9, 10, 13, 11, 13, 13, 14, 11, 12, 12, 10, 10, 12, 12, 14, 14, 15, 14, 14, 14, 15, 15], 'row2': [0, 1, 4, 4, 7, 9, 9, 9, 9, 10, 10, 10, 11, 10, 12, 12, 15, 14, 14, 14, 18, 17, 16, 16, 12, 16, 13, 17, 18, 19, 17], 
                     'row3': [0, 2, 2, 7, 5, 9, 8, 11, 14, 13, 16, 13, 12, 12, 13, 13, 16, 14, 15, 15, 16, 19, 18, 19, 19, 19, 20, 19, 19, 21, 20], 'row4': [0, 3, 2, 4, 5, 7, 10, 10, 9, 10, 13, 12, 13, 12, 12, 12, 14, 14, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 16, 16, 17], 
                     'row5': [0, 1, 1, 2, 2, 4, 4, 4, 4, 5, 5, 6, 7, 6, 8, 9, 8, 9, 10, 11, 11, 11, 13, 14, 13, 13, 15, 18, 17, 17, 18], 'row6': [0, 3, 2, 5, 4, 7, 7, 8, 7, 9, 11, 8, 9, 11, 13, 14, 18, 16, 17, 17, 18, 18, 18, 19, 20, 18, 18, 20, 20, 19, 19], 
                     'row7': [0, 1, 1, 4, 4, 5, 4, 5, 7, 8, 8, 8, 10, 11, 13, 13, 13, 11, 13, 11, 10, 11, 11, 13, 14, 12, 12, 14, 14, 15, 16], 'row8': [0, 1, 2, 4, 5, 6, 5, 7, 7, 8, 8, 8, 9, 9, 8, 7, 9, 9, 8, 8, 9, 9, 9, 10, 11, 11, 11, 12, 11, 11, 11]}

resB_dict_nano = {'row1': [0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 3, 3, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 7, 6, 8, 10, 9, 9, 11], 'row2': [0, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 4, 3, 2, 4, 4], 
                  'row3': [0, 2, 2, 6, 7, 11, 11, 15, 19, 19, 21, 20, 26, 22, 24, 26, 28, 28, 26, 28, 30, 28, 29, 29, 30, 29, 30, 31, 31, 33, 34], 'row4': [0, 2, 5, 3, 9, 7, 8, 10, 11, 9, 9, 14, 14, 16, 16, 14, 13, 16, 15, 16, 16, 15, 15, 17, 16, 16, 16, 18, 18, 21, 21], 
                  'row5': [0, 2, 6, 3, 5, 6, 9, 7, 6, 10, 7, 10, 14, 10, 10, 11, 13, 11, 12, 11, 13, 12, 13, 13, 14, 14, 14, 15, 15, 16, 16], 'row6': [0, 1, 3, 5, 7, 6, 8, 10, 9, 10, 11, 11, 12, 13, 15, 15, 16, 14, 15, 16, 17, 18, 20, 18, 16, 18, 18, 18, 17, 19, 19], 
                  'row7': [0, 1, 2, 3, 3, 3, 5, 5, 3, 4, 3, 5, 6, 5, 6, 6, 6, 6, 6, 6, 6, 8, 9, 8, 9, 9, 11, 11, 12, 12, 12], 'row8': [0, 1, 2, 2, 2, 3, 2, 3, 5, 6, 6, 7, 8, 8, 6, 8, 8, 7, 7, 7, 7, 7, 8, 8, 8, 9, 8, 10, 10, 9, 9]}

In [72]:
gt_nanoA = gen_gt_results('nano', 'A')

accuracy_results_nanoA = calculate_accuracies(gt_nanoA, resultA_dict_nano) 
print(accuracy_results_nanoA)

gt_nanoB = gen_gt_results('nano', 'B')

accuracy_results_nanoB = calculate_accuracies(gt_nanoB, resB_dict_nano) 
print(accuracy_results_nanoB)

nano_accuracies = get_avg_accuracy(accuracy_results_nanoA, accuracy_results_nanoB)

print(nano_accuracies)

{'accuracy1': [0.0, 6.67, 5.88, 22.22, 31.58, 36.84, 40.0, 38.1, 40.91, 37.5, 41.67, 52.0, 44.0, 50.0, 50.0, 51.85, 40.74, 44.44, 44.44, 35.71, 33.33, 40.0, 40.0, 45.16, 43.75, 45.45, 42.42, 41.18, 41.18, 42.86, 42.86], 'accuracy2': [0.0, 5.56, 22.22, 21.05, 36.84, 45.0, 42.86, 40.91, 39.13, 41.67, 41.67, 41.67, 45.83, 40.0, 46.15, 46.15, 55.56, 51.85, 51.85, 50.0, 64.29, 60.71, 57.14, 55.17, 41.38, 55.17, 44.83, 56.67, 60.0, 63.33, 56.67], 'accuracy3': [0.0, 8.7, 8.33, 29.17, 19.23, 33.33, 29.63, 36.67, 46.67, 41.94, 50.0, 40.62, 37.5, 36.36, 39.39, 39.39, 48.48, 42.42, 45.45, 45.45, 45.71, 54.29, 50.0, 52.78, 51.35, 51.35, 52.63, 50.0, 48.72, 53.85, 51.28], 'accuracy4': [0.0, 11.54, 7.69, 14.81, 18.52, 25.93, 34.48, 34.48, 29.03, 31.25, 38.24, 35.29, 37.14, 33.33, 33.33, 32.43, 37.84, 37.84, 39.47, 38.46, 38.46, 41.03, 40.0, 39.02, 39.02, 38.1, 38.1, 40.48, 37.21, 37.21, 39.53], 'accuracy5': [0.0, 5.26, 5.26, 9.52, 9.52, 18.18, 16.67, 16.67, 16.67, 20.0, 19.23, 22.22, 25.93, 22.22, 2

In [71]:

plot_accuracy(nano_accuracies)

#### v8s & bt

In [74]:
# -- video2

# -- processing of rows from 1-6
resultB_dict_small = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], 'small', 'wc_vid2.mp4')

# -- row7
resultsB_row7_small = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'small', 'wc_vid2.mp4')

# -- row8
resultsB_row8_small = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'small', 'wc_vid2.mp4')

# -- row7 and ro8 count data to the 'result_dict'

resultB_dict_small['row7'] = resultsB_row7_small
resultB_dict_small['row8'] = resultsB_row8_small


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [75]:
resultA_dict_small = {  'row1': [0, 1, 1, 3, 4, 5, 7, 7, 8, 10, 10, 11, 11, 13, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 19, 19, 20, 19, 20, 19, 19], 'row2': [0, 3, 2, 4, 6, 12, 12, 10, 9, 17, 18, 17, 17, 18, 21, 19, 23, 25, 23, 24, 29, 29, 29, 30, 30, 36, 34, 35, 33, 33, 33], 
                        'row3': [0, 2, 6, 9, 12, 15, 16, 24, 22, 25, 18, 27, 27, 29, 23, 29, 31, 32, 25, 24, 23, 32, 24, 25, 25, 27, 27, 29, 28, 28, 27], 'row4': [0, 2, 7, 8, 14, 18, 19, 19, 22, 22, 27, 24, 28, 31, 29, 31, 32, 32, 30, 30, 32, 31, 31, 31, 33, 31, 34, 36, 36, 35, 35], 
                        'row5': [0, 2, 3, 6, 10, 11, 11, 15, 14, 17, 21, 20, 18, 17, 20, 25, 25, 24, 24, 24, 25, 26, 27, 26, 26, 27, 28, 29, 28, 30, 30], 'row6': [0, 2, 3, 6, 5, 9, 8, 10, 12, 15, 16, 16, 17, 17, 18, 18, 19, 19, 23, 23, 23, 23, 26, 25, 26, 27, 26, 26, 28, 28, 28], 
                        'row7': [0, 2, 1, 4, 6, 7, 8, 9, 10, 13, 14, 14, 17, 18, 19, 21, 23, 26, 24, 23, 25, 28, 27, 28, 29, 30, 33, 35, 33, 34, 34], 'row8': [0, 2, 4, 4, 5, 10, 10, 13, 17, 17, 18, 17, 17, 20, 20, 19, 22, 23, 23, 25, 22, 27, 26, 26, 28, 29, 31, 32, 31, 32, 30]
                    }


In [76]:
gt_smallA = gen_gt_results('small', 'A')

accuracy_results_smallA = calculate_accuracies(gt_smallA, resultA_dict_small) 
print(accuracy_results_smallA)

gt_smallB = gen_gt_results('small', 'B')

accuracy_results_smallB = calculate_accuracies(gt_smallB, resultB_dict_small) 
print(accuracy_results_smallB)

small_accuracies = get_avg_accuracy(accuracy_results_smallA, accuracy_results_smallB)

print(small_accuracies)

{'accuracy1': [0.0, 6.67, 5.88, 16.67, 21.05, 26.32, 35.0, 33.33, 36.36, 41.67, 41.67, 44.0, 44.0, 50.0, 57.69, 59.26, 59.26, 59.26, 59.26, 57.14, 53.33, 56.67, 56.67, 54.84, 59.38, 57.58, 60.61, 55.88, 58.82, 54.29, 54.29], 'accuracy2': [0.0, 16.67, 11.11, 21.05, 31.58, 60.0, 57.14, 45.45, 39.13, 70.83, 75.0, 70.83, 70.83, 72.0, 80.77, 73.08, 85.19, 92.59, 85.19, 85.71, 96.43, 96.43, 96.43, 96.55, 96.55, 75.86, 82.76, 83.33, 90.0, 90.0, 90.0], 'accuracy3': [0.0, 8.7, 25.0, 37.5, 46.15, 55.56, 59.26, 80.0, 73.33, 80.65, 56.25, 84.38, 84.38, 87.88, 69.7, 87.88, 93.94, 96.97, 75.76, 72.73, 65.71, 91.43, 66.67, 69.44, 67.57, 72.97, 71.05, 76.32, 71.79, 71.79, 69.23], 'accuracy4': [0.0, 7.69, 26.92, 29.63, 51.85, 66.67, 65.52, 65.52, 70.97, 68.75, 79.41, 70.59, 80.0, 86.11, 80.56, 83.78, 86.49, 86.49, 78.95, 76.92, 82.05, 79.49, 77.5, 75.61, 80.49, 73.81, 80.95, 85.71, 83.72, 81.4, 81.4], 'accuracy5': [0.0, 10.53, 15.79, 28.57, 47.62, 50.0, 45.83, 62.5, 58.33, 68.0, 80.77, 74.07, 66.67, 62

In [77]:
plot_accuracy(small_accuracies)

#### v8nano-botsort

In [20]:
# -- video2

# -- processing of rows from 1-6
resultB_dict_nano = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], 'nano', 'wc_vid2.mp4')

# -- row7
resultsB_row7_nano = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, 'nano', 'wc_vid2.mp4')

# -- row8
resultsB_row8_nano = count_crops_rows_78B(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, 'nano', 'wc_vid2.mp4')

# -- row7 and ro8 count data to the 'result_dict'

resultB_dict_nano['row7'] = resultsB_row7_nano
resultB_dict_nano['row8'] = resultsB_row8_nano


Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully completed.
Polygon Counter Initiated.
Video frame is empty or video processing has been successfully c

In [21]:
print(resultB_dict_nano)


{'row1': [0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 3, 3, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 7, 6, 8, 10, 9, 9, 11], 'row2': [0, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 4, 3, 2, 4, 4], 'row3': [0, 2, 2, 6, 7, 11, 11, 15, 19, 19, 21, 20, 26, 22, 24, 26, 28, 28, 26, 28, 30, 28, 29, 29, 30, 29, 30, 31, 31, 33, 34], 'row4': [0, 2, 5, 3, 9, 7, 8, 10, 11, 9, 9, 14, 14, 16, 16, 14, 13, 16, 15, 16, 16, 15, 15, 17, 16, 16, 16, 18, 18, 21, 21], 'row5': [0, 2, 6, 3, 5, 6, 9, 7, 6, 10, 7, 10, 14, 10, 10, 11, 13, 11, 12, 11, 13, 12, 13, 13, 14, 14, 14, 15, 15, 16, 16], 'row6': [0, 1, 3, 5, 7, 6, 8, 10, 9, 10, 11, 11, 12, 13, 15, 15, 16, 14, 15, 16, 17, 18, 20, 18, 16, 18, 18, 18, 17, 19, 19], 'row7': [0, 1, 2, 3, 3, 3, 5, 5, 3, 4, 3, 5, 6, 5, 6, 6, 6, 6, 6, 6, 6, 8, 9, 8, 9, 9, 11, 11, 12, 12, 12], 'row8': [0, 1, 2, 2, 2, 3, 2, 3, 5, 6, 6, 7, 8, 8, 6, 8, 8, 7, 7, 7, 7, 7, 8, 8, 8, 9, 8, 10, 10, 9, 9]}


In [ ]:
print(resultB_dict_nano)

In [22]:
resultA_dict_nano =     {'row1': [0, 1, 2, 8, 8, 9, 10, 10, 10, 11, 12, 13, 13, 13, 13, 14, 14, 14, 14, 15, 16, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20], 'row2': [0, 1, 3, 10, 11, 12, 12, 12, 14, 13, 13, 13, 14, 13, 15, 15, 18, 16, 16, 16, 17, 18, 18, 17, 17, 18, 18, 19, 19, 19, 19], 
                        'row3': [0, 2, 8, 17, 18, 18, 20, 20, 21, 21, 22, 24, 23, 24, 25, 24, 25, 25, 25, 24, 24, 25, 26, 27, 27, 28, 27, 27, 27, 28, 28], 'row4': [0, 4, 3, 17, 17, 19, 22, 23, 24, 23, 23, 24, 26, 26, 26, 25, 24, 25, 25, 26, 25, 26, 26, 26, 26, 26, 27, 28, 27, 27, 27], 
                        'row5': [0, 1, 2, 10, 13, 13, 16, 16, 17, 16, 15, 17, 17, 18, 18, 19, 20, 19, 21, 21, 21, 21, 23, 23, 22, 22, 22, 22, 22, 22, 23], 'row6': [0, 2, 4, 14, 15, 16, 17, 17, 17, 18, 17, 20, 20, 21, 22, 21, 24, 23, 22, 22, 23, 24, 23, 23, 23, 23, 25, 25, 26, 26, 26], 
                        'row7': [0, 1, 3, 11, 12, 12, 13, 15, 15, 15, 16, 17, 18, 18, 19, 20, 20, 19, 21, 18, 18, 20, 21, 21, 22, 20, 20, 22, 23, 23, 24], 'row8': [0, 1, 3, 5, 7, 7, 8, 9, 7, 9, 9, 10, 10, 10, 10, 11, 12, 11, 11, 11, 13, 12, 13, 12, 11, 11, 13, 13, 12, 12, 14]}

In [29]:
gt_nanoA = gen_gt_results('nano', 'A')

accuracy_results_nanoA = calculate_accuracies(gt_nanoA, resultA_dict_nano) 
print(accuracy_results_nanoA)

gt_nanoB = gen_gt_results('nano', 'B')

accuracy_results_nanoB = calculate_accuracies(gt_nanoB, resultB_dict_nano) 
print(accuracy_results_nanoB)

nano_accuracies = get_avg_accuracy(accuracy_results_nanoA, accuracy_results_nanoB)

print(nano_accuracies)


{'accuracy1': [0.0, 6.67, 11.76, 44.44, 42.11, 47.37, 50.0, 47.62, 45.45, 45.83, 50.0, 52.0, 52.0, 50.0, 50.0, 51.85, 51.85, 51.85, 51.85, 53.57, 53.33, 60.0, 60.0, 58.06, 56.25, 57.58, 57.58, 55.88, 55.88, 57.14, 57.14], 'accuracy2': [0.0, 5.56, 16.67, 52.63, 57.89, 60.0, 57.14, 54.55, 60.87, 54.17, 54.17, 54.17, 58.33, 52.0, 57.69, 57.69, 66.67, 59.26, 59.26, 57.14, 60.71, 64.29, 64.29, 58.62, 58.62, 62.07, 62.07, 63.33, 63.33, 63.33, 63.33], 'accuracy3': [0.0, 8.7, 33.33, 70.83, 69.23, 66.67, 74.07, 66.67, 70.0, 67.74, 68.75, 75.0, 71.88, 72.73, 75.76, 72.73, 75.76, 75.76, 75.76, 72.73, 68.57, 71.43, 72.22, 75.0, 72.97, 75.68, 71.05, 71.05, 69.23, 71.79, 71.79], 'accuracy4': [0.0, 15.38, 11.54, 62.96, 62.96, 70.37, 75.86, 79.31, 77.42, 71.88, 67.65, 70.59, 74.29, 72.22, 72.22, 67.57, 64.86, 67.57, 65.79, 66.67, 64.1, 66.67, 65.0, 63.41, 63.41, 61.9, 64.29, 66.67, 62.79, 62.79, 62.79], 'accuracy5': [0.0, 5.26, 10.53, 47.62, 61.9, 59.09, 66.67, 66.67, 70.83, 64.0, 57.69, 62.96, 62.96,

In [31]:
plot_accuracy(nano_accuracies)

#### v10

In [17]:
# -- row7
resultsB_row7_nano = count_crops_rows_78B(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, '10n', 'wc_vid2.mp4')


AttributeError: Can't get attribute 'YOLOv10DetectionModel' on <module 'ultralytics.nn.tasks' from 'c:\\Users\\ashis\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\ultralytics\\nn\\tasks.py'>

### New Approach (Independent detection & tracking)

#### Utilities

##### Get detection results

In [6]:

video1 = "C:/Users/ashis/Desktop/THESIS/DT_flow/resources/data/wc_vid1.AVI"
video2 = "C:/Users/ashis/Desktop/THESIS/DT_flow/resources/data/wc_vid2.mp4"

# -- function to get the detection results

def detect_results(set, model, video_source):
    
    '''
    model : (str) - nano, small, medium etc..
    video_source : (str)
    set : (str) - A or B
    
    video is 1 then set will be 'B' & vice versa
    '''
    
    weight = f"C:/Users/ashis/Desktop/THESIS/DT_flow/resources/weights/set{set}/{model}.pt"
    
    model = YOLO(weight)

    results = model.predict(source=video_source, project='files/roi_detection/custom_botsort_detections', conf=.4, verbose=False,
                      save=True, save_frames=True, line_width=2)
    return results

# detect_results('A', 'large', video2)
# detect_results('B', 'large', video1)


In [4]:
# -- video1 - set B

results_nano1 = detect_results('B', 'nano', video1)
results_small1 = detect_results('B', 'small', video1)
results_medium1 = detect_results('B', 'medium', video1)
results_large1 = detect_results('B', 'large', video1)
results_v9c1 = detect_results('B', 'v9c', video1)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to files\roi_detection\custom_botsort_detections\predict

WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for b

In [5]:
# -- video2 - set A

results_nano2 = detect_results('A', 'nano', video2)
results_small2 = detect_results('A', 'small', video2)
results_medium2 = detect_results('A', 'medium', video2)
results_large2 = detect_results('A', 'large', video2)
results_v9c2 = detect_results('A', 'v9c', video2)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to files\roi_detection\custom_botsort_detections\predict6

WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for 

##### Dividing detections into rows

In [7]:

xmin_limits1 = [350, 735, 1147, 1689, 2270, 2716, 3090, 3493]
xmax_limits1 = [736, 1089, 1540, 2089, 2533, 3048, 3453, 3747]

xmin_limits2 = [350, 725, 1134, 1674, 2236, 2678, 3140, 3476]
xmax_limits2 = [720, 1100, 1517, 1974, 2540, 3034, 3440, 3768]

y1 = [1045, 1010, 975, 940, 905, 870, 835, 800, 765, 730, 695, 660, 625, 590, 555, 520, 485, 450, 415, 380, 345, 
      310, 275, 240, 205, 170, 135, 100, 65, 30, 0]

y2 = [1115, 1150, 1185, 1220, 1255, 1290, 1325, 1360, 1395, 1430, 1465, 1500, 1535, 1570, 1605, 1640, 1675, 1710, 
      1745, 1780, 1815, 1850, 1885, 1920, 1955, 1990, 2025, 2060, 2095, 2130, 2160]

# xcenters1 = [543.0, 886.5, 1343.5, 1889.0, 2401.5, 2882.0, 3253.5, 3561.5] 
# xcenters2 = [487.0, 760.0, 1150.0, 1673.5, 2226.0, 2746.0, 3201.0, 3510.5] 

xcenters1 = [(a + b) / 2 for a, b in zip(xmin_limits1, xmax_limits1)]     # -- X centers of setA
xcenters2 = [(c + d) / 2 for c, d in zip(xmin_limits2, xmax_limits2)]   # -- X centers of setA

xlow_lim = [321, 684, 1134, 1674, 2236, 2678, 3054, 3363]
xhigh_lim = [749, 1092, 1540, 2089, 2539, 3048, 3452, 3768]

ycenters = [1080] * 8

horizontal_va1 = win_analysis.calculate_angles(xlow_lim, ycenters)
horizontal_va2 = win_analysis.calculate_angles(xhigh_lim, ycenters)

vertical_fov = win_analysis.calculate_vertical_fov(y1, y2)

In [8]:
# -- Counting accuracy related functions

def ret_gt_cropcount(df, y1, y2):
    
    trim_df = df[(df['CenterY'] > y1) & (df['CenterY'] < y2)]
    crop_count = trim_df['id'].nunique()
    
    return crop_count

def get_accy(gt_count, pred_count):
    
    error_rate = abs(pred_count - gt_count) / gt_count
    if error_rate > 1:
        return 0  # Returns 0% accuracy if error rate exceeds 100%
    else:
        accuracy = (1 - error_rate) * 100
        return round(accuracy, 2)

def process_gt_counts(row_data, y1_data, y2_data):      # row_data - a list of 'row' dataframes
    results = {}  # Dictionary to store the results for each row

    for row_idx in range(1, len(row_data) + 1):
        row_key = f'gt_row{row_idx}'
        gt_counts = []
        
        for i in range(len(y1)):  # Assuming each row has 25 entries
            gt_count = ret_gt_cropcount(row_data[row_idx - 1], y1_data[i], y2_data[i])
            gt_counts.append(gt_count)
        
        results[row_key] = gt_counts
    
    return results

def calculate_accuracies(gt_data, count_data):
    results = {}  # Dictionary to store the accuracy lists for each row

    for row_idx in range(1, len(gt_data) + 1):
        accuracies = []
        gt_row = gt_data[f'gt_row{row_idx}']
        count_row = count_data[f'row{row_idx}']

        for i in range(len(y1)):  # Assuming 25 measurements per row
            accuracy = get_accy(gt_row[i], count_row[i])
            accuracies.append(accuracy)
        
        results[f'accuracy{row_idx}'] = accuracies
    
    return results

def gen_gt_results(model, type):

    gt_counts_results={}
    df = pd.read_csv(f'C:/Users/ashis/Desktop/THESIS/DT_flow/resources/dataframes{type}/{model}_gt.csv')  # -- type=A/B, model=small
    
    if type=='A': 
        row1, row2, row3, row4, row5, row6, row7, row8 = det_utils.rows_setA(df)
    else:
        row1, row2, row3, row4, row5, row6, row7, row8 = det_utils.rows_setB(df)
        
    gt_df = [row1, row2, row3, row4, row5, row6, row7, row8]
    gt_counts_results = process_gt_counts(gt_df, y1, y2)
    
    return gt_counts_results

def get_avg_accuracy(accuracy_results1, accuracy_results2):

    average_results = {}

    for key in accuracy_results1:
        list_a = accuracy_results1[key]
        list_b = accuracy_results2[key]
        average_list = [(a + b) / 2 for a, b in zip(list_a, list_b)]
        average_results[key] = average_list
    
    return average_results


In [9]:

# -- mask/split the detection into window limits as per 'x' and 'y' limits

def ret_frame_dets(tensor, i):

    xyxy = tensor[i].boxes.xyxy
    conf = tensor[i].boxes.conf
    
    df1 = pd.DataFrame(xyxy)
    df2 = pd.DataFrame(conf)
    
    df_appended = pd.concat([df1, df2], axis=1)
    final_df = df_appended[df_appended.iloc[:,1] >= 660]  # -- excluding the detections above the white pipe as they were not included in the Ground Truths
    
    return final_df


def ret_trimmed_df(df_appended, xmin, xmax, y1, y2):
    
    trim_df = df_appended[((((df_appended.iloc[:,0]+df_appended.iloc[:,2])/2 >= xmin) & ((df_appended.iloc[:,0]+df_appended.iloc[:,2])/2 <= xmax)) & 
                       (((df_appended.iloc[:,1]+df_appended.iloc[:,3])/2 >= y1) & ((df_appended.iloc[:,1]+df_appended.iloc[:,3])/2 <= y2)))]
    
    xyxy = trim_df.iloc[:, :-1].values
    confidence = trim_df.iloc[:, -1].values

    return (xyxy, confidence)

# -- function that gives the count of crops of the window through 'ByteTrack' tracker

def get_count(window_listup):
    
    tracker = sv.ByteTrack(minimum_matching_threshold=.8, track_activation_threshold=.5, lost_track_buffer=24)
    
    id = []
    count = []
    track_ids = []
    
    for i in range(29):
        
        results = window_listup[i]
        boxes, confidences = results
        detections = sv.Detections(
            xyxy=boxes,
            confidence=confidences,
            class_id=np.zeros(len(confidences), dtype=int)  # -- class_id is 0
        )

        detections = tracker.update_with_detections(detections)
        ids = detections.tracker_id.tolist()
        track_ids.append(ids)

    for i in range(len(track_ids)):
        for j in range(len(track_ids[i])):
            id.append(track_ids[i][j])
            
    count = len(set(id))
    
    return count

In [350]:

def count_crops_rows_1to6(y1_bounds, y2_bounds, xmin_list, xmax_list, model):

    results = {}
    
    for i in range(len(xmin_list)):
        
        count_in_row = []   
        
        for ymin, ymax in zip(y1_bounds, y2_bounds):
            
            results_window = []   # -- window 
            
            for j in range(29):

                df = ret_frame_dets(model, j)      # 'model' -- need to update variable - model = 'results_nano' for different models
                
                frame_detections = ret_trimmed_df(df, xmin_list[i], xmax_list[i], ymin, ymax)

                results_window.append(frame_detections)
            
            crops_count = get_count(results_window)
            
            count_in_row.append(crops_count)
            
        print(f'row{i+1}: {count_in_row}')
        
        results[f'row{i+1}'] = count_in_row
        
    return results


In [352]:

def count_crops_rows_78(y1_bounds, y2_bounds, xmin_list, xmax_list, limit, model):
        
    # -- here we will write a, if - else cond, where 20 is the threshold for the limit 
    # -- for row 7 - limit > 20 as the limit is 25
    # -- for row 8 - limit < 20 as the limit is 15

    if limit > 20:
        
        # -- logic for row 7
        
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            results_window = []   # -- window 
            
            for j in range(29):

                df = ret_frame_dets(model, j) 
                frame_detections = ret_trimmed_df(df, xmin_list, xmax_list, ymin, ymax)
                results_window.append(frame_detections)
                
            crops_count1 = get_count(results_window)
            count_in_row.append(crops_count1)
            
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            results_window = []   # -- window 
            
            for j in range(29):

                df = ret_frame_dets(model, j)      
                
                df = df[((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3400) & (df.iloc[:,3] <= 2160) & (df.iloc[:,3] > 1500)) | 
                         ((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3500) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))] 

                frame_detections = ret_trimmed_df(df, xmin_list, xmax_list, ymin, ymax)
                results_window.append(frame_detections)
            
            crops_count2 = get_count(results_window)
            
            count_in_row.append(crops_count2)
            
        return count_in_row
        
        
    else:
    
    # -- logic for row 8
    
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            results_window = []   # -- window 
            
            for j in range(29):

                df = ret_frame_dets(model, j) 
                frame_detections = ret_trimmed_df(df, xmin_list, xmax_list, ymin, ymax)
                results_window.append(frame_detections)
                
            crops_count1 = get_count(results_window)
            count_in_row.append(crops_count1)
            
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            results_window = []   # -- window 
            
            for j in range(29):

                df = ret_frame_dets(model, j)      
                
                df = df[((df.iloc[:,2] >= 3420) & (df.iloc[:,3] > 1500)) | ((df.iloc[:,2] >= 3493) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]
                
                frame_detections = ret_trimmed_df(df, xmin_list, xmax_list, ymin, ymax)
                results_window.append(frame_detections)
            
            crops_count2 = get_count(results_window)
            
            count_in_row.append(crops_count2)
            
        return count_in_row


###### Processing

In [353]:
# -- video 1

row16_nano1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_nano1)
row7_nano1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_nano1)
row8_nano1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_nano1)
row16_nano1['row7'] = row7_nano1
row16_nano1['row8'] = row8_nano1
print(row16_nano1['row7'])
print(row16_nano1['row8'])
print()

row16_small1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_small1)
row7_small1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_small1)
row8_small1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_small1)
row16_small1['row7'] = row7_small1
row16_small1['row8'] = row8_small1
print(row16_small1['row7'])
print(row16_small1['row8'])
print()

row16_medium1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_medium1)
row7_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_medium1)
row8_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_medium1)
row16_medium1['row7'] = row7_medium1
row16_medium1['row8'] = row8_medium1
print(row16_medium1['row7'])
print(row16_medium1['row8'])
print()

row16_large1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_large1)
row7_large1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_large1)
row8_large1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_large1)
row16_large1['row7'] = row7_large1
row16_large1['row8'] = row8_large1
print(row16_large1['row7'])
print(row16_large1['row8'])
print()

row16_v9c1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_v9c1)
row7_v9c1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_v9c1)
row8_v9c1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_v9c1)
row16_v9c1['row7'] = row7_v9c1
row16_v9c1['row8'] = row8_v9c1
print(row16_v9c1['row7'])
print(row16_v9c1['row8'])
print()

# -- video 2

row16_nano2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_nano2)
row7_nano2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_nano2)
row8_nano2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_nano2)
row16_nano2['row7'] = row7_nano2
row16_nano2['row8'] = row8_nano2
print(row16_nano2['row7'])
print(row16_nano2['row8'])
print()

row16_small2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_small2)
row7_small2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_small2)
row8_small2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_small2)
row16_small2['row7'] = row7_small2
row16_small2['row8'] = row8_small2
print(row16_small2['row7'])
print(row16_small2['row8'])
print()

row16_medium2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_medium2)
row7_medium2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_medium2)
row8_medium2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_medium2)
row16_medium2['row7'] = row7_medium2
row16_medium2['row8'] = row8_medium2
print(row16_medium2['row7'])
print(row16_medium2['row8'])
print()

row16_large2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_large2)
row7_large2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_large2)
row8_large2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_large2)
row16_large2['row7'] = row7_large2
row16_large2['row8'] = row8_large2
print(row16_large2['row7'])
print(row16_large2['row8'])
print()

row16_v9c2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_v9c2)
row7_v9c2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_v9c2)
row8_v9c2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_v9c2)
row16_v9c2['row7'] = row7_v9c2
row16_v9c2['row8'] = row8_v9c2
print(row16_v9c2['row7'])
print(row16_v9c2['row8'])
print()


row1: [1, 1, 2, 2, 4, 4, 5, 5, 6, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 11, 11, 11, 13, 13, 13, 13, 13, 14, 15, 15]
row2: [1, 3, 6, 6, 8, 7, 9, 12, 12, 14, 15, 16, 18, 18, 19, 20, 20, 21, 22, 22, 22, 22, 23, 24, 24, 24, 24, 25, 25, 25, 25]
row3: [2, 5, 9, 11, 10, 12, 14, 17, 21, 22, 22, 22, 23, 23, 23, 26, 26, 26, 26, 27, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 31]
row4: [2, 4, 6, 12, 14, 17, 16, 23, 24, 28, 29, 31, 32, 36, 36, 35, 38, 39, 39, 41, 42, 44, 44, 45, 45, 46, 46, 46, 47, 47, 47]
row5: [1, 2, 5, 6, 7, 7, 9, 8, 11, 11, 11, 12, 12, 14, 15, 16, 16, 17, 17, 18, 18, 19, 21, 21, 21, 22, 22, 23, 24, 24, 24]
row6: [1, 3, 4, 4, 6, 6, 6, 9, 9, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 13, 14, 15, 16, 16, 17, 18, 18, 19]
[0, 2, 3, 6, 6, 9, 9, 10, 12, 12, 12, 14, 14, 15, 16, 17, 18, 19, 20, 21, 22, 21, 22, 22, 22, 23, 24, 25, 26, 28, 28]
[1, 4, 4, 4, 7, 8, 10, 11, 12, 13, 13, 13, 13, 14, 14, 14, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17]

row1: [1, 1, 3, 4, 6, 8,

In [466]:
# -- Accuracies

# -- nano
gt_nano1 = gen_gt_results('nano', 'A')
accuracy_results_nano1 = calculate_accuracies(gt_nano1, row16_nano1) 

gt_nano2 = gen_gt_results('nano', 'B')
accuracy_results_nano2 = calculate_accuracies(gt_nano2, row16_nano2) 

nano_accuracies = get_avg_accuracy(accuracy_results_nano1, accuracy_results_nano2)

# -- small
gt_small1 = gen_gt_results('nano', 'A')
accuracy_results_small1 = calculate_accuracies(gt_small1, row16_small1) 

gt_small2 = gen_gt_results('nano', 'B')
accuracy_results_small2 = calculate_accuracies(gt_small2, row16_small2) 

small_accuracies = get_avg_accuracy(accuracy_results_small1, accuracy_results_small2)

#-- medium
gt_medium1 = gen_gt_results('nano', 'A')
accuracy_results_medium1 = calculate_accuracies(gt_medium1, row16_medium1) 

gt_medium2 = gen_gt_results('nano', 'B')
accuracy_results_medium2 = calculate_accuracies(gt_medium2, row16_medium2) 

medium_accuracies = get_avg_accuracy(accuracy_results_medium1, accuracy_results_medium2)

# -- large
gt_large1 = gen_gt_results('nano', 'A')
accuracy_results_large1 = calculate_accuracies(gt_large1, row16_large1) 

gt_large2 = gen_gt_results('nano', 'B')
accuracy_results_large2 = calculate_accuracies(gt_large2, row16_large2) 

large_accuracies = get_avg_accuracy(accuracy_results_large1, accuracy_results_large2)

# -- v9c
gt_v9c1 = gen_gt_results('nano', 'A')
accuracy_results_v9c1 = calculate_accuracies(gt_v9c1, row16_v9c1) 

gt_v9c2 = gen_gt_results('nano', 'B')
accuracy_results_v9c2 = calculate_accuracies(gt_v9c2, row16_v9c2) 

v9c_accuracies = get_avg_accuracy(accuracy_results_v9c1, accuracy_results_v9c2)


In [467]:
accuracy_results_list = [nano_accuracies, small_accuracies, medium_accuracies, v9c_accuracies, large_accuracies]
model_names = ['nano', 'small', 'medium', 'v9c', 'large']

In [233]:

def plot_accuracy(accuracy_results):
    
    threshold = 95  # Set the threshold for accuracy
    fig, axs = plt.subplots(1, 8, figsize=(20, 5))  # Create 1 row and 8 columns of subplots

    for i in range(8):
        accuracy_key = f'accuracy{i+1}'
        data_x = vertical_fov
        data_y = accuracy_results[accuracy_key]

        # Plot line segments individually
        for j in range(len(data_x) - 1):
            # Select color based on the value of the current point
            color = 'red' if data_y[j] >= threshold else 'green'
            axs[i].plot([data_x[j], data_x[j+1]], [data_y[j], data_y[j+1]], marker='o', markersize=1, color=color, linewidth=0.5)

        # Draw a horizontal line at the threshold value
        axs[i].axhline(y=threshold, color='red', linestyle='--', linewidth=1)

        axs[i].set_title(f'Row {i+1}')
        axs[i].set_ylim(0, 105)
        axs[i].grid()
        
    axs[3].set_xlabel('Vertical FOV')      
    axs[0].set_ylabel('Accuracy')  

    plt.tight_layout()
    plt.show()


In [243]:
plot_accuracy(v9c_accuracies)

In [101]:
import numpy as np

horz_va1 = []
horz_va2 = []

for i in range(8):
    
    horz_va1.append(np.round(horizontal_va1[i], 2))
    horz_va2.append(np.round(horizontal_va2[i], 2))
    
print(horz_va1)
print(horz_va2)

[-49.14, -37.98, -24.15, -7.56, 9.71, 23.29, 34.85, 44.34]
[-35.98, -25.44, -11.68, 5.19, 19.02, 34.66, 47.08, 56.79]


In [10]:

def plot_accuracy9(accuracy_results_list, model_names, vertical_fov):
    
    threshold = 95  # Set the threshold for accuracy
    fig, axs = plt.subplots(1, 8, figsize=(20, 5))  # Create 1 row and 8 columns of subplots

    # Define specific colors for each model
    colors = ['orange', 'blue', 'cyan', 'magenta', 'green']

    if len(model_names) > len(colors):
        raise ValueError("Not enough unique colors defined for the number of models.")

    for i in range(8):
        for accuracy_results, model_name, color in zip(accuracy_results_list, model_names, colors):
            accuracy_key = f'accuracy{i+1}'
            data_x = vertical_fov
            data_y = accuracy_results[accuracy_key]

            # Plot line segments individually
            for j in range(len(data_x) - 1):
                segment_color = 'red' if data_y[j] >= threshold else color
                axs[i].plot([data_x[j], data_x[j+1]], [data_y[j], data_y[j+1]], marker='o', markersize=.7, color=segment_color, linewidth=0.3, label=model_name if j == 0 else "")

            # Draw a horizontal line at the threshold value
            axs[i].axhline(y=threshold, color='red', linestyle='--', linewidth=.5)

        axs[i].set_title(f'Row{i+1} ({horz_va1[i]}°, {horz_va2[i]}°)', fontsize=10)  # Set the font size here
        axs[i].set_ylim(0, 105)
        axs[i].grid()

    axs[3].set_xlabel('Vertical FOV (degrees°)')      
    axs[0].set_ylabel('Accuracy')  

    # Create a legend for the models and place it above the subplots
    handles = [plt.Line2D([0], [0], color=color, lw=1) for color in colors[:len(model_names)]]
    labels = model_names

    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=len(model_names))
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  # Adjust the top to make room for the legend
    plt.show()


In [472]:
plot_accuracy9(accuracy_results_list, model_names, vertical_fov)

In [163]:
# --- gt_row counts

row_1 = 35
row_2 = 30
row_3 = 39
row_4 = 43
row_5 = 34
row_6 = 37
row_7 = 41
row_8 = 27

##### samp check work

In [278]:

def check_r78(res_list):
    
    nano_r7 = []

    dfx = pd.DataFrame()
    
    for i in range(29):

        df = ret_frame_dets(res_list, i)
        dfx.append(df[((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3300) & (df.iloc[:,3] <= 2160) & (df.iloc[:,3] > 1500)) | ((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3500) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))])
        
        xconf = ret_trimmed_df(dfx, xmin_limits1[6], xmax_limits1[6], y1[-1], y2[-1])

        nano_r7.append(xconf)
        
    return dfx, nano_r7
    

In [306]:

def check_r78(res_list):
    nano_r7 = []
    dfx_list = []  # Use a list to accumulate dataframes
    
    for i in range(29):
        df = ret_frame_dets(res_list, i)
        # filtered_df = df[((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3400) & (df.iloc[:,3] <= 2160) & (df.iloc[:,3] > 1500)) | 
        #                  ((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3500) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]
        dfx_list.append(df)  # Append the filtered dataframe to the list
        
        # Concatenate the list of dataframes into a single dataframe
        df = pd.concat(dfx_list, ignore_index=True)
        
        xconf = ret_trimmed_df(df, xmin_limits1[7], xmax_limits1[7], y1[-1], y2[-1])
        nano_r7.append(xconf)
        
    return df, nano_r7

fdf, r7_win = check_r78(results_nano1)



In [ ]:
# -- vid1
 
# -- row7

# df[((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3400) & (df.iloc[:,3] <= 2160) & (df.iloc[:,3] > 1500)) | 
#                         ((df.iloc[:,2] >= 3050) & (df.iloc[:,2] <= 3500) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]

# df[((df.iloc[:,2] >= 3420) & (df.iloc[:,3] > 1500)) | ((df.iloc[:,2] >= 3493) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]


# -- vid2 - row7


In [390]:

def check_r78(res_list):
    
    nano_r7 = []
    dfx_list = []  # Use a list to accumulate dataframes
    
    for i in range(29):
        
        df = ret_frame_dets(res_list, i)
        
        filtered_df = df[((df.iloc[:,2] >= 3420) & (df.iloc[:,3] > 1500)) | ((df.iloc[:,2] >= 3493) & (df.iloc[:,3] <= 1500) & (df.iloc[:,3] >= 660))]
        
        dfx_list.append(filtered_df)  # Append the filtered dataframe to the list
        
        # Concatenate the list of dataframes into a single dataframe
        dfx = pd.concat(dfx_list, ignore_index=True)
        
        xconf = ret_trimmed_df(df, xmin_limits1[7], xmax_limits1[7], y1[-1], y2[-1])
        nano_r7.append(xconf)
        
    return dfx, nano_r7


In [391]:
fdf88, r88_win = check_r78(results_nano1)


In [392]:
fdf888, r888_win = check_r78(results_nano2)

In [393]:
fdf88

,0,1,2,3,0
0,3564.418945,776.727783,3721.547852,879.947021,0.743559
1,3564.979004,973.250305,3730.929688,1085.503174,0.725217
2,3421.188965,1856.112549,3565.822754,1948.961670,0.716815
3,3551.052734,1095.480713,3675.388672,1200.943115,0.677955
4,3437.324951,1935.346191,3539.529541,2015.586182,0.670485
...,...,...,...,...,...
434,3511.386963,1564.732300,3658.320068,1666.865112,0.614785
435,3560.176270,1405.921143,3681.041504,1501.049072,0.592738
436,3479.461426,1649.425293,3594.210938,1732.896973,0.577883
437,3516.999023,1478.383301,3641.160156,1564.515869,0.577607


In [394]:
fdf888

,0,1,2,3,0
0,3611.759766,1437.272705,3710.928711,1521.841064,0.758786
1,3629.320557,1228.602051,3718.145752,1300.865723,0.716349
2,3587.534668,670.373169,3678.248535,744.061646,0.713845
3,3630.875000,972.874268,3725.855469,1052.664795,0.710508
4,3584.002441,1315.692871,3667.189941,1395.043945,0.700748
...,...,...,...,...,...
401,3487.358398,1743.513428,3569.643555,1816.818115,0.530262
402,3432.350586,2016.750732,3510.400391,2087.111572,0.523328
403,3520.723877,1730.195801,3602.597900,1789.663574,0.498390
404,3517.194336,1660.334351,3599.697266,1730.676147,0.475918


In [395]:

plt.figure(figsize=(11, 5))
plt.scatter(fdf888.iloc[:,0], fdf888.iloc[:,1], s=5)

plt.xlim(0, 3840)
plt.ylim(2160, 0)
plt.tight_layout()
plt.show()

In [397]:

target_path = 'files/roi_detection/videos/vid2_r88.mp4'

tracker2 = sv.ByteTrack(minimum_matching_threshold=.8, track_activation_threshold=.5, lost_track_buffer=24)
track_ids=[]

# Define annotators
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results = r888_win[index]
    boxes, confidences = results

    # Convert the results to Detections object
    
    detections = sv.Detections(
        xyxy=boxes,
        confidence=confidences,
        class_id=np.zeros(len(confidences), dtype=int)  # class_id is 0
    )

    detections = tracker2.update_with_detections(detections)
    # print(detections.tracker_id)
    track_ids.append(detections.tracker_id)
    
    labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]

    annotated_frame = bounding_box_annotator.annotate(
        scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels)
    
    return annotated_frame

# Process the video
sv.process_video(
    source_path=video1, 
    target_path=target_path,
    callback=callback
)

list_tid=[]

for i in range(len(track_ids)):
    track_list = track_ids[i].tolist()
    for j in range(len(track_list)):
        list_tid.append(track_list[j])
        
print(list_tid)
print(len(set(list_tid)))

[1284120, 1284121, 1284122, 1284123, 1284124, 1284125, 1284126, 1284127, 1284128, 1284129, 1284130, 1284131, 1284125, 1284128, 1284126, 1284124, 1284127, 1284123, 1284131, 1284126, 1284121, 1284127, 1284130, 1284121, 1284130, 1284129, 1284123, 1284126, 1284120, 1284129, 1284120, 1284121, 1284129, 1284121, 1284130, 1284125, 1284120, 1284129, 1284130, 1284125, 1284123, 1284128, 1284120, 1284130, 1284123, 1284120, 1284122, 1284130, 1284128, 1284129, 1284120, 1284123, 1284121, 1284129, 1284129, 1284120, 1284121, 1284129, 1284120, 1284129, 1284129, 1284121, 1284123, 1284122, 1284123, 1284121, 1284121, 1284123, 1284120, 1284122, 1284294, 1284123, 1284122]
13


1. Check where we are loosing the detection in row3 (also check whether there are detections in the frames after 20 - do a debug thing)
2. Try tuning the hyper parameters - I believe the current tuned parameters are a good fit - do try the other combinations as well.
3. Look for row7 and 8, as the implementation is differnt for row 7 & 8 
4. After the completeion of video1, move to video2 and then plot lin_plots for 5 models in a single plot


In [ ]:
def return_tdf(df_appended):
     
    xyxy = df_appended.iloc[:, :-1].values
    confidence = df_appended.iloc[:, -1].values

    return (xyxy, confidence)

v9c_res_window = []   # -- window 

for j in range(29):
    
    trim_v9c = ret_frame_dets(results_v9c1, j)
    if trim_v9c.iloc[:,1].min() < 660:
        print(j)
    # else:
    #     print('None..')
    v9c_result_det = return_tdf(trim_v9c)
    
    v9c_res_window.append(v9c_result_det)


In [91]:
trim_v9c.iloc[:,1].min()

680.5613

In [182]:
tracker2 = sv.ByteTrack(minimum_matching_threshold=.8, track_activation_threshold=.5, lost_track_buffer=24)
track_ids=[]

sv.Color.RED

# Define annotators
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results = v9c_res_window[index]
    boxes, confidences = results

    # Convert the results to Detections object
    
    detections = sv.Detections(
        xyxy=boxes,
        confidence=confidences,
        class_id=np.zeros(len(confidences), dtype=int)  # class_id is 0
    )

    detections = tracker2.update_with_detections(detections)
    # print(detections.tracker_id)
    track_ids.append(detections.tracker_id)
    
    labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]

    annotated_frame = bounding_box_annotator.annotate(
        scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels)
    annotated_frame = trace_annotator.annotate(
           scene=annotated_frame.copy(), detections=detections) 
    
    return annotated_frame

# Process the video
sv.process_video(
    source_path=video1, 
    target_path='files/roi_detection/videos/v9c_004.mp4',
    callback=callback
)

track_list2=[]
list_tid2 = []
 
for i in range(len(track_ids)):
    track_list2 = track_ids[i].tolist()
    for j in range(len(track_list2)):
        list_tid2.append(track_list2[j])
 
print(len(set(list_tid2)))

152


In [473]:
v9c_res_window[0]

(array([[     1387.4,      1875.7,      1490.2,      1968.7],
        [     3545.5,      1239.2,      3668.6,      1328.5],
        [     1239.3,      839.46,      1341.5,      938.33],
        [     1784.4,      1426.6,      1879.1,      1516.8],
        [     1847.9,         749,      1963.3,      883.01],
        [     2401.8,      1206.6,      2533.2,        1330],
        [     3573.4,      965.26,      3730.9,      1080.5],
        [     3202.3,      1256.5,      3304.8,      1353.7],
        [     3195.1,      1484.6,      3315.3,      1584.8],
        [     796.45,      1231.6,      909.78,      1321.2],
        [     2329.7,      1121.3,      2455.6,      1242.8],
        [     1862.4,      1387.9,        1964,      1491.8],
        [     2893.5,      791.05,      3001.6,      885.61],
        [     1886.3,      1792.2,      1978.2,      1887.8],
        [     1875.6,      1236.5,      2003.4,      1363.9],
        [     2410.5,      714.26,      2524.2,      825.23],
        

GT crops - 286

In [ ]:

299 (.8, .25)
299 (.8, .3)
293 (.8, .4)

273 (.8, .5) # -- ideal comparing with 'gt' - count

254 (.8, .6)
152 (.8, .7)

215 (.9, .5)
359 (.7, .5)

#### botsort

In [19]:

weight2 = "C:/Users/ashis/Desktop/THESIS/DT_flow/resources/weights/setB/nano.pt"

video1 = "C:/Users/ashis/Desktop/THESIS/DT_flow/resources/data/wc_vid1.AVI"
video2 = "C:/Users/ashis/Desktop/THESIS/DT_flow/resources/data/wc_vid2.mp4"

cfg_path = "C:/Users/ashis/Desktop/THESIS/DT_flow/det_models/detect_track/new_meth/botsort.yaml"

In [31]:
import numpy as np

horz_va1 = []
horz_va2 = []

for i in range(8):
    
    horz_va1.append(np.round(horizontal_va1[i], 2))
    horz_va2.append(np.round(horizontal_va2[i], 2))
    
print(horz_va1)
print(horz_va2)

[-49.14, -37.98, -24.15, -7.56, 9.71, 23.29, 34.85, 44.34]
[-35.98, -25.44, -11.68, 5.19, 19.02, 34.66, 47.08, 56.79]


In [25]:
# -- botsort custom implementation function

from types import SimpleNamespace
import cv2
from ultralytics.utils import yaml_load
from ultralytics.trackers.bot_sort import BOTSORT

# Video input

def cust_botsort(cust_dets, video):
    
    tracker_cfg_path = cfg_path
    cfg_dict = yaml_load(tracker_cfg_path)
    cfg = SimpleNamespace(**cfg_dict)    
    tracker = BOTSORT(cfg)
    video_capture = cv2.VideoCapture(video)
    
    frame_id = 0
    tracking_ids = []

    while video_capture.isOpened():
        if frame_id >=29:
            break
        else:
            ret, frame = video_capture.read()
            if not ret:
                break

        dets = cust_dets[frame_id]
        tracks = tracker.update(dets, frame)

        for track in tracks:
            track_id = int(track[4])
            tracking_ids.append(track_id)

        frame_id += 1

    video_capture.release()
    # out.release()
    # cv2.destroyAllWindows()

    # print(set(tracking_ids))
    # print(len(set(tracking_ids)))
    crop_count = len(set(tracking_ids))
    
    return crop_count

In [12]:
# -- create a list of dataframes for processing

class CustomDataFrame(pd.DataFrame):
    @property
    def _constructor(self):
        return CustomDataFrame

    @property
    def xywh(self):
        return self[['x', 'y', 'w', 'h']].values.tolist()

    @property
    def conf(self):
        return self['conf'].tolist()

    @property
    def cls(self):
        return self['cls'].tolist()

# Function to convert an existing DataFrame to CustomDataFrame
def convert_to_df(df):
    return CustomDataFrame(df)

# Function to create a list of DataFrames for each frame
def get_list_of_dfs(results):
    dataframes_per_frame = []
    for result in results:
        xywh = result.boxes.xywh.numpy()
        conf = result.boxes.conf.numpy()
        cls = result.boxes.cls.numpy()

        data = {
            'x': [item[0] for item in xywh],
            'y': [item[1] for item in xywh],
            'w': [item[2] for item in xywh],
            'h': [item[3] for item in xywh],
            'conf': conf,
            'cls': cls
        }

        df = pd.DataFrame(data)
        custom_df = convert_to_df(df)
        dataframes_per_frame.append(custom_df)

    return dataframes_per_frame

# -- converting the dataframe into a format that the botsort tracker accepts

# Define your DetectionResults class
class DetectionResults:
    def __init__(self, xywh, conf, cls):
        self.xywh = xywh
        self.conf = conf
        self.cls = cls

    @property
    def xywh_list(self):
        return self.xywh.tolist()

    @property
    def conf_list(self):
        return self.conf.tolist()

    @property
    def cls_list(self):
        return self.cls.tolist()

# Function to create DetectionResults for each DataFrame in a list of DataFrames
def get_dets_from_dfs(dataframes):
    detection_results_per_frame = []
    for df in dataframes:
        xywh = df[['x', 'y', 'w', 'h']].values  # Extract as NumPy array
        conf = df['conf'].values                # Extract as NumPy array
        cls = df['cls'].values                  # Extract as NumPy array

        detection_results = DetectionResults(xywh, conf, cls)
        detection_results_per_frame.append(detection_results)

    return detection_results_per_frame


Adjust the following funcs that gives the list of the count of crops by using the above functions : - Done

In [13]:
# -- rows 1-6

def custom_return_df(df, xmin, xmax, y1, y2):
    
    trim_df = df[((df['x']>xmin) & (df['x'] < xmax)) & ((df['y']>y1) & (df['y']<y2) & (df['y']>660))] 
    
    return trim_df

# -- row7
def custom_return_df7(df):
    
    df77 = df[((df['x'] >= 3050) & (df['x'] <= 3400) & (df['y'] <= 2160) & (df['y'] > 1500)) | 
                         ((df['x'] >= 3050) & (df['x'] <= 3500) & (df['y'] <= 1500) & (df['y'] >= 660))] 
    
    return df77

def trim_row78(df, y1, y2):

    df78 = df[(df['y'] >= y1) & (df['y'] <= y2)]

    return df78

# -- row8

def custom_return_df8(df):

    df8 = df[((df['x'] >= 3420) & (df['y'] > 1500)) | ((df['x'] >= 3493) & (df['y'] <= 1500) & (df['y'] >= 660))]
    
    return df8

    # df8 = df[((df['x'] >= 3420) & (df['y'] > y2)) | ((df['x'] >= 3493) & (df['y'] <= ) & (df['y'] >= 660))]

def count_crops_rows_1to6(y1_bounds, y2_bounds, xmin_list, xmax_list, model, video):

    results = {}
    
    for i in range(len(xmin_list)):
        
        count_in_row = []   
        
        for ymin, ymax in zip(y1_bounds, y2_bounds):
            
            list_dfs = get_list_of_dfs(model)      # 'model' -- need to update variable - model = 'results_nano' for different models
           
            cust_dfs_list = []
            
            for j in range(29):
                
                cust_dfs_list.append(custom_return_df(list_dfs[j], xmin_list[i], xmax_list[i], ymin, ymax))
    
            cust_tens_dets = get_dets_from_dfs(cust_dfs_list)
            
            crops_count = cust_botsort(cust_tens_dets, video)
            
            count_in_row.append(crops_count)
            
        print(f'row{i+1}: {count_in_row}')
        
        results[f'row{i+1}'] = count_in_row
        
    return results


In [14]:

def count_crops_rows_78(y1_bounds, y2_bounds, xmin_list, xmax_list, limit, model, video):
        
    # -- here we will write a, if - else cond, where 20 is the threshold for the limit 
    # -- for row 7 - limit > 20 as the limit is 25
    # -- for row 8 - limit < 20 as the limit is 15

    if limit > 20:
        
        # -- logic for row 7
        
        count_in_row = []
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            list_dfs = get_list_of_dfs(model)      # 'model' -- need to update variable - model = 'results_nano' for different models
           
            cust_dfs_list1 = []
            
            for i in range(29):
                
                cust_dfs_list1.append(custom_return_df(list_dfs[i], xmin_list, xmax_list, ymin, ymax))
    
            cust_tens_dets = get_dets_from_dfs(cust_dfs_list1)
            
            crops_count = cust_botsort(cust_tens_dets, video)
            
            count_in_row.append(crops_count)
        
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            list_dfs = get_list_of_dfs(model)      # 'model' -- need to update variable - model = 'results_nano' for different models
           
            cust_dfs_list2 = []
            
            for i in range(29):
                
                dfds = custom_return_df7(list_dfs[i])
                
                cust_dfs_list2.append(trim_row78(dfds, ymin, ymax))
                
            
            cust_tens_dets = get_dets_from_dfs(cust_dfs_list2)
                       
            crops_count = cust_botsort(cust_tens_dets, video)
            
            count_in_row.append(crops_count)
            
        return count_in_row
        
        
    else:
    
    # -- logic for row 8
    
        count_in_row = []   
            
        for ymin, ymax in zip(y1_bounds[:limit], y2_bounds[:limit]):
            
            list_dfs = get_list_of_dfs(model)      # 'model' -- need to update variable - model = 'results_nano' for different models
           
            cust_dfs_list1 = []
            
            for i in range(29):
                
                cust_dfs_list1.append(custom_return_df(list_dfs[i], xmin_list, xmax_list, ymin, ymax))
    
            cust_tens_dets = get_dets_from_dfs(cust_dfs_list1)
            
            crops_count = cust_botsort(cust_tens_dets, video)
            
            count_in_row.append(crops_count)
            
        for ymin, ymax in zip(y1_bounds[limit:], y2_bounds[limit:]):
            
            list_dfs = get_list_of_dfs(model)      # 'model' -- need to update variable - model = 'results_nano' for different models
           
            cust_dfs_list2 = []
            
            for i in range(29):
                
                dfds8 = custom_return_df8(list_dfs[i])
                
                cust_dfs_list2.append(trim_row78(dfds8, ymin, ymax))
    
            cust_tens_dets = get_dets_from_dfs(cust_dfs_list2)
            
            crops_count = cust_botsort(cust_tens_dets, video)
            
            count_in_row.append(crops_count)
            
        return count_in_row


#### v9 initial stage

In [27]:
# -- video 1

# row16_nano1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_nano1, video1)
# row7_nano1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_nano1, video1)
# row8_nano1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_nano1, video1)
# row16_nano1['row7'] = row7_nano1
# row16_nano1['row8'] = row8_nano1
# print(row16_nano1['row7'])
# print(row16_nano1['row8'])
# print()

row16_small1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_small1, video1)
row7_small1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_small1, video1)
row8_small1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_small1, video1)
row16_small1['row7'] = row7_small1
row16_small1['row8'] = row8_small1
print(row16_small1['row7'])
print(row16_small1['row8'])
print()

row16_medium1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_medium1, video1)
row7_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_medium1, video1)
row8_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_medium1, video1)
row16_medium1['row7'] = row7_medium1
row16_medium1['row8'] = row8_medium1
print(row16_medium1['row7'])
print(row16_medium1['row8'])
print()

row16_large1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_large1, video1)
row7_large1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_large1, video1)
row8_large1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_large1, video1)
row16_large1['row7'] = row7_large1
row16_large1['row8'] = row8_large1
print(row16_large1['row7'])
print(row16_large1['row8'])
print()

row16_v9c1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_v9c1, video1)
row7_v9c1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_v9c1, video1)
row8_v9c1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_v9c1, video1)
row16_v9c1['row7'] = row7_v9c1
row16_v9c1['row8'] = row8_v9c1
print(row16_v9c1['row7'])
print(row16_v9c1['row8'])
print()


row1: [4, 10, 12, 12, 13, 14, 14, 15, 16, 16, 17, 17, 17, 18, 19, 19, 19, 20, 21, 21, 22, 22, 23, 24, 24, 25, 26, 26, 26, 27, 27]
row2: [3, 15, 16, 18, 19, 20, 20, 22, 22, 22, 23, 25, 26, 26, 27, 27, 28, 28, 29, 29, 29, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33]
row3: [3, 21, 22, 24, 27, 27, 27, 29, 30, 30, 30, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 34, 35, 35, 36, 36, 37, 38, 38, 38, 38]
row4: [3, 24, 25, 26, 26, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 43, 43, 43]
row5: [2, 20, 21, 21, 21, 22, 22, 22, 24, 24, 24, 26, 26, 27, 28, 28, 28, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33]
row6: [4, 19, 19, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 35, 35, 36, 36, 36, 36, 36]
[6, 19, 20, 22, 23, 24, 26, 26, 28, 29, 29, 30, 30, 31, 31, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 38, 39, 39, 39]
[6, 12, 12, 12, 13, 14, 15, 16, 16, 17, 17, 18, 18, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 

KeyboardInterrupt: 

In [26]:
row16_nano1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_nano1, video1)
row7_nano1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_nano1, video1)
row8_nano1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_nano1, video1)
row16_nano1['row7'] = row7_nano1
row16_nano1['row8'] = row8_nano1
print(row16_nano1['row7'])
print(row16_nano1['row8'])
print()

row1: [3, 8, 10, 11, 12, 12, 13, 14, 15, 15, 16, 16, 16, 17, 17, 17, 17, 18, 18, 19, 20, 20, 20, 21, 21, 21, 21, 21, 22, 23, 23]
row2: [1, 11, 12, 15, 16, 17, 17, 19, 20, 21, 22, 24, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29]
row3: [4, 20, 21, 23, 25, 25, 26, 27, 28, 28, 29, 29, 29, 30, 30, 30, 30, 30, 30, 31, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 35]
row4: [2, 23, 23, 24, 24, 26, 27, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 41, 42, 42, 42]
row5: [1, 18, 19, 19, 20, 21, 21, 21, 22, 22, 22, 24, 24, 25, 26, 26, 26, 28, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 31, 31, 31]
row6: [4, 17, 18, 18, 20, 21, 21, 23, 24, 25, 26, 28, 28, 28, 29, 29, 29, 29, 29, 31, 31, 31, 32, 32, 33, 33, 33, 33, 33, 34, 34]
[3, 13, 15, 18, 20, 20, 22, 22, 23, 24, 24, 25, 25, 26, 26, 27, 28, 30, 30, 30, 31, 32, 32, 32, 32, 32, 32, 32, 33, 34, 34]
[3, 10, 11, 12, 12, 13, 13, 13, 13, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 1

In [ ]:
# -- small - (vid1)
row16_small1 = {'row1': [4, 10, 12, 12, 13, 14, 14, 15, 16, 16, 17, 17, 17, 18, 19, 19, 19, 20, 21, 21, 22, 22, 23, 24, 24, 25, 26, 26, 26, 27, 27],
        'row2': [3, 15, 16, 18, 19, 20, 20, 22, 22, 22, 23, 25, 26, 26, 27, 27, 28, 28, 29, 29, 29, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33],
        'row3': [3, 21, 22, 24, 27, 27, 27, 29, 30, 30, 30, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 34, 35, 35, 36, 36, 37, 38, 38, 38, 38],
        'row4': [3, 24, 25, 26, 26, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 43, 43, 43],
        'row5': [2, 20, 21, 21, 21, 22, 22, 22, 24, 24, 24, 26, 26, 27, 28, 28, 28, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33],
        'row6': [4, 19, 19, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 35, 35, 36, 36, 36, 36, 36],
        'row7': [6, 19, 20, 22, 23, 24, 26, 26, 28, 29, 29, 30, 30, 31, 31, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 38, 39, 39, 39],
        'row8': [6, 12, 12, 12, 13, 14, 15, 16, 16, 17, 17, 18, 18, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 22, 22, 22, 22, 22, 22]}

        # - medium 

row16_medium1 = {'row1': [3, 9, 10, 11, 13, 14, 14, 15, 16, 16, 17, 17, 18, 20, 20, 21, 21, 22, 23, 23, 24, 24, 24, 26, 26, 26, 27, 27, 28, 28, 28],
        'row2': [4, 16, 17, 19, 20, 21, 21, 23, 23, 23, 24, 25, 25, 25, 26, 27, 28, 28, 29, 29, 29, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33],
        'row3': [2, 19, 22, 24, 26, 26, 27, 28, 29, 29, 29, 30, 30, 31, 31, 31, 31, 31, 31, 31, 33, 34, 34, 34, 35, 35, 35, 35, 36, 37, 37],
        'row4': [2, 24, 25, 26, 26, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 41, 42, 43, 43],
        'row5': [1, 20, 21, 22, 22, 23, 23, 23, 25, 25, 25, 27, 27, 28, 29, 29, 29, 31, 31, 31, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34],
        'row6': [4, 19, 20, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 35, 36, 36, 37, 37, 37, 37],
        'row7': [8, 19, 22, 23, 24, 24, 26, 26, 28, 29, 29, 30, 30, 31, 31, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 38, 39, 39, 39],
        'row8': [6, 13, 13, 13, 13, 14, 15, 16, 16, 17, 17, 18, 19, 20, 21, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 24, 24, 24, 24, 24, 24]}

        # -- large

row16_large1={'row1': [4, 9, 12, 12, 13, 14, 14, 15, 16, 16, 17, 17, 18, 19, 21, 21, 21, 21, 22, 22, 23, 24, 24, 25, 25, 26, 26, 27, 27, 27, 27],
        'row2': [3, 15, 16, 19, 19, 20, 20, 22, 22, 22, 23, 24, 24, 24, 24, 25, 26, 27, 28, 28, 28, 28, 28, 29, 29, 30, 31, 31, 31, 32, 32],
        'row3': [4, 21, 23, 25, 27, 27, 27, 29, 30, 31, 31, 32, 32, 33, 33, 33, 33, 33, 33, 33, 35, 36, 36, 36, 37, 37, 37, 37, 38, 39, 39],
        'row4': [2, 24, 25, 26, 26, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 43, 43, 43],
        'row5': [2, 20, 21, 21, 21, 22, 22, 22, 24, 24, 24, 26, 26, 27, 28, 28, 28, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33],
        'row6': [4, 19, 20, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 35, 36, 37, 37, 37, 37, 37],
        'row7': [9, 20, 21, 22, 23, 23, 25, 26, 28, 30, 30, 31, 31, 32, 32, 33, 34, 35, 35, 35, 36, 37, 37, 37, 38, 38, 38, 39, 40, 40, 40],
        'row8': [6, 13, 13, 13, 13, 15, 16, 17, 17, 18, 19, 20, 20, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 24, 24, 24, 24, 24, 24]}

        # -- v9c

row16_v9c1 = {'row1': [3, 11, 12, 13, 14, 15, 15, 16, 17, 17, 18, 19, 20, 21, 22, 22, 22, 23, 23, 23, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29],
        'row2': [2, 17, 18, 20, 20, 22, 22, 24, 24, 24, 25, 26, 27, 27, 28, 28, 29, 29, 30, 30, 30, 30, 30, 31, 31, 32, 33, 34, 34, 35, 35],
        'row3': [4, 22, 23, 25, 27, 27, 28, 29, 30, 31, 31, 32, 32, 33, 33, 33, 33, 33, 33, 33, 35, 36, 36, 36, 37, 37, 38, 39, 39, 39, 39],
        'row4': [2, 23, 24, 25, 25, 27, 29, 30, 31, 31, 33, 35, 35, 36, 36, 37, 37, 37, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 44, 44, 44],
        'row5': [1, 20, 21, 21, 22, 23, 23, 23, 25, 25, 25, 27, 27, 28, 29, 29, 29, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34],
        'row6': [4, 19, 20, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 30, 30, 30, 31, 32, 33, 33, 33, 34, 34, 35, 36, 36, 36, 36, 38, 38],
        'row7': [5, 18, 19, 20, 22, 23, 25, 25, 27, 29, 29, 30, 31, 31, 31, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 38, 38, 39, 39, 39],
        'row8': [5, 13, 13, 14, 14, 15, 16, 17, 17, 18, 18, 19, 19, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 23, 24, 24, 24, 24, 24]}

# -- nano2 

row1: [3, 7, 8, 8, 9, 10, 11, 12, 12, 14, 14, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20]
row2: [2, 7, 8, 10, 10, 10, 12, 14, 14, 14, 14, 15, 15, 16, 16, 16, 16, 16, 16, 17, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20]
row3: [5, 20, 22, 23, 25, 25, 28, 29, 29, 30, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 36, 36, 37, 37, 37, 38, 38]
row4: [2, 18, 20, 20, 21, 21, 21, 21, 22, 23, 24, 25, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 29, 30, 30]
row5: [4, 20, 21, 22, 23, 23, 25, 25, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 29, 29, 30, 30, 31, 32, 32, 32, 32, 32, 32, 32, 32]
row6: [3, 18, 21, 22, 23, 23, 24, 25, 27, 28, 28, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 35, 35, 35, 35, 35, 35]
[2, 12, 13, 16, 16, 18, 18, 18, 19, 20, 21, 23, 23, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27]
[4, 10, 11, 12, 13, 14, 14, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 20, 20, 21, 21, 21, 21, 21, 21, 21]

In [15]:
def cust_botsort(cust_dets, video):
    # Load tracker configuration
    cfg_dict = yaml_load(cfg_path)
    cfg = SimpleNamespace(**cfg_dict)    
    tracker = BOTSORT(cfg)
    video_capture = cv2.VideoCapture(video)
    
    if not video_capture.isOpened():
        print(f"Error: Unable to open video file {video}")
        return
    
    frame_id = 0
    tracking_ids = []

    try:
        while video_capture.isOpened():
            ret, frame = video_capture.read()
            if not ret:
                break

            if frame_id < len(cust_dets):
                dets = cust_dets[frame_id]
                tracks = tracker.update(dets, frame)

                for track in tracks:
                    track_id = int(track[4])
                    tracking_ids.append(track_id)

            frame_id += 1
    finally:
        video_capture.release()
        cv2.destroyAllWindows()

    crop_count = len(set(tracking_ids))
    return crop_count

In [75]:

# -- video 2

row16_nano2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_nano2, video2)
row7_nano2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_nano2, video2)
row8_nano2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_nano2, video2)
row16_nano2['row7'] = row7_nano2
row16_nano2['row8'] = row8_nano2
print(row16_nano2['row7'])
print(row16_nano2['row8'])
print()


row1: [3, 7, 8, 8, 9, 10, 11, 12, 12, 14, 14, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20]
row2: [2, 7, 8, 10, 10, 10, 12, 14, 14, 14, 14, 15, 15, 16, 16, 16, 16, 16, 16, 17, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20]
row3: [5, 20, 22, 23, 25, 25, 28, 29, 29, 30, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 36, 36, 37, 37, 37, 38, 38]
row4: [2, 18, 20, 20, 21, 21, 21, 21, 22, 23, 24, 25, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 29, 30, 30]
row5: [4, 20, 21, 22, 23, 23, 25, 25, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 29, 29, 30, 30, 31, 32, 32, 32, 32, 32, 32, 32, 32]
row6: [3, 18, 21, 22, 23, 23, 24, 25, 27, 28, 28, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 35, 35, 35, 35, 35, 35]
[2, 12, 13, 16, 16, 18, 18, 18, 19, 20, 21, 23, 23, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27]
[4, 10, 11, 12, 13, 14, 14, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 20, 20, 21, 21

In [91]:
print(row16_small1)

{'row1': [4, 10, 12, 12, 13, 14, 14, 15, 16, 16, 17, 17, 17, 18, 19, 19, 19, 20, 21, 21, 22, 22, 23, 24, 24, 25, 26, 26, 26, 27, 27], 'row2': [3, 15, 16, 18, 19, 20, 20, 22, 22, 22, 23, 25, 26, 26, 27, 27, 28, 28, 29, 29, 29, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33], 'row3': [3, 21, 22, 24, 27, 27, 27, 29, 30, 30, 30, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 34, 35, 35, 36, 36, 37, 38, 38, 38, 38], 'row4': [3, 24, 25, 26, 26, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 43, 43, 43], 'row5': [2, 20, 21, 21, 21, 22, 22, 22, 24, 24, 24, 26, 26, 27, 28, 28, 28, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33], 'row6': [4, 19, 19, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 35, 35, 36, 36, 36, 36, 36], 'row7': [6, 19, 20, 22, 23, 24, 26, 26, 28, 29, 29, 30, 30, 31, 31, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 38, 39, 39, 39], 'row8': [6, 12, 12, 12, 13, 14, 15, 16, 16, 17, 17, 18, 18, 19, 20, 

In [87]:

# -- video 2

row16_nano2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_nano2, video2)
row7_nano2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_nano2, video2)
row8_nano2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_nano2, video2)
row16_nano2['row7'] = row7_nano2
row16_nano2['row8'] = row8_nano2
print(row16_nano2['row7'])
print(row16_nano2['row8'])
print()


row16_small2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_small2, video2)
row7_small2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_small2, video2)
row8_small2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_small2, video2)
row16_small2['row7'] = row7_small2
row16_small2['row8'] = row8_small2
print(row16_small2['row7'])
print(row16_small2['row8'])
print()

row16_medium2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_medium2, video2)
row7_medium2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_medium2, video2)
row8_medium2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_medium2, video2)
row16_medium2['row7'] = row7_medium2
row16_medium2['row8'] = row8_medium2
print(row16_medium2['row7'])
print(row16_medium2['row8'])
print()

row16_large2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_large2, video2)
row7_large2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_large2, video2)
row8_large2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_large2, video2)
row16_large2['row7'] = row7_large2
row16_large2['row8'] = row8_large2
print(row16_large2['row7'])
print(row16_large2['row8'])
print()

row16_v9c2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_v9c2, video2)
row7_v9c2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_v9c2, video2)
row8_v9c2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_v9c2, video2)
row16_v9c2['row7'] = row7_v9c2
row16_v9c2['row8'] = row8_v9c2
print(row16_v9c2['row7'])
print(row16_v9c2['row8'])
print()

row1: [5, 10, 12, 13, 14, 14, 14, 14, 14, 16, 16, 18, 18, 19, 20, 20, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24]
row2: [5, 12, 13, 15, 16, 17, 19, 19, 20, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25]
row3: [6, 24, 25, 26, 26, 27, 29, 30, 30, 31, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 37, 38, 38, 39, 39, 39, 40, 40]
row4: [3, 18, 19, 19, 20, 21, 21, 22, 22, 23, 24, 25, 25, 25, 25, 25, 26, 26, 26, 28, 28, 28, 29, 29, 29, 30, 31, 31, 31, 32, 32]
row5: [5, 20, 21, 22, 23, 23, 25, 25, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 31, 31, 31, 33, 33, 33, 33, 33]
row6: [3, 20, 21, 22, 23, 23, 26, 26, 27, 28, 28, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33, 33, 34, 35, 35, 35, 35, 36, 36]
[1, 14, 16, 17, 17, 18, 19, 19, 19, 20, 21, 22, 23, 23, 23, 23, 24, 24, 25, 25, 26, 26, 26, 26, 26, 26, 26, 27, 28, 28, 28]
[6, 10, 12, 14, 15, 16, 16, 16, 17, 18, 18, 20, 20, 20, 20, 21, 21, 21, 21, 23, 24, 24, 24, 24, 

In [ ]:
# --- gt_row counts (vid1)

row_1 = 35
row_2 = 30
row_3 = 39
row_4 = 43
row_5 = 34
row_6 = 37
row_7 = 41
row_8 = 27

In [92]:
# -- Accuracies

# -- nano
gt_nano1 = gen_gt_results('nano', 'A')
accuracy_results_nano1 = calculate_accuracies(gt_nano1, row16_nano1) 

gt_nano2 = gen_gt_results('nano', 'B')
accuracy_results_nano2 = calculate_accuracies(gt_nano2, row16_nano2) 

nano_accuracies = get_avg_accuracy(accuracy_results_nano1, accuracy_results_nano2)

# -- small
gt_small1 = gen_gt_results('nano', 'A')
accuracy_results_small1 = calculate_accuracies(gt_small1, row16_small1) 

gt_small2 = gen_gt_results('nano', 'B')
accuracy_results_small2 = calculate_accuracies(gt_small2, row16_small2) 

small_accuracies = get_avg_accuracy(accuracy_results_small1, accuracy_results_small2)

#-- medium
gt_medium1 = gen_gt_results('nano', 'A')
accuracy_results_medium1 = calculate_accuracies(gt_medium1, row16_medium1) 

gt_medium2 = gen_gt_results('nano', 'B')
accuracy_results_medium2 = calculate_accuracies(gt_medium2, row16_medium2) 

medium_accuracies = get_avg_accuracy(accuracy_results_medium1, accuracy_results_medium2)

# -- large
gt_large1 = gen_gt_results('nano', 'A')
accuracy_results_large1 = calculate_accuracies(gt_large1, row16_large1) 

gt_large2 = gen_gt_results('nano', 'B')
accuracy_results_large2 = calculate_accuracies(gt_large2, row16_large2) 

large_accuracies = get_avg_accuracy(accuracy_results_large1, accuracy_results_large2)

# -- v9c
gt_v9c1 = gen_gt_results('nano', 'A')
accuracy_results_v9c1 = calculate_accuracies(gt_v9c1, row16_v9c1) 

gt_v9c2 = gen_gt_results('nano', 'B')
accuracy_results_v9c2 = calculate_accuracies(gt_v9c2, row16_v9c2) 

v9c_accuracies = get_avg_accuracy(accuracy_results_v9c1, accuracy_results_v9c2)


In [93]:
accuracy_results_list = [nano_accuracies, small_accuracies, medium_accuracies, v9c_accuracies, large_accuracies]
model_names = ['nano', 'small', 'medium', 'v9c', 'large']

In [20]:

def plot_accuracy9(accuracy_results_list, model_names, vertical_fov):
    
    threshold = 95  # Set the threshold for accuracy
    fig, axs = plt.subplots(1, 8, figsize=(20, 5))  # Create 1 row and 8 columns of subplots

    # Define specific colors for each model
    colors = ['orange', 'blue', 'cyan', 'magenta', 'green']

    if len(model_names) > len(colors):
        raise ValueError("Not enough unique colors defined for the number of models.")

    for i in range(8):
        for accuracy_results, model_name, color in zip(accuracy_results_list, model_names, colors):
            accuracy_key = f'accuracy{i+1}'
            data_x = vertical_fov
            data_y = accuracy_results[accuracy_key]

            # Plot line segments individually
            for j in range(len(data_x) - 1):
                segment_color = 'red' if data_y[j] >= threshold else color
                axs[i].plot([data_x[j], data_x[j+1]], [data_y[j], data_y[j+1]], marker='o', markersize=.7, color=segment_color, linewidth=0.3, label=model_name if j == 0 else "")

            # Draw a horizontal line at the threshold value
            axs[i].axhline(y=threshold, color='red', linestyle='--', linewidth=.5)

        axs[i].set_title(f'Row{i+1} ({horz_va1[i]}°, {horz_va2[i]}°)', fontsize=10)  # Set the font size here
        axs[i].set_ylim(50, 105)
        axs[i].grid()

    axs[3].set_xlabel('Vertical FOV (degrees°)')      
    axs[0].set_ylabel('Accuracy')  

    # Create a legend for the models and place it above the subplots
    handles = [plt.Line2D([0], [0], color=color, lw=1) for color in colors[:len(model_names)]]
    labels = model_names

    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=len(model_names))
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  # Adjust the top to make room for the legend
    plt.show()


In [107]:
plot_accuracy9(accuracy_results_list, model_names, vertical_fov)

Do tha analysis plots for the remaining object detection models - YOLOv9 (nano, small, medium, c) & YOLOv10 (nano, small, medium, balanced, large)

Train version 9 and 10 models, then save those weights.
Perform detection using those weights as we did earlier and then get the tracking performance plots.

## YOLO9

Detection Results

In [16]:
# -- video1 - set B

results_tiny1 = detect_results('B', 'v9t', video1)
results_small1 = detect_results('B', 'v9s', video1)
results_medium1 = detect_results('B', 'v9m', video1)
# results_v9c1 = detect_results('B', 'v9c', video1)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to files\roi_detection\custom_botsort_detections\predict11

WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for

In [17]:
# -- video2 - set A

results_tiny2 = detect_results('A', 'v9t', video2)
results_small2 = detect_results('A', 'v9s', video2)
results_medium2 = detect_results('A', 'v9m', video2)
# results_v9c2 = detect_results('A', 'v9c', video2)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to files\roi_detection\custom_botsort_detections\predict14

WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for

In [26]:
# -- video 1

row16_tiny1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_tiny1, video1)
row7_tiny1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_tiny1, video1)
row8_tiny1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_tiny1, video1)
row16_tiny1['row7'] = row7_tiny1
row16_tiny1['row8'] = row8_tiny1
print(row16_tiny1['row7'])
print(row16_tiny1['row8'])
print()

row16_small1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_small1, video1)
row7_small1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_small1, video1)
row8_small1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_small1, video1)
row16_small1['row7'] = row7_small1
row16_small1['row8'] = row8_small1
print(row16_small1['row7'])
print(row16_small1['row8'])
print()

row16_medium1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_medium1, video1)
row7_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_medium1, video1)
row8_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_medium1, video1)
row16_medium1['row7'] = row7_medium1
row16_medium1['row8'] = row8_medium1
print(row16_medium1['row7'])
print(row16_medium1['row8'])
print()

# row16_v9c1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_v9c1, video1)
# row7_v9c1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_v9c1, video1)
# row8_v9c1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_v9c1, video1)
# row16_v9c1['row7'] = row7_v9c1
# row16_v9c1['row8'] = row8_v9c1
# print(row16_v9c1['row7'])
# print(row16_v9c1['row8'])
# print()


row1: [4, 9, 10, 11, 14, 14, 14, 15, 16, 17, 17, 17, 17, 18, 18, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25]
row2: [3, 15, 17, 18, 19, 20, 20, 23, 24, 24, 25, 26, 27, 27, 28, 28, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 33, 33]
row3: [3, 21, 22, 24, 26, 26, 27, 28, 29, 29, 30, 31, 31, 32, 33, 33, 33, 33, 33, 34, 35, 36, 36, 37, 37, 37, 38, 38, 38, 38, 38]
row4: [4, 23, 24, 25, 25, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 37, 37, 37, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 44, 44, 44]
row5: [2, 20, 21, 21, 21, 22, 22, 22, 24, 24, 24, 26, 26, 27, 28, 28, 28, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 33, 33, 33, 33]
row6: [5, 19, 20, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 31, 31, 31, 31, 33, 34, 34, 35, 35, 36, 36, 37, 37, 37, 37, 37]
[4, 17, 19, 21, 22, 23, 25, 25, 27, 27, 27, 28, 29, 29, 29, 30, 31, 31, 31, 32, 33, 34, 34, 34, 35, 35, 35, 35, 36, 36, 36]
[5, 13, 14, 15, 15, 16, 18, 18, 18, 19, 19, 20, 20, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 2

In [23]:
row16_v9c1 = {  'row1': [3, 11, 12, 13, 14, 15, 15, 16, 17, 17, 18, 19, 20, 21, 22, 22, 22, 23, 23, 23, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29],
                'row2': [2, 17, 18, 20, 20, 22, 22, 24, 24, 24, 25, 26, 27, 27, 28, 28, 29, 29, 30, 30, 30, 30, 30, 31, 31, 32, 33, 34, 34, 35, 35],
                'row3': [4, 22, 23, 25, 27, 27, 28, 29, 30, 31, 31, 32, 32, 33, 33, 33, 33, 33, 33, 33, 35, 36, 36, 36, 37, 37, 38, 39, 39, 39, 39],
                'row4': [2, 23, 24, 25, 25, 27, 29, 30, 31, 31, 33, 35, 35, 36, 36, 37, 37, 37, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 44, 44, 44],
                'row5': [1, 20, 21, 21, 22, 23, 23, 23, 25, 25, 25, 27, 27, 28, 29, 29, 29, 31, 31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34],
                'row6': [4, 19, 20, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 30, 30, 30, 31, 32, 33, 33, 33, 34, 34, 35, 36, 36, 36, 36, 38, 38],
                'row7': [5, 18, 19, 20, 22, 23, 25, 25, 27, 29, 29, 30, 31, 31, 31, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 38, 38, 39, 39, 39],
                'row8': [5, 13, 13, 14, 14, 15, 16, 17, 17, 18, 18, 19, 19, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 23, 24, 24, 24, 24, 24]}


In [28]:

# -- video 2

row16_tiny2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_tiny2, video2)
row7_tiny2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_tiny2, video2)
row8_tiny2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_tiny2, video2)
row16_tiny2['row7'] = row7_tiny2
row16_tiny2['row8'] = row8_tiny2
print(row16_tiny2['row7'])
print(row16_tiny2['row8'])
print()


row16_small2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_small2, video2)
row7_small2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_small2, video2)
row8_small2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_small2, video2)
row16_small2['row7'] = row7_small2
row16_small2['row8'] = row8_small2
print(row16_small2['row7'])
print(row16_small2['row8'])
print()

row16_medium2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_medium2, video2)
row7_medium2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_medium2, video2)
row8_medium2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_medium2, video2)
row16_medium2['row7'] = row7_medium2
row16_medium2['row8'] = row8_medium2
print(row16_medium2['row7'])
print(row16_medium2['row8'])
print()

# row16_v9c2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_v9c2, video2)
# row7_v9c2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_v9c2, video2)
# row8_v9c2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_v9c2, video2)
# row16_v9c2['row7'] = row7_v9c2
# row16_v9c2['row8'] = row8_v9c2
# print(row16_v9c2['row7'])
# print(row16_v9c2['row8'])
# print()

row1: [2, 6, 9, 9, 9, 9, 10, 11, 12, 14, 15, 15, 15, 15, 16, 16, 17, 17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20]
row2: [2, 6, 6, 7, 8, 10, 11, 11, 11, 12, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17]
row3: [4, 18, 19, 20, 20, 23, 26, 27, 27, 29, 31, 31, 31, 31, 31, 31, 32, 33, 33, 33, 33, 34, 35, 35, 36, 36, 37, 37, 37, 37, 38]
row4: [2, 19, 20, 20, 21, 21, 21, 21, 22, 23, 24, 25, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 29, 30, 30]
row5: [5, 20, 21, 22, 23, 23, 25, 25, 25, 25, 25, 26, 27, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 31, 31, 31, 32, 32, 32, 33, 33]
row6: [3, 17, 20, 22, 23, 23, 25, 25, 27, 28, 28, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34]
[1, 9, 10, 10, 10, 12, 12, 12, 12, 12, 12, 14, 14, 14, 14, 14, 14, 15, 15, 16, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19]
[2, 7, 7, 7, 8, 8, 9, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 

In [27]:
row16_v9c2 = {  'row1': [6, 13, 14, 14, 16, 16, 16, 16, 16, 19, 19, 21, 21, 22, 23, 23, 23, 23, 24, 24, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27],
                'row2': [4, 9, 14, 15, 17, 17, 19, 21, 21, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25],
                'row3': [5, 19, 21, 23, 25, 27, 29, 30, 30, 31, 33, 33, 33, 33, 34, 34, 36, 37, 37, 37, 37, 38, 38, 38, 39, 39, 40, 40, 42, 42, 42],
                'row4': [2, 19, 20, 20, 21, 21, 21, 22, 22, 23, 24, 25, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 30, 31, 31],
                'row5': [5, 20, 21, 22, 23, 23, 25, 25, 25, 26, 27, 27, 27, 27, 27, 27, 28, 28, 28, 29, 30, 31, 31, 33, 33, 33, 34, 34, 34, 34, 34],
                'row6': [2, 18, 21, 22, 23, 23, 25, 26, 27, 28, 29, 30, 30, 31, 31, 31, 31, 31, 31, 32, 33, 33, 33, 34, 34, 36, 36, 36, 36, 36, 36],
                'row7': [3, 16, 17, 18, 18, 20, 20, 20, 20, 21, 22, 23, 23, 23, 23, 24, 25, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28, 29, 30, 30, 30],
                'row8': [8, 12, 12, 14, 15, 16, 16, 17, 17, 19, 19, 21, 21, 21, 21, 23, 23, 23, 23, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26]}

modify the following cells for tiny model etc (get v9c results from the previos generated results)

In [29]:
# -- Accuracies

# -- nano
gtA = gen_gt_results('nano', 'A')
accuracy_results_tiny1 = calculate_accuracies(gtA, row16_tiny1) 
gtB = gen_gt_results('nano', 'B')
accuracy_results_tiny2 = calculate_accuracies(gtB, row16_tiny2) 

tiny_accuracies = get_avg_accuracy(accuracy_results_tiny1, accuracy_results_tiny2)

# -- small
accuracy_results_small1 = calculate_accuracies(gtA, row16_small1) 
accuracy_results_small2 = calculate_accuracies(gtB, row16_small2) 

small_accuracies = get_avg_accuracy(accuracy_results_small1, accuracy_results_small2)

#-- medium
accuracy_results_medium1 = calculate_accuracies(gtA, row16_medium1) 
accuracy_results_medium2 = calculate_accuracies(gtB, row16_medium2) 

medium_accuracies = get_avg_accuracy(accuracy_results_medium1, accuracy_results_medium2)

# -- v9c
accuracy_results_v9c1 = calculate_accuracies(gtA, row16_v9c1) 
accuracy_results_v9c2 = calculate_accuracies(gtB, row16_v9c2) 

v9c_accuracies = get_avg_accuracy(accuracy_results_v9c1, accuracy_results_v9c2)


In [50]:
yolov9_results = [tiny_accuracies, small_accuracies, medium_accuracies, v9c_accuracies]
v9_names = ['tiny', 'small', 'medium', 'v9c']

plot_accuracy9(yolov9_results, v9_names, vertical_fov)


## YOLOv10

Detection Results

In [39]:
# -- video1 - set B

results_nano1 = detect_results('B', '10n', video1)
results_small1 = detect_results('B', '10s', video1)
results_medium1 = detect_results('B', '10m', video1)
results_bal1 = detect_results('B', '10b', video1)

# -- video2 - set A

results_nano2 = detect_results('A', '10n', video2)
results_small2 = detect_results('A', '10s', video2)
results_medium2 = detect_results('A', '10m', video2)
results_bal2 = detect_results('A', '10b', video2)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to files\roi_detection\custom_botsort_detections\predict17

WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for

In [40]:
# -- video 1

row16_nano1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_nano1, video1)
row7_nano1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_nano1, video1)
row8_nano1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_nano1, video1)
row16_nano1['row7'] = row7_nano1
row16_nano1['row8'] = row8_nano1
print(row16_nano1['row7'])
print(row16_nano1['row8'])
print()

row16_small1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_small1, video1)
row7_small1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_small1, video1)
row8_small1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_small1, video1)
row16_small1['row7'] = row7_small1
row16_small1['row8'] = row8_small1
print(row16_small1['row7'])
print(row16_small1['row8'])
print()

row16_medium1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_medium1, video1)
row7_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_medium1, video1)
row8_medium1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_medium1, video1)
row16_medium1['row7'] = row7_medium1
row16_medium1['row8'] = row8_medium1
print(row16_medium1['row7'])
print(row16_medium1['row8'])
print()

row16_bal1 = count_crops_rows_1to6(y1, y2, xmin_limits1[:6], xmax_limits1[:6], results_bal1, video1)
row7_bal1 = count_crops_rows_78(y1, y2, xmin_limits1[6], xmax_limits1[6], 25, results_bal1, video1)
row8_bal1 = count_crops_rows_78(y1, y2, xmin_limits1[7], xmax_limits1[7], 15, results_bal1, video1)
row16_bal1['row7'] = row7_bal1
row16_bal1['row8'] = row8_bal1
print(row16_bal1['row7'])
print(row16_bal1['row8'])
print()


row1: [0, 2, 6, 6, 8, 8, 9, 12, 13, 14, 16, 16, 16, 16, 16, 17, 17, 18, 19, 19, 20, 20, 20, 21, 21, 22, 22, 22, 22, 22, 22]
row2: [0, 9, 10, 13, 14, 14, 15, 17, 18, 19, 20, 21, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25]
row3: [2, 15, 17, 20, 24, 24, 24, 27, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 30, 30, 31, 31, 31, 31, 32, 32, 32, 32, 32]
row4: [2, 12, 16, 17, 20, 22, 24, 25, 26, 26, 29, 30, 30, 32, 32, 33, 33, 33, 34, 35, 35, 36, 36, 37, 37, 38, 39, 39, 40, 40, 40]
row5: [1, 13, 16, 16, 19, 20, 21, 21, 21, 21, 22, 24, 24, 25, 26, 26, 26, 27, 27, 28, 28, 28, 28, 28, 28, 29, 29, 29, 31, 31, 31]
row6: [1, 14, 16, 16, 20, 21, 21, 22, 22, 23, 23, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 27, 29, 29, 29, 29, 29, 30, 30]
[2, 9, 13, 14, 15, 16, 19, 20, 22, 22, 23, 24, 24, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 27, 29, 29, 29, 29, 30, 30]
[2, 8, 9, 10, 12, 13, 13, 13, 13, 15, 16, 17, 18, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21

In [41]:

# -- video 2

row16_nano2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_nano2, video2)
row7_nano2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_nano2, video2)
row8_nano2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_nano2, video2)
row16_nano2['row7'] = row7_nano2
row16_nano2['row8'] = row8_nano2
print(row16_nano2['row7'])
print(row16_nano2['row8'])
print()

row16_small2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_small2, video2)
row7_small2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_small2, video2)
row8_small2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_small2, video2)
row16_small2['row7'] = row7_small2
row16_small2['row8'] = row8_small2
print(row16_small2['row7'])
print(row16_small2['row8'])
print()

row16_medium2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_medium2, video2)
row7_medium2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_medium2, video2)
row8_medium2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_medium2, video2)
row16_medium2['row7'] = row7_medium2
row16_medium2['row8'] = row8_medium2
print(row16_medium2['row7'])
print(row16_medium2['row8'])
print()

row16_bal2 = count_crops_rows_1to6(y1, y2, xmin_limits2[:6], xmax_limits2[:6], results_bal2, video2)
row7_bal2 = count_crops_rows_78(y1, y2, xmin_limits2[6], xmax_limits2[6], 21, results_bal2, video2)
row8_bal2 = count_crops_rows_78(y1, y2, xmin_limits2[7], xmax_limits2[7], 15, results_bal2, video2)
row16_bal2['row7'] = row7_bal2
row16_bal2['row8'] = row8_bal2
print(row16_bal2['row7'])
print(row16_bal2['row8'])
print()


row1: [1, 2, 4, 4, 5, 6, 7, 8, 8, 10, 11, 11, 12, 13, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 17, 17, 17, 17, 17, 17, 17]
row2: [1, 3, 5, 5, 6, 8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13]
row3: [2, 10, 11, 12, 13, 15, 16, 18, 19, 19, 20, 21, 23, 23, 23, 23, 23, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27]
row4: [2, 10, 14, 15, 16, 17, 18, 18, 18, 19, 21, 21, 21, 21, 22, 23, 23, 23, 23, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25]
row5: [3, 14, 16, 19, 19, 19, 23, 23, 23, 25, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 30, 30, 30, 30, 30]
row6: [2, 11, 14, 15, 18, 19, 21, 21, 21, 22, 22, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 26, 26, 27, 27, 27, 27, 28, 28]
[0, 0, 2, 4, 5, 6, 6, 7, 8, 9, 10, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13]
[1, 1, 1, 1, 2, 3, 3, 4, 6, 6, 7, 8, 8, 8, 8, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11]

row1: [3, 

In [42]:
# -- Accuracies

# -- gt
gtA = gen_gt_results('nano', 'A')
gtB = gen_gt_results('nano', 'B')

# -- nano
accuracy_results_nano1 = calculate_accuracies(gtA, row16_nano1) 
accuracy_results_nano2 = calculate_accuracies(gtB, row16_nano2) 
nano_accuracies = get_avg_accuracy(accuracy_results_nano1, accuracy_results_nano2)

# -- small
accuracy_results_small1 = calculate_accuracies(gtA, row16_small1) 
accuracy_results_small2 = calculate_accuracies(gtB, row16_small2) 
small_accuracies = get_avg_accuracy(accuracy_results_small1, accuracy_results_small2)

#-- medium
accuracy_results_medium1 = calculate_accuracies(gtA, row16_medium1) 
accuracy_results_medium2 = calculate_accuracies(gtB, row16_medium2) 
medium_accuracies = get_avg_accuracy(accuracy_results_medium1, accuracy_results_medium2)

# -- large
accuracy_results_large1 = calculate_accuracies(gtA, row16_large1) 
accuracy_results_large2 = calculate_accuracies(gtB, row16_large2) 
large_accuracies = get_avg_accuracy(accuracy_results_large1, accuracy_results_large2)


In [51]:
yolov10_results = [nano_accuracies, small_accuracies, medium_accuracies, large_accuracies]
v10_names = ['nano', 'small', 'medium', 'large']

plot_accuracy9(yolov10_results, v10_names, vertical_fov)


## YOLOv8

In [35]:
# -- vid1

# -- nano1

row16_nano1 = {'row1': [3, 8, 10, 11, 12, 12, 13, 14, 15, 15, 16, 16, 16, 17, 17, 17, 17, 18, 18, 19, 20, 20, 20, 21, 21, 21, 21, 21, 22, 23, 23],
        'row2': [1, 11, 12, 15, 16, 17, 17, 19, 20, 21, 22, 24, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29],
        'row3': [4, 20, 21, 23, 25, 25, 26, 27, 28, 28, 29, 29, 29, 30, 30, 30, 30, 30, 30, 31, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 35],
        'row4': [2, 23, 23, 24, 24, 26, 27, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 41, 42, 42, 42],
        'row5': [1, 18, 19, 19, 20, 21, 21, 21, 22, 22, 22, 24, 24, 25, 26, 26, 26, 28, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 31, 31, 31],
        'row6': [4, 17, 18, 18, 20, 21, 21, 23, 24, 25, 26, 28, 28, 28, 29, 29, 29, 29, 29, 31, 31, 31, 32, 32, 33, 33, 33, 33, 33, 34, 34],
        'row7': [3, 13, 15, 18, 20, 20, 22, 22, 23, 24, 24, 25, 25, 26, 26, 27, 28, 30, 30, 30, 31, 32, 32, 32, 32, 32, 32, 32, 33, 34, 34],
        'row8': [3, 10, 11, 12, 12, 13, 13, 13, 13, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 18, 18, 18, 19, 19, 19]}

# -- small 
row16_small1 = {'row1': [4, 10, 12, 12, 13, 14, 14, 15, 16, 16, 17, 17, 17, 18, 19, 19, 19, 20, 21, 21, 22, 22, 23, 24, 24, 25, 26, 26, 26, 27, 27],
        'row2': [3, 15, 16, 18, 19, 20, 20, 22, 22, 22, 23, 25, 26, 26, 27, 27, 28, 28, 29, 29, 29, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33],
        'row3': [3, 21, 22, 24, 27, 27, 27, 29, 30, 30, 30, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 34, 35, 35, 36, 36, 37, 38, 38, 38, 38],
        'row4': [3, 24, 25, 26, 26, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 43, 43, 43],
        'row5': [2, 20, 21, 21, 21, 22, 22, 22, 24, 24, 24, 26, 26, 27, 28, 28, 28, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33],
        'row6': [4, 19, 19, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 35, 35, 36, 36, 36, 36, 36],
        'row7': [6, 19, 20, 22, 23, 24, 26, 26, 28, 29, 29, 30, 30, 31, 31, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 38, 39, 39, 39],
        'row8': [6, 12, 12, 12, 13, 14, 15, 16, 16, 17, 17, 18, 18, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 22, 22, 22, 22, 22, 22]}

# - medium 

row16_medium1 = {'row1': [3, 9, 10, 11, 13, 14, 14, 15, 16, 16, 17, 17, 18, 20, 20, 21, 21, 22, 23, 23, 24, 24, 24, 26, 26, 26, 27, 27, 28, 28, 28],
        'row2': [4, 16, 17, 19, 20, 21, 21, 23, 23, 23, 24, 25, 25, 25, 26, 27, 28, 28, 29, 29, 29, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33],
        'row3': [2, 19, 22, 24, 26, 26, 27, 28, 29, 29, 29, 30, 30, 31, 31, 31, 31, 31, 31, 31, 33, 34, 34, 34, 35, 35, 35, 35, 36, 37, 37],
        'row4': [2, 24, 25, 26, 26, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 41, 42, 43, 43],
        'row5': [1, 20, 21, 22, 22, 23, 23, 23, 25, 25, 25, 27, 27, 28, 29, 29, 29, 31, 31, 31, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 34],
        'row6': [4, 19, 20, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 35, 36, 36, 37, 37, 37, 37],
        'row7': [8, 19, 22, 23, 24, 24, 26, 26, 28, 29, 29, 30, 30, 31, 31, 32, 33, 34, 34, 34, 35, 36, 36, 36, 37, 37, 37, 38, 39, 39, 39],
        'row8': [6, 13, 13, 13, 13, 14, 15, 16, 16, 17, 17, 18, 19, 20, 21, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 24, 24, 24, 24, 24, 24]}

# -- large

row16_large1={'row1': [4, 9, 12, 12, 13, 14, 14, 15, 16, 16, 17, 17, 18, 19, 21, 21, 21, 21, 22, 22, 23, 24, 24, 25, 25, 26, 26, 27, 27, 27, 27],
        'row2': [3, 15, 16, 19, 19, 20, 20, 22, 22, 22, 23, 24, 24, 24, 24, 25, 26, 27, 28, 28, 28, 28, 28, 29, 29, 30, 31, 31, 31, 32, 32],
        'row3': [4, 21, 23, 25, 27, 27, 27, 29, 30, 31, 31, 32, 32, 33, 33, 33, 33, 33, 33, 33, 35, 36, 36, 36, 37, 37, 37, 37, 38, 39, 39],
        'row4': [2, 24, 25, 26, 26, 27, 28, 29, 30, 30, 32, 34, 34, 35, 35, 36, 36, 36, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 43, 43, 43],
        'row5': [2, 20, 21, 21, 21, 22, 22, 22, 24, 24, 24, 26, 26, 27, 28, 28, 28, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33],
        'row6': [4, 19, 20, 20, 21, 22, 22, 24, 26, 26, 26, 29, 30, 30, 31, 32, 32, 32, 32, 33, 33, 33, 34, 34, 35, 36, 37, 37, 37, 37, 37],
        'row7': [9, 20, 21, 22, 23, 23, 25, 26, 28, 30, 30, 31, 31, 32, 32, 33, 34, 35, 35, 35, 36, 37, 37, 37, 38, 38, 38, 39, 40, 40, 40],
        'row8': [6, 13, 13, 13, 13, 15, 16, 17, 17, 18, 19, 20, 20, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 24, 24, 24, 24, 24, 24]}



In [36]:
# -- vid2

# -- nano2 
row16_nano2 = {'row1': [3, 7, 8, 8, 9, 10, 11, 12, 12, 14, 14, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20],
        'row2': [2, 7, 8, 10, 10, 10, 12, 14, 14, 14, 14, 15, 15, 16, 16, 16, 16, 16, 16, 17, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20],
        'row3': [5, 20, 22, 23, 25, 25, 28, 29, 29, 30, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 36, 36, 37, 37, 37, 38, 38],
        'row4': [2, 18, 20, 20, 21, 21, 21, 21, 22, 23, 24, 25, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 29, 30, 30],
        'row5': [4, 20, 21, 22, 23, 23, 25, 25, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 29, 29, 30, 30, 31, 32, 32, 32, 32, 32, 32, 32, 32],
        'row6': [3, 18, 21, 22, 23, 23, 24, 25, 27, 28, 28, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 35, 35, 35, 35, 35, 35],
        'row7': [2, 12, 13, 16, 16, 18, 18, 18, 19, 20, 21, 23, 23, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27],
        'row8': [4, 10, 11, 12, 13, 14, 14, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 20, 20, 21, 21, 21, 21, 21, 21, 21]}

row16_small2 = {'row1': [5, 10, 12, 13, 14, 14, 14, 14, 14, 16, 16, 18, 18, 19, 20, 20, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24],
        'row2': [5, 12, 13, 15, 16, 17, 19, 19, 20, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25],
        'row3': [6, 24, 25, 26, 26, 27, 29, 30, 30, 31, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 35, 36, 36, 37, 38, 38, 39, 39, 39, 40, 40],
        'row4': [3, 18, 19, 19, 20, 21, 21, 22, 22, 23, 24, 25, 25, 25, 25, 25, 26, 26, 26, 28, 28, 28, 29, 29, 29, 30, 31, 31, 31, 32, 32],
        'row5': [5, 20, 21, 22, 23, 23, 25, 25, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 31, 31, 31, 33, 33, 33, 33, 33],
        'row6': [3, 20, 21, 22, 23, 23, 26, 26, 27, 28, 28, 30, 30, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33, 33, 34, 35, 35, 35, 35, 36, 36],
        'row7': [1, 14, 16, 17, 17, 18, 19, 19, 19, 20, 21, 22, 23, 23, 23, 23, 24, 24, 25, 25, 26, 26, 26, 26, 26, 26, 26, 27, 28, 28, 28],
        'row8': [6, 10, 12, 14, 15, 16, 16, 16, 17, 18, 18, 20, 20, 20, 20, 21, 21, 21, 21, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]}

row16_medium2 = {'row1': [8, 13, 15, 15, 15, 17, 18, 18, 18, 20, 20, 22, 22, 22, 23, 23, 24, 24, 25, 25, 25, 25, 25, 26, 26, 27, 27, 27, 28, 28, 28],
        'row2': [5, 10, 12, 15, 17, 17, 20, 21, 22, 22, 23, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27],
        'row3': [7, 19, 22, 26, 26, 27, 29, 30, 30, 31, 33, 33, 33, 34, 35, 35, 36, 37, 37, 37, 37, 38, 38, 38, 39, 39, 40, 40, 40, 41, 41],
        'row4': [3, 19, 20, 20, 21, 21, 21, 22, 22, 23, 24, 25, 25, 25, 25, 25, 26, 26, 26, 28, 28, 28, 28, 28, 28, 29, 30, 30, 30, 31, 31],
        'row5': [5, 20, 21, 22, 23, 23, 25, 25, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 29, 30, 31, 31, 31, 33, 33, 33, 34, 34, 34, 34, 34],
        'row6': [3, 19, 21, 22, 23, 23, 26, 26, 27, 28, 28, 30, 30, 31, 31, 31, 31, 31, 31, 32, 33, 33, 33, 34, 34, 36, 36, 36, 36, 36, 36],
        'row7': [5, 15, 17, 18, 18, 20, 20, 20, 20, 21, 22, 23, 23, 23, 23, 24, 25, 25, 25, 25, 26, 26, 26, 26, 28, 28, 28, 29, 30, 30, 30],
        'row8': [8, 14, 14, 14, 15, 16, 16, 17, 18, 18, 18, 20, 20, 21, 21, 22, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25]}

row16_large2 = {'row1': [7, 14, 15, 15, 15, 16, 16, 16, 16, 19, 19, 20, 20, 20, 21, 22, 22, 22, 23, 24, 24, 25, 25, 26, 26, 27, 27, 27, 28, 28, 28],
        'row2': [3, 11, 13, 14, 15, 16, 18, 19, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23],
        'row3': [5, 18, 22, 24, 24, 25, 28, 29, 29, 30, 32, 32, 32, 33, 33, 33, 34, 35, 35, 35, 35, 36, 36, 36, 37, 37, 38, 38, 39, 39, 39],
        'row4': [2, 18, 19, 19, 20, 21, 21, 22, 22, 23, 23, 25, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 29, 30, 30],
        'row5': [5, 20, 21, 22, 23, 23, 25, 25, 25, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 29, 30, 30, 30, 32, 32, 32, 32, 32, 32, 33, 33],
        'row6': [4, 20, 21, 22, 23, 23, 26, 26, 27, 28, 28, 30, 30, 31, 31, 31, 31, 31, 31, 31, 32, 33, 33, 34, 34, 36, 36, 36, 36, 36, 36],
        'row7': [4, 13, 14, 15, 15, 17, 18, 18, 18, 19, 20, 21, 21, 21, 21, 22, 23, 23, 23, 23, 25, 25, 25, 25, 26, 26, 26, 27, 28, 28, 28],
        'row8': [6, 12, 13, 14, 15, 16, 16, 16, 16, 17, 17, 19, 19, 19, 19, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]}


In [37]:
# -- Accuracies

# -- gt
gtA = gen_gt_results('nano', 'A')
gtB = gen_gt_results('nano', 'B')

# -- nano
accuracy_results_nano1 = calculate_accuracies(gtA, row16_nano1) 
accuracy_results_nano2 = calculate_accuracies(gtB, row16_nano2) 
nano_accuracies = get_avg_accuracy(accuracy_results_nano1, accuracy_results_nano2)

# -- small
accuracy_results_small1 = calculate_accuracies(gtA, row16_small1) 
accuracy_results_small2 = calculate_accuracies(gtB, row16_small2) 
small_accuracies = get_avg_accuracy(accuracy_results_small1, accuracy_results_small2)

#-- medium
accuracy_results_medium1 = calculate_accuracies(gtA, row16_medium1) 
accuracy_results_medium2 = calculate_accuracies(gtB, row16_medium2) 
medium_accuracies = get_avg_accuracy(accuracy_results_medium1, accuracy_results_medium2)

# -- large
accuracy_results_large1 = calculate_accuracies(gtA, row16_large1) 
accuracy_results_large2 = calculate_accuracies(gtB, row16_large2) 
large_accuracies = get_avg_accuracy(accuracy_results_large1, accuracy_results_large2)


In [52]:
yolov8_results = [nano_accuracies, small_accuracies, medium_accuracies, large_accuracies]
v8_names = ['nano', 'small', 'medium', 'large']

plot_accuracy9(yolov8_results, v8_names, vertical_fov)


In [44]:
print(yolov8_results)
print(yolov9_results)
print(yolov10_results)

[{'accuracy1': [19.05, 46.11, 50.465, 51.61, 55.265, 56.58, 58.69, 59.42, 58.09, 57.175, 58.335, 57.0, 57.0, 57.69, 58.495, 57.285, 56.480000000000004, 58.335, 59.095, 59.69, 59.095, 60.61, 60.61, 61.81, 60.75, 58.965, 58.965, 58.025, 60.135, 60.635, 60.635], 'accuracy2': [10.715, 55.555, 58.335, 68.885, 67.10499999999999, 65.225, 67.75, 72.345, 72.645, 70.675, 72.76, 77.78, 77.78, 76.57, 75.66, 74.74000000000001, 74.815, 73.955, 75.805, 74.775, 78.125, 78.125, 78.125, 78.78999999999999, 78.78999999999999, 78.78999999999999, 78.78999999999999, 76.905, 76.905, 76.905, 76.905], 'accuracy3': [19.14, 81.94, 83.035, 88.985, 89.74000000000001, 85.355, 90.57499999999999, 88.94, 90.60499999999999, 88.01499999999999, 89.755, 88.555, 88.73, 87.765, 87.765, 86.705, 85.69999999999999, 85.93, 85.93, 87.445, 86.19, 87.845, 86.535, 86.535, 86.85499999999999, 85.945, 84.95, 86.27000000000001, 84.22999999999999, 85.29499999999999, 85.29499999999999], 'accuracy4': [9.43, 89.22999999999999, 91.85, 92.065

In [53]:

def plot_accuracy9(accuracy_results_list, model_names, vertical_fov):
    
    threshold = 95  # Set the threshold for accuracy
    fig, axs = plt.subplots(1, 8, figsize=(20, 5))  # Create 1 row and 8 columns of subplots

    # Define specific colors for each model
    colors = ['orange', 'blue', 'cyan', 'magenta', 'green']

    if len(model_names) > len(colors):
        raise ValueError("Not enough unique colors defined for the number of models.")

    for i in range(8):
        for accuracy_results, model_name, color in zip(accuracy_results_list, model_names, colors):
            accuracy_key = f'accuracy{i+1}'
            data_x = vertical_fov
            data_y = accuracy_results[accuracy_key]

            # Plot line segments individually
            for j in range(len(data_x) - 1):
                segment_color = 'red' if data_y[j] >= threshold else color
                axs[i].plot([data_x[j], data_x[j+1]], [data_y[j], data_y[j+1]], marker='o', markersize=.7, color=segment_color, linewidth=0.3, label=model_name if j == 0 else "")

            # Draw a horizontal line at the threshold value
            axs[i].axhline(y=threshold, color='red', linestyle='--', linewidth=.5)

        axs[i].set_title(f'Row{i+1} ({horz_va1[i]}°, {horz_va2[i]}°)', fontsize=10)  # Set the font size here
        axs[i].set_ylim(50, 105)
        axs[i].grid()

    axs[3].set_xlabel('Vertical FOV (degrees°)')      
    axs[0].set_ylabel('Accuracy')  

    # Create a legend for the models and place it above the subplots
    handles = [plt.Line2D([0], [0], color=color, lw=1) for color in colors[:len(model_names)]]
    labels = model_names

    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1), ncol=len(model_names))
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  # Adjust the top to make room for the legend
    plt.show()


In [60]:
plot_accuracy9(yolov8_results, v8_names, vertical_fov)


In [61]:

plot_accuracy9(yolov9_results, v9_names, vertical_fov)

In [62]:

plot_accuracy9(yolov10_results, v10_names, vertical_fov)

In [1]:

yolov8_results = [{'accuracy1': [19.05, 46.11, 50.465, 51.61, 55.265, 56.58, 58.69, 59.42, 58.09, 57.175, 58.335, 57.0, 57.0, 57.69, 58.495, 57.285, 56.480000000000004, 58.335, 59.095, 59.69, 59.095, 60.61, 60.61, 61.81, 60.75, 58.965, 58.965, 58.025, 60.135, 60.635, 60.635], 'accuracy2': [10.715, 55.555, 58.335, 68.885, 67.10499999999999, 65.225, 67.75, 72.345, 72.645, 70.675, 72.76, 77.78, 77.78, 76.57, 75.66, 74.74000000000001, 74.815, 73.955, 75.805, 74.775, 78.125, 78.125, 78.125, 78.78999999999999, 78.78999999999999, 78.78999999999999, 78.78999999999999, 76.905, 76.905, 76.905, 76.905], 'accuracy3': [19.14, 81.94, 83.035, 88.985, 89.74000000000001, 85.355, 90.57499999999999, 88.94, 90.60499999999999, 88.01499999999999, 89.755, 88.555, 88.73, 87.765, 87.765, 86.705, 85.69999999999999, 85.93, 85.93, 87.445, 86.19, 87.845, 86.535, 86.535, 86.85499999999999, 85.945, 84.95, 86.27000000000001, 84.22999999999999, 85.29499999999999, 85.29499999999999], 'accuracy4': [9.43, 89.22999999999999, 91.85, 92.065, 94.445, 98.15, 94.275, 97.725, 98.38499999999999, 94.78999999999999, 97.06, 100.0, 98.57, 98.61, 98.61, 96.725, 98.65, 98.65, 98.685, 96.935, 96.935, 95.0, 93.75, 93.78, 93.78, 93.97, 94.12, 94.12, 94.14500000000001, 94.28999999999999, 94.28999999999999], 'accuracy5': [12.629999999999999, 97.37, 97.725, 95.24000000000001, 95.535, 95.64, 93.75, 93.75, 95.83500000000001, 94.0, 87.14, 90.995, 90.995, 92.845, 91.38, 91.38, 91.495, 93.33, 91.935, 90.47, 90.615, 90.765, 92.28, 91.025, 91.025, 91.17, 89.9, 89.9, 90.035, 88.83500000000001, 88.83500000000001], 'accuracy6': [17.67, 85.645, 93.02, 90.68, 91.455, 91.955, 88.195, 94.21000000000001, 94.55, 96.35, 96.425, 95.15, 93.53999999999999, 93.63499999999999, 95.305, 93.745, 93.745, 92.49000000000001, 92.49000000000001, 94.155, 94.275, 94.275, 94.32, 94.43, 94.475, 92.28, 90.89500000000001, 88.52000000000001, 88.52000000000001, 88.63, 88.63], 'accuracy7': [12.7, 61.125, 63.75, 77.5, 78.46000000000001, 79.37, 80.195, 80.195, 77.92, 80.0, 78.89, 81.39500000000001, 78.715, 79.39500000000001, 79.39500000000001, 79.705, 80.325, 83.18, 81.99000000000001, 81.99000000000001, 82.515, 83.87, 82.72999999999999, 82.72999999999999, 81.81, 80.72999999999999, 80.72999999999999, 78.57, 77.745, 78.965, 78.965], 'accuracy8': [22.5, 62.5, 68.75, 72.795, 73.53, 75.0, 75.0, 74.21000000000001, 72.305, 71.36500000000001, 69.785, 70.41499999999999, 69.715, 65.3, 65.3, 63.395, 63.395, 65.48, 65.48, 66.095, 65.0, 63.980000000000004, 65.595, 66.25999999999999, 66.81, 65.155, 65.155, 65.155, 67.005, 66.065, 66.065]}, {'accuracy1': [28.175, 61.115, 66.875, 67.545, 71.05000000000001, 71.84, 68.33500000000001, 66.15, 64.36500000000001, 62.965, 63.985, 64.0, 64.0, 66.28, 68.8, 67.445, 67.995, 69.845, 72.225, 70.83500000000001, 70.0, 70.0, 71.67, 72.535, 71.325, 70.735, 72.25, 71.09, 71.57, 71.905, 71.905], 'accuracy2': [28.57, 84.52, 85.07, 91.49, 90.0, 88.63499999999999, 90.8, 89.58500000000001, 89.49000000000001, 86.22, 88.3, 88.655, 86.575, 87.285, 86.005, 84.74000000000001, 84.815, 83.63499999999999, 81.78, 84.155, 84.155, 84.155, 84.155, 84.64, 84.64, 82.91499999999999, 82.91499999999999, 82.38, 82.38, 80.715, 80.715], 'accuracy3': [18.685, 91.805, 90.48, 96.43, 91.41, 92.19, 93.94, 93.78999999999999, 95.455, 92.66999999999999, 92.71000000000001, 93.035, 91.86, 92.075, 92.075, 90.985, 89.95, 90.15, 90.15, 90.15, 88.81, 90.43, 90.47, 91.63499999999999, 91.83, 90.87, 91.075, 92.39, 90.21000000000001, 91.275, 91.275], 'accuracy4': [14.145, 91.155, 93.315, 93.39, 95.77, 100.0, 96.0, 100.0, 98.38499999999999, 94.78999999999999, 97.06, 100.0, 98.57, 98.61, 98.61, 96.725, 98.65, 98.65, 98.685, 98.72, 98.72, 96.66499999999999, 97.08500000000001, 97.11500000000001, 97.11500000000001, 97.195, 97.25, 98.44, 98.44, 98.485, 98.485], 'accuracy5': [17.765, 97.37, 92.46000000000001, 100.0, 97.91499999999999, 97.91499999999999, 95.83500000000001, 95.83500000000001, 100.0, 98.0, 90.985, 94.69999999999999, 94.69999999999999, 96.55, 94.82499999999999, 94.82499999999999, 94.94, 96.66499999999999, 93.54499999999999, 92.13499999999999, 92.32499999999999, 92.38, 92.38, 92.725, 92.725, 92.77, 94.32, 94.32, 94.36500000000001, 93.125, 93.125], 'accuracy6': [17.67, 95.455, 95.65, 95.445, 93.725, 94.22999999999999, 93.985, 98.15, 94.55, 98.275, 96.425, 96.875, 96.875, 96.97, 95.305, 95.35499999999999, 95.35499999999999, 94.1, 94.1, 94.155, 94.275, 94.275, 94.32, 93.08, 94.475, 92.28, 92.32, 92.685, 92.685, 92.55, 92.55], 'accuracy7': [17.065, 80.02000000000001, 81.66499999999999, 88.33500000000001, 86.72999999999999, 87.065, 89.61, 89.61, 86.25, 88.33500000000001, 87.225, 87.66999999999999, 86.53, 85.305, 85.305, 85.39500000000001, 85.855, 87.28, 87.545, 87.545, 87.92, 89.275, 87.995, 87.995, 86.92, 85.67, 85.67, 86.07, 86.45, 86.45, 86.45], 'accuracy8': [38.75, 68.75, 75.0, 78.675, 82.35499999999999, 83.33500000000001, 86.11, 82.10499999999999, 82.58, 83.41, 81.63, 86.66499999999999, 82.85499999999999, 79.765, 81.94, 79.16499999999999, 79.16499999999999, 79.16499999999999, 79.16499999999999, 81.32, 81.66499999999999, 80.375, 80.375, 80.71000000000001, 79.5, 77.105, 77.105, 77.105, 77.105, 76.035, 76.035]}, {'accuracy1': [32.935, 66.11, 68.885, 70.03, 73.685, 79.34, 77.85499999999999, 74.845, 72.36500000000001, 70.37, 71.13, 70.66499999999999, 72.66499999999999, 75.125, 75.555, 75.985, 76.39, 78.24000000000001, 80.475, 78.95, 77.88, 77.88, 77.88, 80.17, 78.86, 77.965, 79.47999999999999, 78.275, 80.065, 78.89, 78.89], 'accuracy2': [32.14, 80.16, 84.72, 94.12, 89.87, 86.13499999999999, 95.455, 91.475, 95.83500000000001, 90.225, 94.22999999999999, 92.36, 92.36, 92.85499999999999, 91.38, 88.075, 88.15, 86.86, 85.005, 87.275, 87.275, 88.84, 88.84, 89.185, 89.185, 87.46, 87.46, 85.235, 85.235, 83.57, 83.57], 'accuracy3': [18.22, 77.845, 85.12, 96.43, 93.33500000000001, 90.34, 93.94, 92.12, 93.78999999999999, 91.06, 91.14500000000001, 91.47, 90.295, 90.56, 91.84, 90.72, 90.87, 91.02, 91.02, 91.02, 91.195, 92.755, 91.405, 91.405, 91.61500000000001, 90.63, 89.535, 89.535, 88.71000000000001, 91.05000000000001, 91.05000000000001], 'accuracy4': [12.059999999999999, 93.655, 95.695, 95.77, 98.15, 100.0, 96.0, 100.0, 98.38499999999999, 94.78999999999999, 97.06, 100.0, 98.57, 98.61, 98.61, 96.725, 98.65, 98.65, 98.685, 98.72, 98.72, 96.66499999999999, 95.41499999999999, 95.445, 95.445, 95.58500000000001, 95.685, 95.685, 95.71000000000001, 96.97, 96.97], 'accuracy5': [15.129999999999999, 97.37, 92.46000000000001, 97.62, 95.535, 95.64, 97.91499999999999, 97.91499999999999, 97.91499999999999, 100.0, 92.905, 96.55, 96.55, 94.69999999999999, 96.55, 96.55, 96.66499999999999, 95.0, 96.775, 96.875, 96.97, 96.97, 96.97, 97.14500000000001, 97.14500000000001, 97.14500000000001, 97.22, 97.22, 97.22, 95.945, 95.945], 'accuracy6': [17.67, 93.18, 93.02, 95.445, 93.725, 94.22999999999999, 93.985, 98.15, 94.55, 98.275, 96.425, 96.875, 96.875, 96.97, 95.305, 95.35499999999999, 95.35499999999999, 92.66999999999999, 92.66999999999999, 94.155, 94.275, 94.275, 94.32, 94.43, 94.475, 92.06, 93.57, 92.50999999999999, 92.50999999999999, 93.9, 93.9], 'accuracy7': [32.94, 82.655, 88.33500000000001, 92.91499999999999, 91.155, 91.61, 91.88499999999999, 91.88499999999999, 88.33, 90.33500000000001, 89.075, 89.455, 86.53, 85.305, 85.305, 87.06, 87.47, 88.89500000000001, 87.545, 87.545, 87.92, 89.275, 87.995, 87.995, 89.86, 88.61, 88.61, 88.93, 89.225, 89.225, 89.225], 'accuracy8': [45.0, 84.375, 84.375, 81.8, 82.35499999999999, 83.33500000000001, 86.11, 84.60499999999999, 84.96, 83.41, 81.63, 86.66499999999999, 85.24000000000001, 83.865, 86.035, 83.035, 86.905, 86.905, 86.905, 87.215, 85.83500000000001, 84.545, 84.545, 84.71000000000001, 83.5, 80.81, 80.81, 80.81, 80.81, 81.21000000000001, 81.21000000000001]}, {'accuracy1': [33.730000000000004, 68.89, 74.77000000000001, 72.81, 73.685, 76.84, 73.095, 70.5, 68.36500000000001, 68.52000000000001, 69.345, 67.33500000000001, 69.33500000000001, 69.875, 74.255, 74.375, 73.265, 73.265, 75.59, 75.65, 74.7, 77.88, 77.88, 78.56, 77.295, 77.965, 77.965, 78.275, 78.595, 77.46000000000001, 77.46000000000001], 'accuracy2': [21.43, 80.94999999999999, 85.07, 91.175, 87.5, 86.36500000000001, 88.53, 89.58500000000001, 89.49000000000001, 84.295, 86.375, 88.89, 88.89, 85.5, 82.36, 83.075, 83.15, 83.87, 82.02, 84.375, 84.375, 84.375, 84.375, 84.85, 84.85, 83.125, 81.4, 81.19, 81.19, 79.52, 79.52], 'accuracy3': [19.14, 80.265, 87.19999999999999, 90.77, 88.075, 89.06, 92.425, 92.275, 93.94, 92.85499999999999, 92.88499999999999, 93.245, 92.10499999999999, 92.31, 92.31, 91.25, 91.465, 91.66499999999999, 91.66499999999999, 91.66499999999999, 91.66499999999999, 90.43, 91.86, 91.86, 92.045, 91.11, 89.99000000000001, 89.99000000000001, 90.21000000000001, 91.49000000000001, 91.49000000000001], 'accuracy4': [9.43, 91.155, 93.315, 93.39, 95.77, 100.0, 96.0, 100.0, 98.38499999999999, 94.78999999999999, 94.975, 100.0, 98.57, 98.61, 98.61, 96.725, 98.65, 98.65, 98.685, 96.935, 96.935, 95.0, 93.75, 93.78, 93.78, 93.97, 94.12, 95.31, 95.31, 95.455, 95.455], 'accuracy5': [17.765, 97.37, 92.46000000000001, 100.0, 97.91499999999999, 97.91499999999999, 95.83500000000001, 95.83500000000001, 100.0, 98.0, 90.985, 94.69999999999999, 94.69999999999999, 96.55, 94.82499999999999, 94.82499999999999, 94.94, 96.66499999999999, 93.54499999999999, 93.695, 95.455, 93.895, 93.895, 94.155, 94.155, 94.2, 92.93, 92.93, 92.975, 93.125, 93.125], 'accuracy6': [20.05, 95.455, 93.02, 95.445, 93.725, 94.22999999999999, 93.985, 98.15, 94.55, 98.275, 96.425, 96.875, 96.875, 96.97, 95.305, 95.35499999999999, 95.35499999999999, 92.66999999999999, 92.66999999999999, 92.725, 92.88499999999999, 94.275, 94.32, 94.43, 94.475, 92.06, 92.14500000000001, 92.50999999999999, 92.50999999999999, 93.9, 93.9], 'accuracy7': [32.54, 79.66499999999999, 78.75, 83.33500000000001, 81.72999999999999, 82.865, 85.555, 87.34, 84.16499999999999, 88.0, 87.035, 87.5, 84.645, 83.485, 83.485, 85.195, 85.66499999999999, 87.095, 85.705, 85.705, 87.71000000000001, 89.06, 87.745, 87.745, 88.235, 86.955, 86.955, 87.32, 87.67, 87.67, 87.67], 'accuracy8': [38.75, 78.125, 81.25, 81.8, 82.35499999999999, 86.11, 88.89, 84.735, 82.83, 83.63499999999999, 84.455, 89.58500000000001, 85.62, 82.19, 84.36500000000001, 81.55000000000001, 83.33500000000001, 83.33500000000001, 83.33500000000001, 82.03999999999999, 80.83500000000001, 79.705, 79.705, 79.87, 78.81, 76.265, 76.265, 76.265, 76.265, 75.325, 75.325]}]
yolov9_results = [{'accuracy1': [19.84, 46.665, 53.095, 54.239999999999995, 60.525000000000006, 59.34, 58.81, 59.63, 60.365, 61.34, 62.2, 59.0, 59.0, 59.615, 60.42, 60.99, 61.745000000000005, 63.595, 64.31, 62.99, 62.275, 62.275, 62.275, 63.425, 62.315, 61.995000000000005, 61.995000000000005, 63.865, 63.075, 63.495000000000005, 63.495000000000005], 'accuracy2': [17.86, 63.095, 65.97, 67.96, 70.0, 72.725, 72.62, 70.64, 70.74000000000001, 73.075, 72.91499999999999, 69.91, 67.825, 69.215, 70.295, 69.49000000000001, 69.63, 68.875, 67.025, 69.87, 69.87, 69.87, 69.87, 70.78999999999999, 70.78999999999999, 70.78999999999999, 70.78999999999999, 69.52, 69.52, 69.285, 69.285], 'accuracy3': [14.835, 80.265, 79.765, 85.715, 83.33500000000001, 84.09, 89.39500000000001, 87.57499999999999, 89.245, 88.205, 89.93, 90.33, 89.22999999999999, 88.22999999999999, 89.745, 88.75, 89.025, 89.285, 89.285, 87.77, 89.285, 88.10499999999999, 90.7, 89.31, 90.91, 90.0, 90.215, 90.215, 88.08, 88.08, 89.145], 'accuracy4': [13.600000000000001, 91.72999999999999, 93.775, 93.91499999999999, 96.295, 100.0, 96.0, 97.725, 98.38499999999999, 94.78999999999999, 97.06, 100.0, 98.57, 98.61, 98.61, 98.075, 100.0, 100.0, 100.0, 98.215, 98.215, 93.72, 95.0, 95.0, 95.0, 95.16, 95.31, 94.12, 94.14500000000001, 94.28999999999999, 94.28999999999999], 'accuracy5': [17.765, 97.37, 92.46000000000001, 100.0, 97.91499999999999, 97.91499999999999, 95.83500000000001, 95.83500000000001, 100.0, 96.075, 89.25999999999999, 92.97999999999999, 94.69999999999999, 96.55, 94.82499999999999, 94.82499999999999, 94.94, 96.66499999999999, 93.54499999999999, 92.13499999999999, 93.94, 92.38, 92.38, 92.725, 92.725, 92.77, 92.93, 94.445, 92.975, 93.125, 93.125], 'accuracy6': [20.305, 88.63499999999999, 90.85, 95.445, 93.725, 94.22999999999999, 92.13, 96.295, 94.55, 98.275, 96.425, 96.875, 96.875, 96.97, 95.305, 96.97, 96.97, 95.715, 95.715, 94.155, 92.71000000000001, 92.71000000000001, 92.805, 92.91499999999999, 93.005, 89.56, 89.64500000000001, 90.075, 90.075, 91.465, 91.465], 'accuracy7': [12.305, 62.31999999999999, 64.58500000000001, 68.75, 67.31, 71.505, 71.92, 71.92, 70.0, 69.0, 67.22, 70.16, 69.45, 67.275, 67.275, 67.455, 66.865, 68.47999999999999, 67.25, 70.25, 72.72, 74.07, 72.86, 72.86, 72.525, 71.34, 71.34, 70.895, 70.28999999999999, 70.28999999999999, 70.28999999999999], 'accuracy8': [22.915, 62.5, 65.625, 67.465, 67.65, 66.66499999999999, 75.0, 72.37, 71.17999999999999, 70.225, 69.24, 72.91499999999999, 69.62, 66.805, 70.9, 67.265, 67.265, 67.265, 67.265, 66.525, 65.83500000000001, 66.805, 66.805, 66.97, 66.31, 64.14, 65.655, 65.655, 65.655, 65.035, 65.035]}, {'accuracy1': [21.825, 66.67, 66.565, 67.545, 68.42, 69.34, 68.33500000000001, 66.15, 64.36500000000001, 64.815, 65.77, 62.335, 65.66499999999999, 67.95, 68.485, 70.905, 71.7, 73.55000000000001, 75.93, 75.92, 74.85, 74.85, 74.85, 75.62, 74.355, 73.68, 73.68, 73.99, 74.425, 74.72, 74.72], 'accuracy2': [25.0, 77.38, 78.82, 82.66499999999999, 79.87, 86.13499999999999, 88.42, 87.31, 89.49000000000001, 84.295, 86.375, 88.89, 86.805, 87.5, 86.205, 85.0, 85.0, 85.485, 83.63499999999999, 84.155, 84.155, 84.155, 84.155, 84.64, 84.64, 82.91499999999999, 82.91499999999999, 80.94999999999999, 80.94999999999999, 79.285, 79.285], 'accuracy3': [18.685, 86.285, 89.285, 92.56, 89.74000000000001, 90.625, 92.425, 92.275, 93.94, 91.24, 91.32, 91.685, 90.54499999999999, 90.795, 90.795, 89.735, 89.95, 90.15, 90.15, 90.15, 90.235, 91.86, 90.47, 91.86, 90.695, 89.75999999999999, 88.9, 91.305, 90.21000000000001, 90.21000000000001, 90.21000000000001], 'accuracy4': [16.23, 93.655, 93.775, 93.91499999999999, 96.295, 100.0, 96.0, 100.0, 98.38499999999999, 94.78999999999999, 97.06, 100.0, 98.57, 98.61, 98.61, 96.725, 98.65, 98.65, 98.685, 98.72, 98.72, 96.66499999999999, 95.41499999999999, 95.445, 95.445, 95.58500000000001, 96.875, 96.875, 96.875, 96.97, 96.97], 'accuracy5': [20.395, 97.37, 92.46000000000001, 100.0, 97.91499999999999, 97.91499999999999, 95.83500000000001, 95.83500000000001, 100.0, 98.0, 92.705, 94.69999999999999, 94.69999999999999, 96.55, 94.82499999999999, 94.82499999999999, 94.94, 96.66499999999999, 93.54499999999999, 92.13499999999999, 92.32499999999999, 92.38, 92.38, 92.725, 92.725, 92.77, 92.93, 92.93, 92.975, 91.775, 91.775], 'accuracy6': [15.54, 95.455, 93.02, 95.445, 93.725, 94.22999999999999, 93.985, 98.15, 94.55, 98.275, 96.425, 96.875, 96.875, 96.97, 95.305, 96.97, 96.97, 95.715, 94.1, 94.155, 94.275, 94.275, 94.32, 94.43, 94.475, 91.03, 91.07, 91.465, 90.075, 91.465, 91.465], 'accuracy7': [17.46, 77.75, 74.58500000000001, 79.16499999999999, 77.88499999999999, 83.22, 83.76499999999999, 83.76499999999999, 80.83500000000001, 83.0, 80.185, 80.875, 78.235, 77.275, 77.275, 77.455, 79.58500000000001, 81.015, 79.705, 79.705, 81.88499999999999, 83.235, 81.995, 81.995, 81.195, 78.695, 78.695, 79.10499999999999, 78.285, 78.285, 78.285], 'accuracy8': [26.04, 71.875, 71.875, 70.035, 70.59, 75.0, 77.775, 74.60499999999999, 75.44, 74.32, 77.285, 82.5, 78.85499999999999, 75.92, 78.095, 73.81, 73.81, 73.81, 73.81, 72.7, 71.66499999999999, 70.695, 72.31, 74.25999999999999, 73.25, 72.56, 72.56, 72.56, 72.56, 71.62, 71.62]}, {'accuracy1': [30.16, 69.445, 71.83, 75.44, 76.315, 76.84, 73.095, 70.5, 68.36500000000001, 68.52000000000001, 69.345, 67.33500000000001, 67.33500000000001, 73.46000000000001, 75.87, 76.225, 76.68, 78.535, 78.96000000000001, 77.435, 76.36500000000001, 76.36500000000001, 76.36500000000001, 77.09, 75.825, 75.11000000000001, 78.14, 78.32, 78.675, 77.5, 77.5], 'accuracy2': [28.575, 88.89, 90.97, 91.49, 90.0, 88.63499999999999, 93.18, 89.39, 93.75, 90.225, 92.31, 90.50999999999999, 88.43, 89.07, 87.73, 86.41, 86.485, 86.86, 85.005, 87.275, 88.84, 88.84, 87.055, 87.46, 87.46, 85.74, 85.74, 83.57, 83.57, 81.905, 81.905], 'accuracy3': [22.985, 84.36500000000001, 85.41499999999999, 85.41499999999999, 84.74000000000001, 87.5, 90.91, 93.78999999999999, 96.97, 95.715, 95.66, 97.295, 96.055, 96.155, 96.155, 95.0, 95.12, 95.24000000000001, 95.24000000000001, 93.725, 95.24000000000001, 93.92, 95.35, 95.35, 95.455, 95.555, 95.65, 94.33500000000001, 94.68, 94.68, 94.68], 'accuracy4': [9.43, 93.655, 93.775, 93.91499999999999, 96.295, 100.0, 96.0, 97.725, 98.38499999999999, 94.78999999999999, 97.06, 100.0, 98.57, 98.61, 98.61, 96.725, 98.65, 98.65, 98.685, 96.935, 96.935, 95.0, 95.41499999999999, 95.445, 95.445, 97.195, 98.44, 98.44, 98.44, 98.485, 98.485], 'accuracy5': [17.63, 100.0, 95.095, 100.0, 95.535, 95.64, 97.91499999999999, 97.91499999999999, 97.91499999999999, 100.0, 94.625, 96.55, 96.55, 94.69999999999999, 96.55, 96.55, 96.66499999999999, 95.0, 95.16, 95.31, 95.455, 95.455, 95.455, 95.715, 95.715, 95.715, 95.83500000000001, 95.83500000000001, 95.83500000000001, 94.595, 94.595], 'accuracy6': [17.92, 95.455, 93.02, 95.445, 93.725, 94.22999999999999, 92.13, 98.15, 94.55, 98.275, 96.425, 96.875, 96.875, 96.97, 95.305, 95.35499999999999, 95.35499999999999, 92.66999999999999, 92.66999999999999, 92.725, 92.88499999999999, 94.275, 94.32, 94.43, 94.475, 92.28, 95.0, 93.72999999999999, 93.72999999999999, 95.12, 95.12], 'accuracy7': [30.16, 80.02000000000001, 83.33500000000001, 90.41499999999999, 91.155, 91.61, 91.88499999999999, 93.67, 88.33, 90.33500000000001, 89.075, 89.455, 86.53, 86.97, 86.97, 87.06, 87.47, 88.89500000000001, 87.545, 87.545, 87.92, 89.275, 87.995, 87.995, 86.92, 85.67, 85.67, 86.07, 85.06, 85.06, 85.06], 'accuracy8': [45.0, 81.25, 81.25, 78.675, 82.35499999999999, 83.33500000000001, 83.33500000000001, 82.10499999999999, 82.58, 83.41, 83.805, 88.75, 84.85499999999999, 81.69, 83.865, 80.94999999999999, 82.735, 82.735, 82.735, 83.045, 81.66499999999999, 80.375, 80.375, 80.71000000000001, 81.06, 78.62, 78.62, 78.62, 78.62, 77.505, 77.505]}, {'accuracy1': [27.38, 72.775, 72.135, 72.95, 78.945, 79.475, 75.595, 72.88, 70.63499999999999, 70.6, 71.43, 73.0, 75.0, 77.05, 79.405, 77.83500000000001, 76.68, 78.535, 78.96000000000001, 77.435, 77.88, 79.545, 79.545, 80.17, 78.86, 78.055, 78.055, 78.32, 78.675, 78.93, 78.93], 'accuracy2': [21.43, 79.36500000000001, 93.75, 91.49, 89.87, 83.63499999999999, 90.8, 89.205, 91.575, 92.31, 90.225, 88.43, 86.345, 87.07, 85.81, 84.49000000000001, 84.63, 83.39, 81.53999999999999, 83.93, 83.93, 83.93, 83.93, 84.43, 84.43, 82.71000000000001, 80.985, 79.05000000000001, 79.05000000000001, 77.38, 77.38], 'accuracy3': [19.14, 84.36500000000001, 85.41499999999999, 88.985, 89.74000000000001, 92.19, 92.09, 93.78999999999999, 95.455, 94.285, 94.275, 94.595, 93.42, 92.31, 93.59, 92.5, 93.9, 94.05, 94.05, 94.05, 94.05, 92.755, 94.185, 94.185, 94.32, 93.33500000000001, 93.47999999999999, 92.16499999999999, 94.68, 94.68, 94.68], 'accuracy4': [9.43, 91.72999999999999, 93.775, 93.91499999999999, 96.295, 100.0, 97.725, 98.275, 100.0, 96.35499999999999, 98.53, 98.53, 100.0, 100.0, 100.0, 98.075, 100.0, 100.0, 100.0, 98.215, 98.215, 93.72, 95.0, 95.0, 95.0, 95.16, 95.31, 94.12, 95.71000000000001, 95.805, 95.805], 'accuracy5': [15.129999999999999, 97.37, 92.46000000000001, 100.0, 95.535, 95.64, 97.91499999999999, 97.91499999999999, 97.91499999999999, 100.0, 94.625, 96.55, 96.55, 94.69999999999999, 96.55, 96.55, 96.66499999999999, 95.0, 95.16, 95.31, 93.84, 96.97, 96.97, 97.14500000000001, 97.14500000000001, 97.14500000000001, 97.22, 97.22, 97.22, 95.945, 95.945], 'accuracy6': [15.285, 90.91, 93.02, 95.445, 93.725, 94.22999999999999, 92.13, 98.15, 94.55, 98.275, 98.15, 96.875, 96.875, 96.97, 96.97, 95.35499999999999, 95.35499999999999, 94.285, 92.66999999999999, 94.155, 94.275, 94.275, 94.32, 94.43, 94.475, 92.06, 93.57, 93.9, 93.9, 92.55, 92.55], 'accuracy7': [20.240000000000002, 83.01499999999999, 82.08500000000001, 86.66499999999999, 87.31, 89.685, 90.1, 90.1, 86.66499999999999, 90.33500000000001, 89.075, 89.455, 88.095, 85.305, 85.305, 87.06, 87.47, 90.505, 89.155, 89.155, 89.485, 90.84, 89.56, 89.56, 89.86, 88.61, 89.895, 88.93, 89.225, 89.225, 89.225], 'accuracy8': [41.665, 78.125, 78.125, 84.925, 85.29499999999999, 86.11, 88.89, 87.235, 85.21000000000001, 88.18, 86.305, 91.25, 87.24000000000001, 83.865, 86.035, 84.82, 84.82, 84.82, 84.82, 85.13, 85.41499999999999, 84.075, 84.075, 84.325, 83.06, 80.475, 82.325, 82.325, 82.325, 82.68, 82.68]}]
yolov10_results = [{'accuracy1': [2.78, 12.219999999999999, 28.17, 27.189999999999998, 34.215, 36.055, 39.165, 45.96, 45.545, 47.685, 52.980000000000004, 50.335, 52.0, 52.435, 53.349999999999994, 54.06, 53.355000000000004, 55.21, 57.910000000000004, 56.655, 56.06, 56.06, 56.06, 57.4, 57.81, 57.620000000000005, 57.620000000000005, 56.64, 55.964999999999996, 55.04, 55.04], 'accuracy2': [3.57, 35.715, 43.405, 48.915, 51.84, 53.18, 62.99, 63.635, 64.13, 62.66, 64.74, 65.97, 68.055, 65.43, 63.0, 62.31, 60.74, 60.095, 61.95, 59.82, 59.82, 59.82, 59.82, 59.56, 59.56, 59.56, 59.56, 58.81, 60.235, 60.235, 60.235], 'accuracy3': [8.605, 51.84, 55.06, 63.095, 67.82, 67.885, 68.685, 72.275, 75.455, 72.30499999999999, 71.53, 72.13, 74.015, 73.425, 73.425, 72.69, 71.99, 73.7, 73.7, 73.7, 72.615, 71.925, 72.125, 72.125, 71.435, 70.78, 70.365, 70.365, 69.75, 69.75, 69.75], 'accuracy4': [9.43, 48.075, 64.105, 67.19500000000001, 75.13, 81.215, 82.28999999999999, 84.01499999999999, 82.845, 80.21000000000001, 86.39500000000001, 86.12, 84.85499999999999, 86.445, 88.445, 88.82499999999999, 88.82499999999999, 88.82499999999999, 88.965, 87.725, 89.515, 87.82, 86.66499999999999, 86.785, 86.785, 85.565, 85.49000000000001, 85.49000000000001, 85.57, 84.39, 84.39], 'accuracy5': [10.129999999999999, 69.21000000000001, 78.47, 81.275, 84.825, 85.03999999999999, 89.75, 89.75, 89.75, 90.075, 87.14, 89.275, 90.995, 92.845, 91.38, 91.38, 89.83, 90.0, 87.1, 87.35, 86.07, 84.66, 84.66, 83.75, 83.75, 83.94, 85.60499999999999, 85.60499999999999, 87.255, 86.13, 86.13], 'accuracy6': [7.39, 61.84, 72.53999999999999, 70.705, 81.455, 84.265, 82.64, 84.725, 80.205, 82.16, 80.525, 79.04499999999999, 77.60499999999999, 78.03, 78.03, 76.69, 76.69, 76.04, 76.04, 76.34, 75.345, 75.345, 74.11500000000001, 76.04499999999999, 77.78, 76.39500000000001, 75.18, 73.205, 73.205, 74.685, 74.685], 'accuracy7': [4.76, 20.455, 32.085, 39.165, 41.345, 44.405, 47.565, 51.625, 53.33, 54.665, 56.855000000000004, 58.355000000000004, 56.465, 57.88, 57.88, 56.765, 56.5, 56.5, 55.465, 55.465, 55.235, 55.235, 55.834999999999994, 55.834999999999994, 54.644999999999996, 56.3, 56.3, 54.82, 53.42, 54.64, 54.64], 'accuracy8': [9.79, 28.125, 31.25, 34.19, 41.175000000000004, 44.445, 44.445, 44.21, 48.495000000000005, 51.135, 55.215, 59.165, 58.855, 56.69, 58.864999999999995, 57.735, 57.735, 57.735, 59.519999999999996, 58.905, 58.33, 57.795, 57.795, 58.129999999999995, 57.625, 54.04, 55.555, 57.405, 57.405, 56.915000000000006, 56.915000000000006]}, {'accuracy1': [19.05, 52.78, 61.61, 67.69, 71.05000000000001, 71.975, 70.83500000000001, 68.53, 66.63499999999999, 65.045, 66.07, 62.665, 64.33500000000001, 64.875, 67.49000000000001, 66.065, 65.16, 67.015, 69.53, 68.07499999999999, 68.64, 68.64, 68.64, 68.12, 66.91, 67.88, 67.88, 68.195, 67.32, 69.09, 69.09], 'accuracy2': [17.86, 55.555, 69.78999999999999, 85.60499999999999, 87.37, 83.86500000000001, 81.605, 85.41499999999999, 83.24000000000001, 82.375, 84.455, 87.035, 84.94999999999999, 87.5, 86.205, 85.0, 85.0, 83.87, 82.02, 82.81, 82.81, 82.81, 84.375, 84.85, 84.85, 83.125, 83.125, 84.285, 84.285, 80.94999999999999, 80.94999999999999], 'accuracy3': [19.14, 77.845, 86.60499999999999, 94.64500000000001, 93.33500000000001, 90.34, 93.94, 92.12, 93.78999999999999, 92.49000000000001, 92.53, 92.82, 91.61, 91.84, 91.84, 90.72, 90.87, 89.82499999999999, 89.82499999999999, 91.34, 91.195, 92.755, 91.405, 91.405, 91.61500000000001, 91.74000000000001, 91.935, 93.25, 91.05000000000001, 92.11500000000001, 92.11500000000001], 'accuracy4': [4.715, 71.53999999999999, 84.25, 89.15, 93.91499999999999, 100.0, 96.0, 97.725, 98.38499999999999, 94.78999999999999, 95.59, 98.53, 97.14500000000001, 97.22, 97.22, 95.37, 97.295, 97.295, 97.37, 97.435, 97.435, 95.38499999999999, 95.83500000000001, 95.89500000000001, 95.89500000000001, 96.005, 96.06, 97.25, 97.275, 97.32, 97.32], 'accuracy5': [12.629999999999999, 92.5, 90.55, 95.345, 95.83500000000001, 95.83500000000001, 95.83500000000001, 95.83500000000001, 100.0, 98.0, 94.625, 96.55, 96.55, 94.69999999999999, 96.55, 96.55, 95.0, 93.33500000000001, 93.55, 95.31, 95.455, 95.455, 95.455, 94.285, 94.285, 94.285, 95.83500000000001, 95.83500000000001, 95.83500000000001, 94.595, 94.595], 'accuracy6': [15.54, 88.63499999999999, 90.85, 93.27, 91.725, 92.31, 92.13, 98.15, 94.55, 98.275, 96.425, 96.875, 96.875, 96.97, 95.305, 96.97, 96.97, 94.285, 94.285, 94.285, 94.445, 94.445, 94.445, 95.945, 95.945, 95.0, 95.0, 92.50999999999999, 92.50999999999999, 91.195, 91.195], 'accuracy7': [15.08, 67.945, 70.0, 76.66499999999999, 77.31, 78.32, 85.555, 85.555, 80.83500000000001, 87.0, 85.74000000000001, 89.805, 86.85499999999999, 87.275, 87.275, 87.455, 87.835, 89.265, 87.995, 87.995, 85.435, 86.785, 88.485, 88.485, 88.7, 87.515, 87.515, 86.60499999999999, 85.565, 85.565, 85.565], 'accuracy8': [29.375, 62.5, 75.0, 72.795, 76.47, 72.22, 77.78, 82.10499999999999, 82.58, 81.13499999999999, 81.955, 87.08500000000001, 83.24000000000001, 80.02, 82.19, 79.465, 79.465, 81.25, 81.25, 81.68, 82.08500000000001, 80.845, 80.845, 81.095, 79.94, 77.445, 77.445, 77.445, 77.445, 76.42, 76.42]}, {'accuracy1': [27.38, 60.555, 66.565, 70.17500000000001, 68.42, 69.21000000000001, 68.33500000000001, 68.325, 66.36500000000001, 64.815, 65.77, 64.0, 64.0, 66.28, 68.8, 70.905, 71.41, 73.265, 73.74000000000001, 74.13499999999999, 73.185, 73.185, 73.185, 72.535, 71.325, 70.735, 70.735, 73.95, 72.955, 74.68, 74.68], 'accuracy2': [14.285, 69.05000000000001, 79.16499999999999, 92.10499999999999, 84.60499999999999, 83.63499999999999, 88.53, 93.75, 91.575, 88.14500000000001, 90.225, 92.595, 90.50999999999999, 91.07, 89.655, 88.33500000000001, 88.33500000000001, 88.71000000000001, 86.86, 87.5, 89.06, 89.06, 90.625, 90.91, 90.91, 90.91, 90.91, 88.33500000000001, 88.33500000000001, 86.66499999999999, 86.66499999999999], 'accuracy3': [18.685, 73.995, 79.465, 92.56, 91.41, 90.045, 91.75, 91.97, 93.63499999999999, 92.49000000000001, 92.53, 94.17, 94.245, 93.125, 94.405, 93.22, 92.09, 92.21000000000001, 92.21000000000001, 94.91499999999999, 93.575, 95.08, 93.72999999999999, 93.72999999999999, 93.885, 93.96000000000001, 94.11, 95.42500000000001, 93.18, 93.18, 93.18], 'accuracy4': [11.515, 91.155, 93.315, 95.77, 98.15, 100.0, 96.0, 100.0, 98.38499999999999, 94.78999999999999, 97.06, 100.0, 98.57, 97.22, 97.22, 95.37, 98.65, 98.65, 98.685, 98.72, 98.72, 96.66499999999999, 95.41499999999999, 95.445, 95.445, 95.58500000000001, 97.25, 98.44, 98.44, 97.32, 97.32], 'accuracy5': [15.265, 95.0, 92.82499999999999, 97.725, 97.91499999999999, 97.91499999999999, 95.83500000000001, 95.83500000000001, 100.0, 98.0, 92.705, 94.69999999999999, 94.69999999999999, 96.55, 94.82499999999999, 94.82499999999999, 93.275, 95.0, 91.935, 93.695, 93.84, 93.895, 93.895, 94.155, 94.155, 94.2, 94.32, 94.32, 94.36500000000001, 94.475, 94.475], 'accuracy6': [17.67, 88.28, 93.02, 95.445, 93.725, 96.155, 93.985, 98.15, 94.55, 98.275, 96.425, 96.875, 96.875, 95.455, 93.78999999999999, 93.84, 95.35499999999999, 92.66999999999999, 92.66999999999999, 92.725, 92.88499999999999, 92.88499999999999, 94.32, 94.43, 94.475, 93.53, 93.57, 93.9, 93.9, 92.55, 92.55], 'accuracy7': [15.08, 70.215, 74.58500000000001, 83.75, 84.81, 85.49000000000001, 86.035, 87.825, 84.58500000000001, 86.66499999999999, 87.405, 91.24, 88.255, 88.63499999999999, 88.63499999999999, 88.725, 89.08000000000001, 90.505, 89.155, 89.155, 91.045, 92.4, 91.12, 91.12, 91.33000000000001, 90.08000000000001, 90.08000000000001, 90.35499999999999, 90.61500000000001, 90.61500000000001, 90.61500000000001], 'accuracy8': [32.085, 78.125, 78.125, 81.8, 82.35499999999999, 80.555, 80.555, 84.60499999999999, 82.58, 83.41, 84.13, 89.16499999999999, 85.24000000000001, 81.94, 84.11, 81.25, 81.25, 83.035, 83.035, 85.13, 83.75, 82.46000000000001, 82.46000000000001, 82.71000000000001, 81.5, 78.96000000000001, 78.96000000000001, 80.475, 80.475, 80.83, 80.83]}, {'accuracy1': [33.730000000000004, 68.89, 74.77000000000001, 72.81, 73.685, 76.84, 73.095, 70.5, 68.36500000000001, 68.52000000000001, 69.345, 67.33500000000001, 69.33500000000001, 69.875, 74.255, 74.375, 73.265, 73.265, 75.59, 75.65, 74.7, 77.88, 77.88, 78.56, 77.295, 77.965, 77.965, 78.275, 78.595, 77.46000000000001, 77.46000000000001], 'accuracy2': [21.43, 80.94999999999999, 85.07, 91.175, 87.5, 86.36500000000001, 88.53, 89.58500000000001, 89.49000000000001, 84.295, 86.375, 88.89, 88.89, 85.5, 82.36, 83.075, 83.15, 83.87, 82.02, 84.375, 84.375, 84.375, 84.375, 84.85, 84.85, 83.125, 81.4, 81.19, 81.19, 79.52, 79.52], 'accuracy3': [19.14, 80.265, 87.19999999999999, 90.77, 88.075, 89.06, 92.425, 92.275, 93.94, 92.85499999999999, 92.88499999999999, 93.245, 92.10499999999999, 92.31, 92.31, 91.25, 91.465, 91.66499999999999, 91.66499999999999, 91.66499999999999, 91.66499999999999, 90.43, 91.86, 91.86, 92.045, 91.11, 89.99000000000001, 89.99000000000001, 90.21000000000001, 91.49000000000001, 91.49000000000001], 'accuracy4': [9.43, 91.155, 93.315, 93.39, 95.77, 100.0, 96.0, 100.0, 98.38499999999999, 94.78999999999999, 94.975, 100.0, 98.57, 98.61, 98.61, 96.725, 98.65, 98.65, 98.685, 96.935, 96.935, 95.0, 93.75, 93.78, 93.78, 93.97, 94.12, 95.31, 95.31, 95.455, 95.455], 'accuracy5': [17.765, 97.37, 92.46000000000001, 100.0, 97.91499999999999, 97.91499999999999, 95.83500000000001, 95.83500000000001, 100.0, 98.0, 90.985, 94.69999999999999, 94.69999999999999, 96.55, 94.82499999999999, 94.82499999999999, 94.94, 96.66499999999999, 93.54499999999999, 93.695, 95.455, 93.895, 93.895, 94.155, 94.155, 94.2, 92.93, 92.93, 92.975, 93.125, 93.125], 'accuracy6': [20.05, 95.455, 93.02, 95.445, 93.725, 94.22999999999999, 93.985, 98.15, 94.55, 98.275, 96.425, 96.875, 96.875, 96.97, 95.305, 95.35499999999999, 95.35499999999999, 92.66999999999999, 92.66999999999999, 92.725, 92.88499999999999, 94.275, 94.32, 94.43, 94.475, 92.06, 92.14500000000001, 92.50999999999999, 92.50999999999999, 93.9, 93.9], 'accuracy7': [32.54, 79.66499999999999, 78.75, 83.33500000000001, 81.72999999999999, 82.865, 85.555, 87.34, 84.16499999999999, 88.0, 87.035, 87.5, 84.645, 83.485, 83.485, 85.195, 85.66499999999999, 87.095, 85.705, 85.705, 87.71000000000001, 89.06, 87.745, 87.745, 88.235, 86.955, 86.955, 87.32, 87.67, 87.67, 87.67], 'accuracy8': [38.75, 78.125, 81.25, 81.8, 82.35499999999999, 86.11, 88.89, 84.735, 82.83, 83.63499999999999, 84.455, 89.58500000000001, 85.62, 82.19, 84.36500000000001, 81.55000000000001, 83.33500000000001, 83.33500000000001, 83.33500000000001, 82.03999999999999, 80.83500000000001, 79.705, 79.705, 79.87, 78.81, 76.265, 76.265, 76.265, 76.265, 75.325, 75.325]}]
